## Arrays & Hashing

### Medium

[665. Non-decreasing Array](https://leetcode.com/problems/non-decreasing-array/)  

`nums[i] > nums[i + 1]`, fix it:
- If `nums[i - 1] > nums[i + 1]`,  increase `nums[i + 1]` to keep the array consistent.
- else decrease `nums[i]`. (no need bc no comparison made with i anymore)

#2,5,4, 4 ->#2-4-44  
#2,5,4, 5 -> 2-4-45  
#423 ->223
#if i+1 < i-1 -> i+1 = i
but 3425 -> 3-2-25, not right, 34-4-5 right ( 3225)

In [ ]:
def checkPossibility(nums: list[int]) -> bool:
    m = 0 #number to modify
    for i in range(len(nums)-1):
        if nums[i] > nums[i+1]:
            if m == 1: 
                return False
            m+=1
            if nums[i+1] < nums[i-1] and i>0:
                nums[i+1] = nums[i]

    return m<=1

checkPossibility([3,4,2,3])
checkPossibility([1,4,1,2])

[1, 1, 1, 2]


True

[49. Group Anagrams](https://leetcode.com/problems/group-anagrams/)

In [ ]:
from collections import defaultdict
def groupAnagrams(strs: list[str]) -> list[list[str]]:
    res = defaultdict(list)
    for s in strs:
        count = [0] * 26
        for c in s:
            count[ord(c) - ord('a')] += 1
        res[tuple(count)].append(s)
    return list(res.values())
'''
Time : O(m∗n) - Space : O(m)
 m is the number of strings and n is length of longest string.
'''


[5. Longest Palindromic Substring](https://leetcode.com/problems/longest-palindromic-substring/?envType=featured-list&envId=top-interview-questions?envType=featured-list&envId=top-interview-questions)


Manacher most effiencient

In **Manacher’s Algorithm**, the step

```python
if i < R:
    p[i] = min(R - i, p[mirr])
```

ensures we have a **good initial radius** for expanding the palindrome at index `i` without **overestimating**. The rationale can be understood by focusing on two key ideas:

1. **Using the Mirror’s Radius**: We reuse information about a palindrome that is symmetric (mirrored) around the current center.
2. **Not Exceeding the Known Boundary**: We cannot trust a radius larger than `R - i` because we only know for sure that the palindrome is valid **inside** the current right boundary (`R`).

Let’s walk through **why** this formula works, step by step, and illustrate it with a simple scenario.

---

 The Manacher’s Setup

- We have a **current palindrome** with **center** = `C` and a **right boundary** = `R`.  
- We are processing a new position `i` such that `i < R` (i.e., **inside** that known palindrome).  
- We define the **mirror** of `i` around `C` as `mirr = 2*C - i`. 

In simpler terms, `mirr` is the **equidistant position** on the left side of `C` that “matches up” with `i` on the right side.

---

 Two Main Cases

When we say

```python
p[i] = min(R - i, p[mirr])
```
we’re combining two considerations:

1. **`p[mirr]`**: The palindrome radius at the mirror index might give a clue how large the palindrome at `i` can be without further checks.  
2. **`R - i`**: We can’t exceed the known boundary `R`. If `p[mirr]` is bigger than `(R - i)`, that portion would fall outside the current palindrome’s boundary, and we don’t yet know if the palindrome at `i` extends beyond `R`.

Hence we pick the smaller of these two as an **initial** radius. If the palindrome truly extends further, we’ll find out in the subsequent expansion step:

```python
while t[i + p[i] + 1] == t[i - p[i] - 1]:
    p[i] += 1
```

---

 Case A: `p[mirr] <= R - i`

- The palindrome at the mirror doesn’t exceed the left boundary of the big palindrome.  
- We can safely **copy** `p[mirr]` over to `p[i]` because we know the palindrome at `i` can at least be that large—no boundary conflict arises.

**Example Visual**  
```
      C             R
...|---|-----|-------|...
      |   i   |
      | mirr  |
      
If p[mirr] is small enough that it fits wholly inside the boundary,
we just set p[i] = p[mirr].
```

 Case B: `p[mirr] > R - i`

- The palindrome at the mirror **would cross** the boundary if we blindly copied it to `p[i]`.  
- We must limit `p[i]` to `R - i`. This is all we **know** for certain without expanding further because it doesn’t exceed the known right boundary.

**Example Visual**  
```
          C        R
...|-------|-------|...
         i    ?

If p[mirr] suggests an even bigger palindrome,
we still can’t trust it beyond R, so we set p[i] = R - i.
```

---

Concrete Example

Let’s take a **transformed** string (as Manacher’s uses extra characters like `#`) for `"aba"`, say `^#a#b#a#$`. Suppose:

- `C = 4` (around `#b#`),  
- `R = 6`, meaning we know the palindrome extends to position `6`.

Now we’re calculating for `i = 5`, which is inside the boundary (`5 < R`):

1. `mirr = 2*C - i = 2*4 - 5 = 3`.  
2. Suppose `p[mirr] = 3`, but `(R - i) = 1`.  
   - This means if we blindly set `p[i] = p[mirr] = 3`, we’d be claiming a palindrome extends to `i + 3 = 8`, beyond our known boundary `6`.  
   - So we must do `p[i] = min(p[mirr], R - i) = min(3, 1) = 1`.  
   - After that, we attempt to expand further if the characters still match.

This approach **never overestimates** the radius, ensuring correctness. If in reality the palindrome does extend beyond the boundary, the **while loop** expansion will catch that. Otherwise, we keep the valid radius limited by `(R - i)`.

---

- **No Over-Expansion**: We never assume beyond what’s known from symmetry plus the boundary.  
- **Reuse of Computed Palindromes**: We gain efficiency because we don’t re-check everything from scratch.  
- **Subsequent Expansion**: If the palindrome is indeed larger, the next expansions step will increment `p[i]` further.

Hence, `p[i] = min(R - i, p[mirr])` is the perfect initial guess for the palindrome radius at `i`, ensuring we remain within the known boundary while leveraging mirrored data from `mirr`.

---

In Summary

1. **Mirror** (`mirr`) tells us how big the palindrome was on the symmetric left side.  
2. **`R - i`** ensures we don’t cross the known right boundary.  
3. The **minimum** of these two values is used as a **starting radius** (not the final). We expand afterwards if possible.  

That’s how **Manacher’s** obtains a **linear-time** palindrome check without redoing expansions that were already implied by the mirrored position.

In [ ]:
def longestPalindrome( s: str) -> str:
      # Preprocess the string with special characters
    t = '#'.join('^{}$'.format(s))
    n = len(t)
    p = [0] * n
    C = R = 0
    maxLen, maxCenter = 0, 0
    for i in range(1, n - 1):

        mirr = 2 * C - i
        #This part starts working when C <= i < R,
        # get p[i] use the mirror trick
        if i < R:
            p[i] = min(R - i, p[mirr])
        
        #2. Keep expand around i to check palindrome
        while t[i + p[i] + 1] == t[i - p[i] - 1]:
            p[i] += 1
        #3. Extend R and C as p[i] has Pal value
        if i + p[i] > R:
            C, R = i, i + p[i]
        
        #update maxLen
        if maxLen < p[i]:
            maxLen = p[i]
            maxCenter = i
    return s[(maxCenter - maxLen) // 2: (maxCenter + maxLen) // 2]

Fastest Solution: only check next longer pal. 
- Loop through the string, r is bounded at index (r = i + 1)
    - odd length check:
        l = r - (res_len + 2)
        
    

In [1]:
def longestPalindrome(self, s: str) -> str:
    if s == s[::-1]:
        return s

    res_L = 0
    res_len = 1

    for i in range(1, len(s)):
        r  = i + 1

        # Odd length palindrome
        l = r - (res_len+2)
        substring = s[l:r]
        # l can be negative, so check if it is >= 0
        if l >= 0 and substring == substring[::-1]:
            res_L = l
            res_len = len(substring)
            # The even wil be shorter, so no need to compute it
            continue

        # Even length palindrome
        l = r - (res_len+1)
        substring = s[l:r]
        # l can be negative, so check if it is >= 0
        if l >= 0 and substring == substring[::-1]:
            res_L = l
            res_len = len(substring)
            
    return s[res_L:res_L + res_len]
longestPalindrome("abaxabaxabb")


'baxabaxab'

- Loop through the whole strings. check palin that going outwards at each element.
- At each element, expand in 2 ways to check odd length pal, and even length. 
    - To check odd length, starts R = 1
    - To check even length, R = L + 1

In [2]:
#Neetcode, not optimal
def longestPalindrome(s: str) -> str:
        #for each letter, expands outwards        
        resL, resR = 0,0

        for i in range(len(s)):
            #odd length
            l,r = i,i
            while l >= 0 and r < len(s) and s[l] == s[r]:
                #update before moving outwards
                if max(resR - resL + 1, r - l + 1 ) == r - l + 1:
                    resR, resL = r, l
                l-=1
                r+=1
            #even length
            l, r = i, i+1
            #exact same move outward code like above
            while l >= 0 and r < len(s) and s[l] == s[r]:
                if max(resR - resL + 1, r - l + 1) == r - l +1 :
                    resR, resL = r, l
                l -=1
                r +=1
        return s[resL: resR + 1]
s = "babad"
longestPalindrome(s)

'aba'

Longest Palindrome with lexicographical smallest

In [ ]:
def longestPalindrome(s):
  # Preprocess the string to handle both even and odd-length palindromes
    t = '#'.join('^{}$'.format(s))  # Add boundaries and separators
    n = len(t)
    p = [0] * n  # Array to store palindrome radii
    center = right = 0  # Center and right boundary of the current longest palindrome
    max_len = 0  # Maximum length of palindrome found
    longest_palindrome = ""  # Longest palindrome found so far

    for i in range(1, n - 1):
        # Mirror of the current position `i` with respect to center `C`
        mirror = 2 * center - i
        
        # Use previously computed palindrome length if within boundary
        if i < right:
            p[i] = min(right - i, p[mirror])
        
        # Expand around center `i`
        while t[i + p[i] + 1] == t[i - p[i] - 1]:
            p[i] += 1
        
        # Update center and right boundary if needed
        if i + p[i] > right:
            center, right = i, i + p[i]
        
        # Calculate the start and end indices of the palindrome in the original string
        length = p[i]
        start = (i - length) // 2
        end = start + length

        # Check if the new palindrome is longer or lexicographically smaller
        current_palindrome = s[start:end]
        if length > max_len or (length == max_len and current_palindrome < longest_palindrome):
            max_len = length
            longest_palindrome = current_palindrome

    return longest_palindrome

s = "abacdfgdcabba"
print(longestPalindrome(s))  # Output: "abba"


[36. Valid Sudoku](https://leetcode.com/problems/valid-sudoku/)- defaultdict(set)

In [54]:
from collections import defaultdict 
#36. Valid Sudoku
def isValidSudoku(board: list[list[str]]) -> bool:
    #Idea: check if each small square belongs to coordinated-associated rows, cols, and sqr 
    #Implement: create hashset (dict of sets) as cols, rows and sqr
    #each row, col, sqr is a set
    #Form of each dict(set): e.g: rows = {0: {set}, 1:{set},...}
    cols = defaultdict(set) 
    rows = defaultdict(set)
    sqr = defaultdict(set)

    for r in range(9):
            for c in range(9):
                if board[r][c] == ".":
                    continue
                if (
                    board[r][c] in rows[r]
                    or board[r][c] in cols[c]
                    or board[r][c] in sqr[(r // 3, c // 3)]
                ):
                    return False
                cols[c].add(board[r][c])
                rows[r].add(board[r][c])
                sqr[(r // 3, c // 3)].add(board[r][c])

    return True

[238. Product of Array Except Self (2) - Only pos var](https://leetcode.com/problems/product-of-array-except-self/submissions/1089079628/)

In [ ]:
def productExceptSelf(nums: list[int]) -> list[int]:
    #Create an preArr stores prefix product up to + (exclude) nums[i]
    #Loop from the end of preArr: at each preArr[i], multiply by current postfix product var

    #Time: O(n), Space: O(n)
    out = [1] * len(nums)
    
    #Create an arr stores prefix product up to nums[i] + (exclude) nums[i]
    for i in range(1, len(nums)):
        out[i] = out[i-1] * nums[i-1]
    #init first and last in output
    pos = nums[-1]
    for i in range(len(nums)-2, -1, -1):
        out[i] = out[i] * pos  #upto prefix * upto postfix 
        pos *= nums[i]

    return out
nums = [1,2,3,4]
productExceptSelf(nums)

[24, 12, 8, 6]

[238. Product of Array Except Self - Pre and pos as arrays](https://leetcode.com/problems/product-of-array-except-self/submissions/1089034138/)

In [3]:
def productExceptSelf(nums: list[int]) -> list[int]:
    #Create 2 arrays: prefix product and postfix product.
    #Output[i] = prefix[i-1] * postfix [i+1]
    #Time: O(n), Space: O(3n)
    l = len(nums)
    pre, pos= [nums[0]]*l, [nums[l-1]]*l
    j = l - 2
    for i in range(1, l):
        pre[i] = pre[i-1] * nums[i]
        pos[j] = pos[j+1] * nums[j]
        j-=1
    #init first and last in output
    out = [pos[1]]*l
    out[l-1] = pre[l-2]

    for i in range(1,l-1):
        out[i] = pre[i-1]*pos[i+1]

    print(pre, pos, out)
    return out

nums = [1,2,3,4]
productExceptSelf(nums)

[1, 2, 6, 24] [24, 24, 12, 4] [24, 12, 8, 6]


[24, 12, 8, 6]

[128. Longest Consecutive Sequence. Original](https://leetcode.com/problems/longest-consecutive-sequence/)

If a number is part of a sequence, it'll visited at most twice. If a number is only a sequence of itself, it'll be visited once. So time complexity = O(2n) = O(n).
Use a set

In [60]:
def longestConsecutive(nums: list[int]) -> int:
    numSet = set(nums)
    longest = 0
    for num in numSet:  
        #check if num is the start of the seq:
        if (num - 1) not in numSet:
            len = 0
            while (num+len) in numSet:
                len+=1
            longest = max(longest,len)

    return longest
nums = [9,1,4,7,3,-1,0,5,8,-1,6]
longestConsecutive(nums)


21


7

[347. Top K Frequent Elements ](https://leetcode.com/problems/top-k-frequent-elements/submissions/)

Bucket sort - Space: Linear, Time: Linear


In [68]:
def topKFrequent(nums: list[int], k: int) -> list[int]:
#bucket sort: create an array where index is the freqs, value is list of nums associating with the freqs
# Space: O(N). Size of bucket always <= len(nums) because max freq of any num is len(nums).
    count = {} # count dict
    
    buckets = [[] for i in range (len(nums)+1)] #freqs start from 1 to including len(nums)

    #manually count instead of collections.Counter(nums)
    for n in nums:
        count[n] = 1 + count.get(n, 0) #count.get(n, 0) means count[n] = 0 if n has not exist in dict
    
    #bucket's index is the freqs, value is list of nums associating with the freqs
    for num,freq in count.items():
        
        buckets[freq].append(num)
    res = []
    for i in range(len(buckets)-1, 0, -1): #start from last index
        if len(buckets[i]) == 0:
            continue
        for num in buckets[i]:
            res.append(num)
            if len(res) == k:
                return res
    return res
nums = [1,1,1,2,2,3]
k = 2
topKFrequent(nums,k)

[1, 2]

Heap - O(KlogN)

In [66]:
from collections import Counter
import heapq 

def topKFrequent(nums: list[int], k: int) -> list[int]:
    count = Counter(nums)
    
    #create a list of tuples (fre, num) for heapifying (we maintain heap by values)
    heap = [(fre, num) for (num, fre) in count.items() ]
    heapq._heapify_max(heap)
    print(heap)

    mlist = []
    for i in range(k):
        mlist.append(heapq._heappop_max(heap)[1]) #get the num (heap[1])
    return mlist
nums = [1,1,1,2,2,3]
k = 2
topKFrequent(nums, k)

[(3, 1), (2, 2), (1, 3)]


[1, 2]

In [65]:
#624. Maximum Distance in Arrays
def maxDistance(arrays: list[list[int]]) -> int:    
    #init so that first array will be 0 diff
    mmin = arrays[0][-1]
    mmax = arrays[0][0]
    res = 0
    for array in arrays:
        #update maxDistance 
        res = max(res, array[-1] - mmin, mmax - array[0])
        mmin = min(mmin, array[0])
        mmax = max(mmax, array[-1])    
    return res

arrs = [[-8,-7,-7,-5,1,1,3,4],[-2],[-10,-10,-7,0,1,3],[2]]
maxDistance(arrs)


14

[6. Zigzag Conversion](https://leetcode.com/problems/zigzag-conversion)


<img src="./assets/l6.jpeg" width="500px"/>

In [16]:
#For each character, append it to the the correct row
# using direction  up/down alternatingly, we can process row to know which row to append the current char to
def convert(s: str, numRows: int) -> str:
    if numRows == 1 or numRows >= len(s):
        return s
    
    rows = [""] * numRows
    row, direction = 0, 1  # Start at row 0 and move down

    for char in s:
        rows[row] += char  # Add character to current row
        if row == 0:  # If at the top, move down
            direction = 1
        elif row == numRows - 1:  # If at the bottom, move up
            direction = -1
        row += direction  # Update row index
    
    return "".join(rows)  # Concatenate all rows
s = "PAYPALISHIRING"
numRows = 4
convert(s, numRows)


'PINALSIGYAHRPI'

In [ ]:
''' 
charsInSection: diff = 2 * (numRows - 1)  # total chars in one section || big vertical step
for i in range(rows)
    while index < n:
        answer += s[index]  # Add the vertical character
        charsInBetween: diagonal_diff  = diff - 2 * i
        second_index = index + diagonal_diff  # diagonal index


'''
def convert(s: str, numRows: int) -> str:
    # Step 1: Handle the Edge Case
    if numRows == 1:
        return s  # If there's only one row, the result is the input string itself.

    # Step 2: Initialize Variables
    answer = ''  # Stores the final converted string
    n = len(s)  # Length of the input string
    diff = 2 * (numRows - 1)  # The "step size" for vertically aligned characters
    diagonal_diff = diff  # The step size for diagonal characters (calculated dynamically)
    second_index = 0  # Index for diagonal characters
    index = 0  # Current index in the string

    # Step 3: Iterate Through Each Row
    for i in range(numRows):
        index = i  # Start at the i-th character of the string for the current row
        while index < n:
            # Step 3a: Add the Vertical Character
            answer += s[index]  # Add the vertical character
            print(index)
            # Step 3b: Handle Diagonal Characters
            if i != 0 and i != numRows - 1:  # Only for rows between the first and last
                diagonal_diff = diff - 2 * i  # Calculate the step for the diagonal character
                second_index = index + diagonal_diff  # Compute the diagonal character's index
                if second_index < n:
                    answer += s[second_index]  # Add the diagonal character to the result string

            # Step 3c: Move to the Next Vertical Character
            index += diff  # Advance to the next vertical character in the current row
        print(answer)
    # Step 4: Return the Final Converted String
    return answer
    
s = "PAYPALISHIRING"
numRows = 4
convert(s, numRows)


In [ ]:
#38. Count and Say
class Solution:
    def countAndSay(self, n: int) -> str:
        # Step 1: Initialize the Starting String
        current_string = "1"  # The first string in the sequence is "1"

        # Step 2: Generate Subsequent Strings
        for _ in range(n - 1):  # Repeat the process n-1 times to generate the nth sequence
            next_string = ""  # Stores the next sequence in the iteration
            j = 0  # Pointer for the current group of digits
            k = 0  # Pointer for traversing and counting consecutive digits

            # Step 2a: Traverse and Count Consecutive Characters
            while j < len(current_string):
                while (
                    k < len(current_string)
                    and current_string[k] == current_string[j]  # Count consecutive digits
                ):
                    k += 1  # Increment k as long as characters match

                # Step 2b: Form the "Count and Say" Segment
                next_string += str(k - j) + current_string[j]  # Append count and the character
                j = k  # Move j to the next group of digits

            # Step 2c: Update the Current String for the Next Iteration
            current_string = next_string  # Prepare for the next iteration

        # Step 3: Return the nth Sequence
        return current_string


In [ ]:
#56. Merge Intervals
def merge(self, intervals: list[list[int]]) -> list[list[int]]:

    intervals.sort(key=lambda x: x[0])

    merged = []
    for interval in intervals:
        # if the list of merged intervals is empty or if the current
        # interval does not overlap with the previous, simply append it.
        if not merged or merged[-1][1] < interval[0]:
            merged.append(interval)
        else:
            # otherwise, there is overlap, so we merge the current and previous
            # intervals.
            merged[-1][1] = max(merged[-1][1], interval[1])

    return merged

[48. Rotate Image](https://leetcode.com/problems/rotate-image/)

In [ ]:
class Solution:
    def rotate(self, matrix: list[list[int]]) -> None:
        self.transpose(matrix)
        self.reflect(matrix)

    def transpose(self, matrix):
        n = len(matrix)
        for i in range(n):
            for j in range(i + 1, n):
                matrix[j][i], matrix[i][j] = matrix[i][j], matrix[j][i]

    def reflect(self, matrix):
        n = len(matrix)
        for i in range(n):
            for j in range(n // 2):
                matrix[i][j], matrix[i][-j - 1] = matrix[i][-j - 1], matrix[i][j]
                
                
def rotate(matrix: list[list[int]]) -> None:
    """
    Do not return anything, modify matrix in-place instead.
    """
    #rotate outer layer first, then inner

    #l,r: same as top,down, for layer boundary
    l, r = 0, len(matrix) - 1
    while l < r: #move inward
        top, bottom = l, r

        for i in range(r-l): #use i variable to move along with direction each side, i is within range 0, r-l.
            topleft = matrix[top][l+i]
            matrix[top][l+i] = matrix[bottom-i][l]
            matrix[bottom-i][l] = matrix[bottom][r-i]
            matrix[bottom][r-i] = matrix[top+i][r]
            matrix[top+i][r] = topleft
        l+=1
        r-=1
    return matrix
            
                





        
        

[2491. Divide Players Into Teams of Equal Skill](https://leetcode.com/problems/divide-players-into-teams-of-equal-skill/description/?envType=company&envId=paypal&favoriteSlug=paypal-all)

In [ ]:
'''
Constraints:
skill.length is even.
1 <= skill[i] <= 1000
We can sort and use 2 pointers. But that's O(nlogn). Use frequency hashmap: O(n) time, 0(1) space bc contraints of frequency hashmap is 1000
'''

class Solution:
    def dividePlayers(self, skill: list[int]) -> int:
        n = len(skill)
        total_skill = sum(skill)
        skill_frequency = {}

        # Calculate total skill and skill frequency
        for s in skill:
            skill_frequency[s] = skill_frequency.get(s, 0) + 1

        # Check if total skill can be evenly distributed among teams
        if total_skill % (n // 2) != 0:
            return -1

        #Get target skill from total skill and #players
        target = total_skill // (n // 2)
        total = 0

        # Calculate total chemistry while verifying valid team formations
        for s in skill:
            partner_skill = target - s

            # Check if a valid partner exists
            if skill_frequency[partner_skill] == 0:
                return -1

            # Calculate and add chemistry of the current pair
            total += s * partner_skill
            skill_frequency[partner_skill] -= 1

        # Return half of total as each pair is counted twice
        return total // 2

In [ ]:
#1010. Pairs of Songs With Total Durations Divisible by 60
'''
(a+b) % 60=0 <=> Either: 1. a % 60=0 AND b%60=0; 2 (a % 60)+(b % 60)=60
Hashmap/array remainders with size 60 to record the frequencies of each remainder - as the range of remainders is [0,59].
Time: linear, space: constant
'''

from collections import defaultdict
def numPairsDivisibleBy60(time: list[int]) -> int:
    remainders = defaultdict(int)
    ret = 0
    for t in time:
        if t % 60 == 0: # check if a%60==0 && b%60==0
            ret += remainders[0]
        else: # check if a%60+b%60==60
            ret += remainders[60-t%60]
        remainders[t % 60] += 1 # remember to update the remainders
    return ret

In [ ]:
#54 Spirtal Matrix
def spiralOrder(matrix: list[list[int]]) -> list[int]:
    result = []
    rows, columns = len(matrix), len(matrix[0])
    up = left = 0
    right = columns - 1
    down = rows - 1

    while len(result) < rows * columns:
        # Traverse from left to right.
        for col in range(left, right + 1):
            result.append(matrix[up][col])

        # Traverse downwards.
        for row in range(up + 1, down + 1):
            result.append(matrix[row][right])

        # Make sure we are now on a different row.
        if up != down:
            # Traverse from right to left.
            for col in range(right - 1, left - 1, -1):
                result.append(matrix[down][col])

        # Make sure we are now on a different column.
        if left != right:
            # Traverse upwards.
            for row in range(down - 1, up, -1):
                result.append(matrix[row][left])

        left += 1
        right -= 1
        up += 1
        down -= 1

    return result

In [242]:
#1328. Break a Palindrome
class Solution:
    def breakPalindrome(self, palindrome: str) -> str:
        # Step 1: Handle Edge Case
        if len(palindrome) == 1:
            return ""  # A single-character palindrome cannot be made non-palindromic
        
        # Step 2: Convert String to List (Since Strings are Immutable)
        palindrome = list(palindrome)
        
        # Step 3: Try to Replace the First Non-'a' Character with 'a'
        for i in range(len(palindrome) // 2):  # Only iterate over the first half
            if palindrome[i] != 'a':  # Find the first non-'a' character
                palindrome[i] = 'a'  # Replace it with 'a' to break the palindrome
                return "".join(palindrome)  # Convert list back to string and return
        
        # Step 4: If All Characters are 'a', Change the Last Character to 'b'
        palindrome[-1] = 'b'
        
        # Step 5: Return the Modified String
        return "".join(palindrome)


In [ ]:
# 2554. Maximum Number of Integers to Choose From a Range I
class Solution:
    def maxCount(self, banned: list[int], n: int, maxSum: int) -> int:
        banned_set = set(banned)  # Use a set for quick lookups
        total_sum = 0  # Track the cumulative sum
        count = 0  # Track the count of valid numbers

        for i in range(1, n + 1):
            if i in banned_set:
                continue  # Skip banned numbers
            total_sum += i
            if total_sum > maxSum:
                break  # Stop if the sum exceeds maxSum
            count += 1

        return count

In [ ]:
''' * Idea is to find the frequencies/(number of times an index appear
* in the requests) and then keep the maximum number from input array
* in the max frequency index and so on.
*
* Example: input array [1,2,3,4,5] requests [[1,3],[0,1]]
* frequencyOfRequests =[1,2,1,1,0];
*
* Since we are try to get maximum sum we have to position max number in the input array in the index such that, index has maximum frequency.
* Now sort the input array and frequencyOfRequests so that we will have max number and max frequency in first indexes.
* After sorting:
* input array = [5,4,3,2,1] 
* frequencyOfRequests =[2,1,1,1,0];
* 
*  for i in input array { 
*   sum += inputArray[i] * frequencyOfRequests[i] 
*   since sum can be very large and can cause overflow int; sum = sum % (10^9 + 7);
*  };
*
* frequencyOfRequests can be found by iterating every request from start to end and mark the inputArray[index]++; in this case Time =                   * O(inputArray.size) * O(request.size) but with marking start index with +1 and (end + 1) index with -1 Time = (requests.size) * (1);
*
*  Time - r + n + nlog(n) + flog(f) + n -> O(nlog(n))
*  Space - O(n) for storing frequencies.'''
class Solution:
    def maxSumRangeQuery(self, nums: List[int], requests: List[List[int]]) -> int:
        n = len(nums)  # Length of the nums array
        count = [0] * (n + 1)  # Array to track the frequency of requests for each index

        # Step 1: Mark the start and end points of each request using the difference array technique
        for start, end in requests:
            count[start] += 1  # Increment at the start index
            count[end + 1] -= 1  # Decrement after the end index

        # Step 2: Compute the actual frequency of requests for each index using prefix sum
        for i in range(1, n):
            count[i] += count[i - 1]  # Add the previous frequency to the current index

        # Remove the extra element used for boundary handling
        count = count[:-1]  # Trim the last element to match the length of nums

        # Step 3: Sort both nums and count in descending order
        # This ensures the largest numbers are paired with the most frequently requested indices
        nums.sort(reverse=True)
        count.sort(reverse=True)

        # Step 4: Compute the maximum sum
        result = 0
        MOD = 10**9 + 7  # Modulus to prevent overflow
        for i in range(n):
            result += count[i] * nums[i]  # Multiply corresponding values and add to the result
            result %= MOD  # Apply modulus to keep the result manageable

        return result  # Return the maximum sum modulo 10^9 + 7


In [ ]:
class Solution:
    def subarraysDivByK(self, nums: list[int], k: int) -> int:
        count = 0
        n = len(nums)
        
        for i in range(n):
            curr_sum = 0
            for j in range(i, n):
                curr_sum += nums[j]
                if curr_sum % k == 0:
                    count += 1
        
        return count
    
    

## Two Pointers

### Medium

[167. Two Sum II - Input Array Is Sorted](https://leetcode.com/problems/two-sum-ii-input-array-is-sorted/)

In [60]:
def twoSum(numbers: list[int], target: int) -> list[int]:
    #2 pointers: at start and at end. 
    # When sum(nums[i]+nums[j])> target -> decrement end p (decrease larger num in the sum)
    #sum(nums[i]+nums[j]) < target -> increase start p 
    #Time: O(n), Space: O(1)

    pre, pos = 0, len(numbers)-1
    while pre < pos:
        sum = numbers[pre]+numbers[pos]
        if sum < target:
            pre += 1
        elif sum > target:
            pos -= 1
        else:
            return [pre+1, pos+1]
    return []

[15. 3Sum](https://leetcode.com/problems/3sum/)

In [61]:
def threeSum(nums: list[int]) -> list[list[int]]:
    nums.sort()
    res = []

    #i is pivot (leftest), l next to i, r is end of array
    for i in range(len(nums)):
        #no repeated triplet, nums[i-1] worked with everything on right of nums[i]
        if i > 0 and nums[i] == nums[i-1]:
            continue 
        #can't offset from positive pivot 
        if nums[i] > 0:
            break
        l, r = i+1, len(nums)-1
        while l < r:
            
            sum = nums[i]+nums[l]+nums[r]
            
            if sum == 0:
                res.append([nums[i],nums[l],nums[r]])
                #only need to update 1 pointer
                l+=1 
                #no repeated triplet
                while nums[l-1] == nums[l] and l<r :
                    l+=1
                
            elif sum > 0:
                r-=1
            elif sum < 0:
                l+=1
        
    return res
nums = [-1,0,1,2,-1,-4]
nums1= [-2,0,1,1,2]
display(threeSum(nums1))
display(threeSum(nums))

[[-2, 0, 2], [-2, 1, 1]]

[[-1, -1, 2], [-1, 0, 1]]

[11. Container With Most Water](https://leetcode.com/problems/container-with-most-water)

In [ ]:
class Solution:
    def maxArea(self, height: List[int]) -> int:
        # Step 1: Initialize Variables
        maxarea = 0  # Variable to store the maximum area found so far
        left = 0  # Pointer starting at the beginning of the array
        right = len(height) - 1  # Pointer starting at the end of the array

        # Step 2: Two-Pointer Technique
        while left < right:
            # Step 2a: Calculate Width and Area
            width = right - left  # The horizontal distance between the two pointers
            # The area is determined by the shorter height between left and right
            maxarea = max(maxarea, min(height[left], height[right]) * width)

            # Step 2b: Move the Pointer with the Smaller Height
            if height[left] <= height[right]:
                left += 1  # Move the left pointer to the right
            else:
                right -= 1  # Move the right pointer to the left

        # Step 3: Return the Maximum Area
        return maxarea


## Stack

### Easy

[20. Valid Parentheses](https://leetcode.com/problems/valid-parentheses/)
1. Loop through the strings
2. If a open -> append to OpenStacks
3. Elif (close) & OF THE TOP of OpenStack: pop
4. Elif stack != 0 (still have non-open char -> illegal close char)

In [62]:
def isValid(s: str) -> bool:
    brack = {'{':'}', '[':']', '(':')'}
    stack = []
    for i in range(len(s)):
        if s[i] in brack: # if current visited char is OPEN
            stack.append(s[i])
        elif len(stack) != 0: #if current char is CLOSE
            if s[i] != brack[stack[-1]]: #but not match TOP OPEN => false
                return False
            stack.pop() #else pop
        # len(stack) == 0 + still have CLOSE char -> illegal close char
        elif len(stack) == 0: 
            return False
    if len(stack) != 0:
        return False
    return True

### Medium
Monotonic stack:
Efficient TRAVERSE and COMPARISON of elements in an ARRAY or sequence.
- Find the next nearest smaller || farthest greater
    - nearest smaller: use increasing stack (store largest so far, bc larger than that top is ignored, smaller than top is recorded)
- Monotonically Decreasing / Inc stack
- Lexicographical ordering

[155. Min Stack](https://leetcode.com/problems/min-stack/)

In [7]:
class MinStack:
    #Create a another minstack next to normal stack that stores upto current min val
    #minST always maintain the same size as stack
    def __init__(self):
        self.stack = []
        self.minSt = []

    def push(self, val: int) -> None:
        self.stack.append(val)
        #temp min
        # Only consider minSt[-1] if minSt not EMPTY
        self.minSt.append(min(val, self.minSt[-1] if self.minSt else val))


    def pop(self) -> None:
        #pop minSt always true. at any of its index, is always the stack's min up to that stack index: 
        # #st = [2, -1, 1, 0,-2]; min=[2,-1,-1,-1,-2]
        self.minSt.pop()
        self.stack.pop()

    def top(self) -> int:
        return self.stack[-1]

    def getMin(self) -> int:
        return self.minSt[-1]

minStack = MinStack()
minStack.push(-2)
minStack.push(0)
minStack.push(-3)
minStack.getMin()
minStack.pop()
minStack.top()
minStack.getMin()

print(f"Minimum element in the stack: {minStack.getMin()}")  

Minimum element in the stack: -2


[150. Evaluate Reverse Polish Notation](https://leetcode.com/problems/evaluate-reverse-polish-notation/)

This is postfix travel, utilize stack. Pop nums when encounter sign. Then push the result back to stack

In [64]:
import math
def evalRPN(tokens: list[str]) -> int:
    st = []
    res = 0
    for i in tokens:
        ch = i
        if ch == '+':
            res = st.pop() + st.pop()
        elif ch == '-':
            #later digit on top, pop first
            b,a = st.pop(), st.pop()
            res = a - b
        elif ch == '*':
            res = st.pop() * st.pop()
        elif ch == '/':
            #cast to int
            #e.g: -0.5 = 0, 8.9=8
            b,a = st.pop(), st.pop()
            res = int(a/b)
        else: #if num
            res = int(i)
        st.append(res)
    return st.pop()
tokens = ["10","6","9","3","+","-11","*","/","*","17","+","5","+"]
evalRPN(tokens)

6

[22. Generate Parentheses](https://leetcode.com/problems/generate-parentheses/)

[Generate Parentheses - Stack - Leetcode 22](https://www.youtube.com/watch?v=s9fokUqJ76A&t=1s)

<img src="assets/l22.png" width=800px />

In [10]:
def generateParenthesis(n: int) -> list[str]:
    out = []
    #comb = [] #each combination
    #Learned recursion TREE with # of o and c parentheses from 0 to n
    #add ( when open < n, add ) when close < open
    def backtrack(ocount, ccount, comb):
        #base case
        if ocount == n and ccount == n:
            out.append("".join(comb))
        
        if ocount < n:
            comb.append('(')
            backtrack(ocount+1, ccount, comb)
            comb.pop()
            
        if ccount < ocount: 
            comb.append(')')
            backtrack(ocount, ccount+1, comb)
            comb.pop()
    backtrack(0,0,[])
    return out
n=3
generateParenthesis(n)

['((()))', '(()())', '(())()', '()(())', '()()()']

[739. Daily Temperatures](https://leetcode.com/problems/daily-temperatures/)

Monotonic Decreasing Stack. O(n). Visit each elem once + Pop at most once -> O(2n)


In [40]:
def dailyTemperatures(temperatures: list[int]) -> list[int]:
    #Pattern reg: find nearest correlation -> stack.pop get the most recent val within the correlation.
    #Add temps to stack maintaining decreasingly monotonic, only (keep) pop when found the nearest warm (increase t)
    st = [] #index stack
    out = [0]*len(temperatures)
    
    for i,t in enumerate(temperatures):
        while st and t > temperatures[st[-1]]: #st[-1][0] = temp at top node
            index= st.pop()
            out[index] = i - index            
        st.append(i) #append all indices to st
    return out
dailyTemperatures([73,74,75,71,69,72,76,73])


[1, 1, 4, 2, 1, 1, 0, 0]

[962. Maximum Width Ramp](https://leetcode.com/problems/maximum-width-ramp/description/)

Max width ramp means: we care about the earliest smallest numbers.  
Approach:
- Use monotonic stack:  nums = [9,8,1,0,1,9,4,0,4,1] = > st=[9,8,1,0], what’s after doesn’t matter because the earliest smallest are here.  
- After forming the stack, go backwards nums bc we need latest end: endnum. 
    - Each endnum, keep popping stack from top while endnum > topnum.
    - why the rightmost end (j) has the right to keep popping the smallest (i)? When popped => already found the widest possible ramp for that i, and i can’t form a wider ramp .


In [36]:
def maxWidthRamp(nums: list[int]) -> int:
    st = [0] #(index)
    for i in range(1,len(nums)-1):
        if nums[i] < nums[st[-1]]: #decreasing order
            st.append(i)
    width = 0
    for i in range(len(nums) - 1, 0, -1):
        while st and nums[st[-1]] <= nums[i]:
            j = st.pop()
            if i - j > width: #get maxwidth at end i
                width = i - j  
    return width
nums = [9,8,10,7,6,5,1,0,1,9,4,0,4,1]
maxWidthRamp(nums)

9

In [4]:
#monotonic stack of all decreasing number that 
# not find its greater yet, and pop when found
def nextGreaterElements(nums: list[int]) -> list[int]:
    #find next greater elements for every number, return -1
    res = [-1]*len(nums)
    st = []
    for i, num in enumerate(nums):
        while st and nums[st[-1]] < num:
            #pop to get index and fill res
            res[st.pop()] = num     
        st.append(i)
    #second loop to search circularly
    if st:
        for i, num in enumerate(nums):
            while st and nums[st[-1]] < num:
                res[st.pop()] = num
    return res
nums = [1,2,3,4,3]
nextGreaterElements(nums)

[2, 3, 4, -1, 4]

<img src="assets/901stock.png" width="400px"/>

In [24]:
#901. Online Stock Span
# Use monotonic stack to skip past days with lower prices. Today's price is A.
# If A > stack.top B, pop B, add B span to A's.
# We will never visit B again bc B included in A’s span. Future price only deal with A and >A prices.
class StockSpanner:
    def __init__(self):
        self.stack = []

    def next(self, price: int) -> int:
        span = 1
        while self.stack and self.stack[-1][0] <= price:
            span+= self.stack.pop()[1] #get the span of the top (smallest)
        self.stack.append([price, span])
        return span
    
ops = ["StockSpanner","next","next","next","next","next","next","next"]
params = [[],[100],[80],[60],[70],[60],[75],[85]]
stockspanner = StockSpanner()
for op, param in zip(ops, params):
    if op == "next":
        result = stockspanner.next(param[0])  # Call next with the given price
        print("result",result)

result 1
result 1
result 1
result 2
result 1
result 4
result 6


<img src="assets/lc2866.jpg" width="400px">

In [58]:
#2866. Beautiful Towers II 
#Note: some prevsmall is -1, we'll never deal with those prevsmall bc they're too small to be used in condition II
# No need to form arrays prev/nextSmall, top stack at each turn is i's prev/nextSmall.
def maximumSumOfHeights(maxHeights: list[int]) -> int:
    #use monotonic stack, each turn, top stack is always i's prevsmall, using it to cal leftCost
    n = len(maxHeights)
    st = [0]    
    left = [maxHeights[0]]
    #To find prevsmall: larger top must be popped -> increasing stack
    for i in range(1,n):
        #use stack to find i's prevSmall 
        while st and maxHeights[st[-1]] > maxHeights[i]: #if not i's smaller
            st.pop()
            
        #use prevSmall (stack top) to find leftsum
        #Case 1: re-use i-1 for i bc i-1 matches mountain pattern
        if maxHeights[i-1] <= maxHeights[i]:
            left.append(maxHeights[i] + left[i-1] )
        #Case 2: chop all larger past i to current i, ** stack top: i's prevSmall
        else:
            #leftsum = height[i]* distance of i - prevsmall + leftsum[prevsmall]
            if st: 
                left.append( maxHeights[i]*(i - st[-1]) + left[st[-1]] )
            else: # Case i is the smallest
                left.append( maxHeights[i]*(i+1) )
        #add i to stack
        st.append(i)
    st = [n-1]
    right = [0]*(n-1)+[maxHeights[-1]]
    #to find nextsmall: go from right to left, larger must be popped
    for i in range(n-2, -1, -1):
        # process stack to get top = i's nextSmall
        while st and maxHeights[st[-1]] > maxHeights[i]:
            st.pop()
        if maxHeights[i+1] <= maxHeights[i]:
            right[i] = maxHeights[i] + right[i+1]
        else:
            if st:
                right[i] = maxHeights[i] * (st[-1]-i) + right[st[-1]]
            else:
                right[i] = maxHeights[i] * (n - i)
        st.append(i)
    maxSum = 0
    for i in range(n):
        maxSum = max(maxSum, left[i] + right[i] - maxHeights[i])
    return maxSum
maximumSumOfHeights([5,9,9,8,1,7,8,3,4,3,1])
maximumSumOfHeights([6,5,3,9,2,7]
)

22

In [75]:
def maxChunksToSorted( arr: list[int]) -> int:
#for all i in sorted arrs: arr[i] = i. arr = [2,0,1,3,4], 
#when can we cut a range: when num is within its range: 4,2,3 is within 2-4, 5 at 5, 1,0 at 0,1. [1 0 4 2 3, 5]
#**if the max found upto i == i: we can cut max 2,0,1 == 2, cut here, 
        cuts = 0
        maxFound = 0
        for i, num in enumerate(arr):
            maxFound = max(maxFound, num)
            if maxFound == i:
                cuts+=1
        return cuts

maxChunksToSorted([1,2,0,3])

2

853. Car Fleet  

I. Sort by Starting Position - Because rightmost car's speed can't never be changed by prev cars whose speed must be higher.

II. Use a stack to store fleets's arrival time
    For each car:
- Check if car can catch up to top car before target by calculating car's time to reach target -> compare with top's time stored in stack
- If car can catch up, ignore, else: add car to stack as it's a new fleet. 

In [ ]:
def carFleet(target: int, position: list[int], speed: list[int]) -> int:
    st = []
    #sort by position Descending (front car firsts)
    cars = list(sorted(zip(position, speed), key=lambda x: x[0], reverse=True))
    
    for pos, speed in cars:
        arriveTime = (target - pos)/speed
        
        #top: front car,  #check if car can pass top's time b4 target
        if st and arriveTime <= st[-1]:
            #if pass: car will merge to top some time 
            #-> ignore car, and top remains the fleet's representative
                continue 
        #add new fleet if car can't pass top before target.           
        st.append(arriveTime)
    
    return len(st)
target = 12
position = [10,8,0,5,3]
speed = [2,4,1,1,3]
carFleet(target, position, speed)

3

In [ ]:
def mostCompetitive(nums: list[int], k: int) -> list[int]:
    #use increasing mono stack to optain sub-array that the first elements are smallest.
    n = len(nums)
    st = []
    for i in range(n):
        #when popping to get smaller first num, make sure the later nums enough to fill in popped num to form k-size subarr:
        #n-1 -i = #later nums to fill in, len(st)-1: st after popped
        while st and st[-1] > nums[i] and (n-1 -i + len(st)) >= k :
            st.pop()
        
        if len(st)<k:
            st.append(nums[i])

    return st
# nums = [2,4,3,3,5,4,9,6]
nums = [3,5,2,6]
k = 2
# k = 4
mostCompetitive(nums, k)

[2, 6]

### Hard

[84. Largest Rectangle in Histogram](https://leetcode.com/problems/largest-rectangle-in-histogram/)

In [65]:
"""
we need:  #figure left and right bound at once at any i
we do: as we go from left to right with a stack, the stack can help record:
- left bound of prev smaller: it is top of stack after pop bigger prev at i
    - right bound: at the time we pop bigger prev, bigger presv also knows that, the current i that pop them, is their right bound
=> we have: 
left bound of i: top of stack after pop bigger prev. store left bound with i in stack.
right bound of i: when i is popped by another bar j, j is right bound. cal area with stored left and right.
"""

def largestRectangleArea(heights: list[int]) -> int:
    st = [[0,-1]] # init st of (self, leftbound). first leftbound of 0 is -1.
    heights.append(0) # 1) Append a 0 to flush out all bars at the end **

    n = len(heights) 
    maxArea = 0
    for i in range(1,n):
        # print(st)
        while st and heights[st[-1][0]] >= heights[i]: # ** pop until found smaller
            #when we pop, calculate popped area, i is popped's rightbound
            idx, leftbound = st.pop()
            area = (i - 1 - leftbound) * heights[idx]
            # print(area, i, leftbound, heights[idx] )
            maxArea = max(maxArea, area)
        if st:
            # print(i, "append ", st[-1])
            st.append([i, st[-1][0]])
        else: #means no prev smaller -> i is smallest -> extend left to idx 0
            st.append([i, -1])
    
    return maxArea
heights = [2,1,5,6,2,3]
heights = [2,1,5,6,5,5,6,3]

largestRectangleArea(heights)

25

[2940. Find Building Where Alice and Bob Can Meet](https://leetcode.com/problems/find-building-where-alice-and-bob-can-meet/description/)

Approach and Time Complexity  
- Mono stack:
    - Loop in q, create `unsolveQ` at b, a < b, heights[a] > heights[b]: `unsolveQ[b].append([heights[a], queryIndex])`
    - Loop b backwards in heights:
        - Binary search in stack for unsolveQ[b]
            - Keep go to rightest (toppest) in stack where st[i] > st[unsolveQ[b].height[a]] => get smallest, nearest greater at stack
        - Pop st if top < b, those tops are unnecessary now, bc we st.append(n)
    - Time : **O(q⋅logn+n)**. Space : O(n+q) monostack + unsolveQ   

- Heap:
    - Loop in q, process and create exact same `unsolveQ`
    - Loop forward in heights, generate and use heap as we go:
        - At each height, look back (at heap) to see while height[heap[top]] < current height, answer it and pop.
        - Push all queries at height to heap.
    - Time : **O(q⋅logq+n)** . Space : O(n+q) minheap + unsolveQ  

In [167]:
'''
Already have process unresolve query at all maxIndex b in queries like previous stack approach
- With stack, we go right to left in height to look forward to see the nearest greater HEIGHT
- With heap, we go left to right in height to look backward to quickly find and resolve QUERY IN MINHEAP that has smaller heights (and ofc idx). Use heap, we can start quickly at the smallest.

Heap intuition approach:
Iterate throught building from l to r, want to find the building in the unsolved query that is smaller than current height.
=> use minheap to keep getting query with smallest height and (ofc smaller index) quickly.

'''
import heapq
def leftmostBuildingQueries(heights: list[int], queries: list[list[int]]) -> list[int]:  
    n = len(heights)
    q = len(queries)
    unsolveQ = [[] for _ in range(n)]
    res = [-1] * q
    
    #1. Process queries to have unsolveQ[b] = [heights[a], queryIndex], b > a, height(a > b)
    
    for i in range(q):
        a, b = queries[i][0], queries[i][1]
        if a > b:
            a, b = b, a
        if heights[a] < heights[b] or a==b:
            res[i] = b
        else:
            unsolveQ[b].append([heights[a], i])
    minheap = []   
    #2. From l to r in height, generate and use minheap as we go. At each height, quickly find qs with smaller height+index
    # ofc the higherIndex in heap's queries always smaller than current index, heap is processed up to current.)
    for i in range(n):
        #2.2, Solve all qs with smaller heights: chekc if heap[top] < height, if yes, record answer and pop (resolved) heap.
        while minheap and minheap[0][0] < heights[i]:
            res[minheap[0][1]] = i
            heapq.heappop(minheap)

        #2.1. Add queries at height to minheap (after find smaller height+index buildings at height.)
        for query in unsolveQ[i]:
            heapq.heappush(minheap, query)
    return res 
heights = [5,3,8,2,3,5,2,4,6,5,5,7]
queries = [(0,3), (5,6), (8,10), (0,6), (7,2), (6,1), (1,5), (0,7), (3,5)]
leftmostBuildingQueries(heights, queries)


[8, 8, 11, 8, -1, 7, 5, 8, 5]

In [72]:
#Find nearest greater on the right -> use mono stack + bin search
def leftmostBuildingQueries(heights: list[int], queries: list[list[int]]) -> list[int]:
    #1. make sure swap all queries q (a,b): a < b
    #resolve all easy queries height. if height(a>= b), unsolveQ[b].append([heights[a], queryindex])
   
    n = len(heights)
    q = len(queries)
    res = [-1]*q
    unsolveQ = [[] for _ in range(n)] # index at [b] = [height[a], queryIdx]
    for i in range(q):
        a, b= queries[i][0],  queries[i][1]
        if a > b: #swap
            a, b = b, a
        if heights[a] < heights[b] or a == b:
            res[i] = b
        else: #b: larger idx, smaller val
            unsolveQ[b].append([heights[a], i])
            
    st = [] #stack form [height b, idx b]
    #Loop thru position b (i), use stack to: 
    for i in range(n-1, -1, -1):
        # 1.right near greater of position b,...
        for height, qid in unsolveQ[i]:
            if st: #...found greater than a in stack using binary search
                pos = binSearch(height, st)
                res[qid] = pos
        # 2. Process stack: pop top < b(ofc num <  a bc b<a, top is useless now)
        while st and st[-1][0] <= heights[i]:
            st.pop()
        st.append([heights[i], i]) #height[i] for bin search comparison
    return res
      
# keep go right (means smallest - nearest/toppest) when mid > height in st 
def binSearch(height, st):
    left = 0
    right = len(st) - 1
    idx = -1
    while left<= right:
        mid = (left+right) // 2
        if st[mid][0] > height: 
            left = mid+1 #keep go rightest - smaller part
            idx = mid #mid always >= old idx bc mid always goes-right-to-idx once idx first got assigned by mid
        else:
            right = mid-1
    return st[idx][1] if idx != -1 else -1
# Time Complexity: O(q⋅logn+n)
# Space Complexity: O(n+q) monostack + newQuery    
    
heights = [5,3,8,2,6,1,4,6]
queries = [[0,7],[3,5],[5,2],[3,0],[1,6]]
heights = [5,3,8,2,3,5,2,4,6,5,5,7]
queries = [(0,3), (5,6), (8,10), (0,6), (7,2), (6,1), (1,5), (0,7), (3,5)]
leftmostBuildingQueries(heights, queries)

[[], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [[5, 0]], [], [], [[5, 1], [5, 3], [3, 5]], [[8, 4], [5, 7]], [], [], [[6, 2]], []]


[8, 8, 11, 8, -1, 7, 5, 8, 5]

```python
class Solution:
    def trap(self, height):
        # Step 1: Initialize Variables
        ans = 0  # Stores the total trapped water
        current = 0  # Pointer to traverse the height array
        st = []  # Stack to store indices of the heights
        
        # Step 2: Traversing the Heights
        while current < len(height):
            # Step 2a: Process When We Find a "Basin"
            while len(st) != 0 and height[current] > height[st[-1]]:
                # Step 2b: Identify the Water Trap
                top = st[-1]  # Get the index of the lowest point
                st.pop()  # Remove it from stack
                
                # Step 2c: Ensure a Left Boundary Exists
                if len(st) == 0:
                    break  # If the stack is empty, there's no left boundary, so no water can be trapped
                
                # Step 2d: Compute the Water Volume
                distance = current - st[-1] - 1  # Width of the trapped water region
                bounded_height = (
                    min(height[current], height[st[-1]]) - height[top]
                )  # Height of trapped water
                
                # Calculate trapped water for this segment
                ans += distance * bounded_height
            
            # Step 3: Push Current Height Index
            st.append(current)  # Store the current height's index in the stack
            current += 1  # Move to the next index
        
        return ans  # Return the total trapped water
```
1. **Monotonic Stack**:
   - Stores indices of heights in **increasing order**.
   - Helps in detecting **left and right boundaries** of water-trapping areas.

2. **Core Idea**:
   - When we find a **right boundary** (i.e., `height[current] > height[st[-1]]`), we pop elements from the stack to calculate trapped water.
   - The **distance** (`current - st[-1] - 1`) and **bounded height** (`min(height[current], height[st[-1]]) - height[top]`) determine the trapped water.

3. **Efficiency**:
   - **Time Complexity:** `O(n)` (Each element is pushed and popped at most once)
   - **Space Complexity:** `O(n)` (For storing indices in the stack)

your answer is a perfect template: adequate, concise, no yapping lengthy, very good, contain code with ggreat explanation steps comments, dsa, core idea and effiency. good! keep do this to 

In [8]:

def minSwaps(s: str) -> int:
    stack_size = 0
    for ch in s:
        # If character is opening bracket, increment the stack size.
        if ch == "[":
            stack_size += 1
        else:
            # If the character is closing bracket, and we have an opening bracket, decrease
            # the stack size.
            if stack_size > 0:
                stack_size -= 1
    return (stack_size + 1) // 2 # + 1 to handle odd case, where if stacksize is 
minSwaps('[]][')

1

## Binary search
Seach in sorted arrays.
Find mid = (left + right) // 2. Avoid overflow: mid = l + ((r-l)//2)

[74. Search a 2D Matrix](https://leetcode.com/problems/search-a-2d-matrix/)

Bin search ROW (a range) -> bin search value in found ROW

Given matrix m*n: O(logm + logn) = O(log(m * n))

Turn into 1 1d array

In [13]:
def searchMatrix(matrix: list[list[int]], target: int) -> bool:
    ROW, COL = len(matrix), len(matrix[0])
    i,j = 0, ROW*COL-1 #start
    
    while i<=j:
        midIndex = (i+j)//2
        ro , co = midIndex//COL , midIndex % COL
        if matrix[ro][co] == target:
            return True
        if matrix[ro][co] < target:
            i = midIndex +1
        else:
            j = midIndex -1
    
    return False

matrix = [[1,3,5,7],[10,11,16,20],[23,30,34,60]]
target = 21
print(searchMatrix(matrix, target))

False


In [14]:
def searchMatrix(matrix: list[list[int]], target: int) -> bool:
    ROW, COL = len(matrix), len(matrix[0])
    top, bottom = 0, ROW - 1
    
    while top <= bottom:
        midRow = (bottom + top)//2
        if target > matrix[midRow][-1]:
            top = midRow + 1
        elif target < matrix[midRow][0]:
            bottom = midRow - 1
        else:
            break
    l, r = 0, COL -1 
    while l <= r:
        mid = (l+r)//2
        if target < matrix[midRow][mid]:
            r = mid - 1
        elif target > matrix[midRow][mid]:
            l = mid + 1
        else: 
            return True
    return False

Convert 2d -> 1d and do easy bin search

In [7]:
def searchMatrix(matrix: list[list[int]], target: int) -> bool:
    nums = [num for sublist in matrix for num in sublist]
    mid = len(nums)//2
    i,j= 0, len(nums)-1
    while i<=j:
        mid = (i+j)//2
        if target == nums[mid]:
            return True
        if target > nums[mid]:
            i = mid+1
        else:
            j = mid -1
    return False

matrix = [[1,3,5,7],[10,11,16,20],[23,30,34,60]]
target = 3
print(searchMatrix(matrix, target))

True


[875. Koko Eating Bananas](https://leetcode.com/problems/koko-eating-bananas/)

Lesson learned:
- If it seems way too complicated, start with worst brute force
- [0...k] search can always reduced to bin search

In [290]:
import math
def minEatingSpeed(piles: list[int], h: int) -> int:
    #h >= len(piles). Always. When h = len(piles), k = max(piles) (each hour, each pile)
    #In general: 1 <= k < = max (piles) for all h
    
    i, j = 1, max(piles)
    k = j

    while i <= j:
        mid = (i + j) // 2
        totalTime = 0
        
        for p in piles:
            totalTime += math.ceil(float(p) / mid)
            
        if totalTime <= h: #This means k is larger than should be, that it eats fater than given hours
            k = mid
            j = mid - 1
        else:
            i = mid + 1
    return k
            
piles = [3,6,7,11]
h = 8
minEatingSpeed(piles,h)


4

In [ ]:
#34. Find First and Last Position of Element in Sorted Array
class Solution:
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        
        def binary_search(nums, target, is_searching_left):
            left = 0
            right = len(nums) - 1
            idx = -1
            
            while left <= right:
                mid = (left + right) // 2
                
                if nums[mid] > target:
                    right = mid - 1
                elif nums[mid] < target:
                    left = mid + 1
                else:
                    idx = mid
                    if is_searching_left:
                        right = mid - 1
                    else:
                        left = mid + 1
            
            return idx
        
        left = binary_search(nums, target, True)
        right = binary_search(nums, target, False)
        
        return [left, right]
            

In [ ]:
# 33. Search in Rotated Sorted Array
#At each step of the binary search: Determine whether the left half or the right half is sorted. 
# Use this information to decide where the target might lie.
class Solution:
    def search(self, nums: List[int], target: int) -> int:
        # Step 1: Initialize Variables
        left, right = 0, len(nums) - 1  # Two pointers for binary search
        
        # Step 2: Perform Modified Binary Search
        while left <= right:
            mid = (left + right) // 2  # Calculate the middle index

            # Step 2a: Check if Target is Found
            if nums[mid] == target:  # If mid element is the target
                return mid  # Return its index

            # Step 2b: Determine Which Half is Sorted
            # Check if the left half is sorted
            if nums[left] <= nums[mid]:
                # Step 2c: Check if Target Lies in the Left Half
                if nums[left] <= target < nums[mid]:
                    right = mid - 1  # Narrow search to the left half
                else:
                    left = mid + 1  # Narrow search to the right half
            # Otherwise, the right half is sorted
            else:
                # Step 2d: Check if Target Lies in the Right Half
                if nums[mid] < target <= nums[right]:
                    left = mid + 1  # Narrow search to the right half
                else:
                    right = mid - 1  # Narrow search to the left half

        # Step 3: Target Not Found
        return -1  # If we exit the loop, the target is not in the array


In [ ]:
from bisect import bisect_left
#binary seach + greedy
class Solution:
    def lengthOfLIS(self, nums: list[int]) -> int:
        # Step 1: Initialize an Array to Track the LIS
        sub = []  # sub will store the smallest ending values for increasing subsequences
        
        # Step 2: Traverse the Array
        for num in nums:
            # Find the position to replace or extend
            i = bisect_left(sub, num)  # Binary search for the position

            # Step 2a: Extend the LIS if num is larger than all elements in sub
            if i == len(sub):
                sub.append(num)
            
            # Step 2b: Replace the smallest number greater than or equal to num
            else:
                sub[i] = num
        
        # Step 3: Return the Length of the LIS
        return len(sub)
#time: nlogn, space: n

In [ ]:
# DP represents the length of longest increasing subsequence ending at index i.
class Solution:
    def lengthOfLIS(self, nums: List[int]) -> int:
        # Step 1: Initialize DP Array
        dp = [1] * len(nums)  # dp[i] stores the LIS ending at index i
        
        # Step 2: Populate DP Array
        for i in range(1, len(nums)):
            for j in range(i):  # Check all previous elements
                # If nums[i] can extend the subsequence ending at nums[j]
                if nums[i] > nums[j]:
                    dp[i] = max(dp[i], dp[j] + 1)  # Update dp[i] to include nums[i]

        # Step 3: Return the Maximum Length
        return max(dp)  # The LIS is the largest value in dp


In [ ]:
#658. Find K Closest Elements
class Solution:
    def findClosestElements(self, arr: List[int], k: int, x: int) -> List[int]:
        # Step 1: Initialize Binary Search Bounds
        left = 0  # Start of the search range
        right = len(arr) - k  # Ensuring we can pick exactly k elements

        # Step 2: Perform Binary Search
        while left < right:
            mid = (left + right) // 2  # Calculate middle index

            # Step 2a: Compare Distances from x
            if x - arr[mid] > arr[mid + k] - x:
                # If arr[mid] is farther from x than arr[mid + k], move right
                left = mid + 1
            else:
                # Otherwise, keep left at mid since we might be in the correct window
                right = mid

        # Step 3: Return the K Closest Elements
        return arr[left:left + k]  # Return the subarray from `left` of size `k`


In [ ]:
class Solution:
    def findMin(self, nums: List[int]) -> int:
        # Step 1: Initialize Two Pointers for Binary Search
        l, r = 0, len(nums) - 1  # Left and right boundaries

        # Step 2: Perform Binary Search
        while l < r:
            m = l + (r - l) // 2  # Compute the middle index

            # Step 2a: Check if the Minimum is in the Right Half
            if nums[m] > nums[r]:  # Minimum must be on the right
                l = m + 1  # Move left boundary to the right
            
            # Step 2b: Otherwise, the Minimum is in the Left Half (or at m)
            else:
                r = m  # Move right boundary to mid

        # Step 3: Return the Minimum Value
        return nums[l]  # Left pointer now points to the minimum element


In [ ]:
#528. Random Pick with Weight
import random

class Solution:
    def __init__(self, w: list[int]):
        """
        Step 1: Precompute Prefix Sums
        - Create a prefix sum array to store cumulative weights.
        - This allows us to efficiently pick an index based on weighted probability.
        """
        self.prefix_sums = []
        prefix_sum = 0

        for weight in w:
            prefix_sum += weight  # Keep adding to get cumulative sum
            self.prefix_sums.append(prefix_sum)  # Store in the list

        self.total_sum = prefix_sum  # Store the total sum for scaling random values

    def pickIndex(self) -> int:
        """
        Step 2: Generate a Weighted Random Index
        - Generate a random number in the range [0, total_sum).
        - Use binary search to efficiently find the corresponding index.
        """
        target = self.total_sum * random.random()  # Scale random float [0, 1) to [0, total_sum)
        
        # Step 3: Perform Binary Search to Locate the Target in Prefix Sums
        low, high = 0, len(self.prefix_sums)

        while low < high:
            mid = low + (high - low) // 2
            if target > self.prefix_sums[mid]:  # Move right if target is greater
                low = mid + 1
            else:  # Move left if target is within this prefix sum
                high = mid
        
        return low  # The index found corresponds to the weighted random pick


In [ ]:
#380. Insert Delete GetRandom O(1)
'''
Time: GetRandom O(1). Insert and Delete O(1) average, O(N) worst-case when the operation exceeds capacity of currently allocated array/hashmap and invokes space reallocation.
Space O(N)
'''

from random import random

class RandomizedSet:
    def __init__(self):
        """
        Step 1: Initialize Data Structures
        - `self.indices`: A dictionary to store the index of each element in `self.nums`.
        - `self.nums`: A list to store the elements in the set.
        """
        self.indices = {}  # Maps values to their index in the list
        self.nums = []  # Stores inserted elements

    def insert(self, val: int) -> bool:
        """
        Step 2: Insert a Value in O(1) Time
        - If `val` is already in the set, return False.
        - Otherwise:
          - Append `val` to `self.nums`.
          - Store its index in `self.indices`.
          - Return True (successful insertion).
        """
        if val not in self.indices:
            self.nums.append(val)
            self.indices[val] = len(self.nums) - 1  # Store index of val
            return True  # Successful insertion

        return False  # Value already exists

    def remove(self, val: int) -> bool:
        """
        Step 3: Remove a Value in O(1) Time
        - If `val` exists in the set:
          - Swap `val` with the last element in `self.nums` (to maintain O(1) deletion).
          - Update the index of the swapped element in `self.indices`.
          - Remove the last element from `self.nums` and delete `val` from `self.indices`.
          - Return True (successful removal).
        - Otherwise, return False.
        """
        if val in self.indices:
            # Get index of val and the last element
            index_to_remove = self.indices[val]
            last_element = self.nums[-1]

            # Swap val with last element in the list
            self.nums[index_to_remove] = last_element
            self.indices[last_element] = index_to_remove  # Update index of swapped element

            # Remove the last element
            self.nums.pop()
            del self.indices[val]  # Remove val from the dictionary

            return True  # Successful removal

        return False  # Value does not exist

    def getRandom(self) -> int:
        """
        Step 4: Get a Random Element in O(1) Time
        - Use `random()` to generate a random index in the range [0, len(self.nums) - 1].
        - Return the element at that index.
        """
        return self.nums[int(random() * len(self.nums))]  # Get a random index and return the value


In [18]:
#162. Find Peak Element   
def findPeakElement(nums: list[int]) -> int:
        l = 0
        r = len(nums) - 1
        while l < r:
            mid = (l + r) // 2
            if nums[mid] > nums[mid + 1]:
                r = mid
            else:
                l = mid + 1
            print(l, r, nums[mid])
        return l
nums = [1,1,1,1,1,2,1,1,1,3,5,6,4]

findPeakElement(nums)

7 12 1
10 12 3
10 11 6
11 11 5


11

## Sliding Window

l = 0;  
for r in len(arr):  

    determine when len(window) reaches max to reduce with
    while
    


### Easy

[121. Best Time to Buy and Sell Stock](https://leetcode.com/problems/best-time-to-buy-and-sell-stock/)

In [2]:
def maxProfit(prices: list[int]) -> int:
    buy = prices[0] #buy price
    mx = 0 #max profit
    for i in prices:
        if i < buy: #update best buy price
            buy = i
        elif i > buy and i - buy > mx: #update max profit
            mx = i - buy
    return mx

### Medium

[3. Longest Substring Without Repeating Characters](https://leetcode.com/problems/longest-substring-without-repeating-characters/)

In [3]:
def lengthOfLongestSubstring(s: str) -> int:
    l = 0 #left pointer
    longest = 0
    cset = set()
    for r in range(len(s)):
        #EXPAND TERMINATION: s[r] in cset
        while s[r] in cset: #cannot proceed with this string's start l, remove that.
            cset.remove(s[l])
            l+=1

        cset.add(s[r])
        longest = max(longest, r-l+1)
    return longest

[424. Longest Repeating Character Replacement](https://leetcode.com/problems/longest-repeating-character-replacement/)

In [25]:
def characterReplacement(s: str, k: int) -> int:
    l = 0
    longest = 0     #  length of longest valid window
    maxfreq = 0    # Keeps track of most freq character count in current window
    count = defaultdict(int)  # Dictionary to count characters in current window
    
    for r in range(len(s)):
        # Process window expansion
        count[s[r]] += 1
        
        # Update maxfreq if the newly added char is now more frequent
        maxfreq = max(maxfreq, count[s[r]])
        
        # WindowLength - maxfreq = numberCharsToBeReplace 
        while (r - l + 1) - maxfreq > k:
            #Process contraction: remove freq of char s[l]
            count[s[l]] -= 1  
            l += 1            
            
        longest = max(longest, r - l + 1)
    
    return longest

s = "ABCFBBBAACCCCC"
k = 4
characterReplacement(s, k)

9

[567. Permutation in String](https://leetcode.com/problems/permutation-in-string/)

Used 2 hashmap d1, d2 + sliding window

In [54]:
from collections import defaultdict
def checkInclusion(s1: str, s2: str) -> bool:
    d1, d2 = defaultdict(int), defaultdict(int)
    l = 0
    for i in s1:
        d1[i]+= 1
            
    for r in range(len(s2)):
        char = s2[r]
        if char not in d1:
            if d1 == d2:
                return True
            #reset d2
            d2.clear()
            continue
        
        if not d2: #set l = r (start new strs) after d2 reset to empty
            l = r

        d2[char]+= 1 #update count of char in d2
        #IMPORTANT: determine while when: d2[char] > d1[char]
        #Ensure ALL CHAR count in d2 <= d1.
        while d2[char] > d1[char]:
            d2[s2[l]] -=1
            l += 1
        if d1 == d2:
            return True
    return False
s1 = "adc"
s2 = "dcda"
checkInclusion(s1,s2)

True

Optimized sliding window with **match** count

Maintain a match variable, which stores the #chars (out of the 26 alphabets), THAT have the same occurence in **s1** and **current window in s2s2s2**. 

When we slide the window, 

- if the deduction of the last element and the addition of the new element leads to a new frequency match of any of the characters, we increment the match by 1. 
- Else if count of a char was the same earlier(prior to addition or removal) as in s1

    => NEW mismatch that was JUST USED TO be taken as prev **match**! -> decrement **match**.
     
- Else, do nothing, since **match** already up-to-date with all the matched and mismatched chars before.
After the shifting of the window, the count evaluates to 26, it means all the characters match in frequency totally. So, we return a True in that case immediately.

In [3]:
def checkInclusion(s1: str, s2: str) -> bool:
    #This not check is essential because if move on, can't access s2[i] as s1[i] in next examine range(len(s1)):
    if len(s2) < len(s1):
        return False
    
    match = 0
    s1c,s2c = [0]*26, [0]*26
    
    #1. Get counts of s1,s2, up to len(s1):
    for i in range(len(s1)):
        #s1c records s1, s2c records s2
        s1c[ord(s1[i]) - ord('a')] += 1
        s2c[ord(s2[i]) - ord('a')] += 1
    
    #2. Init match of 26 chars:
    for i in range(26):
        if s1c[i] == s2c[i]:
            match += 1
    
    l = 0
    #3. Work on the rest
    for r in range(len(s1), len(s2)):
        if match == 26: return True
        
        #Move l + r, consider both moves
        indexR = ord(s2[r]) - ord('a') #right char
        s2c[indexR] +=1 #(I)
        #After (I) update        
        #Match ONLY INCRE when count of current char in 2 str matches <3
        #Match ONLY DECRE when what JUST PREVIOUS match => now mismatch, must be taken out of match
        if s1c[indexR] == s2c[indexR]:
            match += 1
        elif s1c[indexR] == s2c[indexR] - 1:
            match -=1 
        
        indexL = ord(s2[l]) - ord('a') #left char
        s2c[indexL] -=1
        if s1c[indexL] == s2c[indexL]:
            match += 1
        elif s1c[indexL] == s2c[indexL] + 1:
            match -=1 
        l+=1
    return match == 26

s1 = "adc"
s2 = "dcdba"
checkInclusion(s1,s2)


False

### Hard

[76. Minimum Window Substring](https://leetcode.com/problems/minimum-window-substring)

- **Key Variables**:
  - need, match: number of unique character: needed in t - has matched or exceeded so far
  - windCount, tCount: counter of window and t


In [198]:
from collections import Counter, defaultdict
def minWindow(s: str, t: str) -> str:
    #Approach: dictCountWindow, dictCountT
    tCount = Counter(t)
    windCount = {}
    l,r = 0,0
    # number of unique characters match and need
    match = 0
    need = len(tCount)
    
    res = [-1,-1] #result L and R
    while r < len(s):
        #1. Increment windCount and check match on s[l]
        windCount[s[r]] = windCount.get(s[r], 0) + 1
        if s[r] in tCount and windCount[s[r]] == tCount[s[r]]:
            match += 1
        
        #2. When finish forming
        while match == need :
            #Get result
            if r - l < res[1] - res[0] or res == [-1,-1]:
                res = [l, r]
            #Contract and update windCount, match
            windCount[s[l]] -= 1
            if s[l] in tCount and windCount[s[l]] < tCount[s[l]]:
                match -= 1
            l +=1 
        
        r+= 1
    return s[res[0]: res[1]+1] if res!=[-1,-1] else ''
        
s = "ADOBECODEBANC"
t = "ABC"
minWindow(s,t)

'BANC'

## Linked List

### <a id='initll'>Init Linked List for testing.</a>

In [202]:
from typing import Optional

# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next
    def __repr__(self):
        s = "("+ str(self.val) + ") => "
        tm = self.next
        while tm:
            s += str(tm.val) + " -> "
            tm = tm.next
        return s

def print_list(head: Optional[ListNode]) -> None:
    while head:
        print(head.val, end=" -> ")
        head = head.next
    print("None")

# Helper function to create a linked list from a list of values
def create_list(values):
    if not values:
        return None
    head = ListNode(values[0])
    current = head
    for val in values[1:]:
        current.next = ListNode(val)
        current = current.next
    return head



In [5]:
# Test Case 1: Basic Test
head1 = create_list([1, 2, 3, 4, 5])
print("Original Linked List:")
print_list(head1)
# reversed_head1 = reverseList(head1)
# print("Reversed Linked List:")
# print_linked_list(reversed_head1)

Original Linked List:
1 -> 2 -> 3 -> 4 -> 5 -> None


### Easy
Run Init [LinkedList](#initll) before testing methods

#### [206. Reverse Linked List](https://leetcode.com/problems/reverse-linked-list/)

Reverse: prev is always the reversed list, not head, or temp

In [12]:

def reverseList(head: Optional[ListNode]) -> Optional[ListNode]:
    next = None #real next's next
    prev = None
    temp = head
    
    while temp:
        #save next
        next = temp.next 
        #point to prev
        temp.next = prev
        #update prev&temp
        prev = temp
        temp = next

    print(head)
    return prev #at the end, prev is head
reverseList(create_list([1,2,3,4]))

(1) => 


(4) => 3 -> 2 -> 1 -> 

#### [21. Merge Two Sorted Lists](https://leetcode.com/problems/merge-two-sorted-lists/)

##### Iterative

Create new list and gradually add next smaller nodes from two lists to the merged.

        Learned: temp = dummy
        
        temp.next = min(l1,l2); then temp = temp.next means temp helps adding things and act for dummy

In [ ]:
def mergeTwoLists(self, list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:
    
        dummy = ListNode()
        temp = dummy
        
        while list1 and list2:
            if list1.val < list2.val:
                temp.next = list1
                list1 = list1.next
            else:
                temp.next = list2
                list2 = list2.next
            temp = temp.next
        
        temp.next = list1 or list2
        dummy = dummy.next
        return dummy

##### Recursive 
Without using another list

In [ ]:
def mergeTwoLists(self, list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:  
    def merging(list1, list2):
        
        #base:list1 or list2 completely examined
        if not list1:
            return list2
        if not list2:
            return list1
        
        #the smaller will be assigned its next
        #mergetwolist function acts like returning, providing the smaller node between two node args and keep moving on"
        if list1.val < list2.val:
            list1.next = merging(list1.next, list2)
            #providing the smaller results for the parent recursion
            return list1 
        else:
            list2.next = merging(list2.next, list1)
            return list2
    
    return merging(list1, list2)

#### [141. Linked List Cycle](https://leetcode.com/problems/linked-list-cycle/)


In [ ]:
def hasCycle(head: Optional[ListNode]) -> bool:
    #Guaranteed to meet when having a circle
    #Note: From meet point -> knot = head -> knot. 
    # ==> Find knot by move f to head, f and s incre 1 until meet
    f,s = head, head

    while f and f.next:
        f = f.next.next
        s = s.next
        if s == f:
            return True
    return False

### Medium

#### [143. Reorder List](https://leetcode.com/problems/reorder-list/)

**Approach 1** (Easy, slow): Create a arr, add each node to list in order. 

-> Reorder by indexing list 

In [3]:
def reorderList(head: Optional[ListNode]) -> None:
    """
    Do not return anything, modify head in-place instead.
    """
    if head.next == None or head.next.next == None:
        return head
        
    cur = head
    nodelist = list()
    while cur:
        nodelist.append(cur)
        cur = cur.next
    
    r = len(nodelist) - 1
    stop =len(nodelist) //2

    for l in range (stop):
        # todo: examine 3-4-5 case.   
        nodelist[l].next = nodelist[r]
        nodelist[r].next = nodelist[l+1]
        r-=1
    if len(nodelist) % 2 == 0:
        nodelist[r+1].next = None
    else:
        nodelist[r].next = None
    return head
    
head = [1,2,3,4]

head1 = create_list(head)
print_list(head1)

print(reorderList(head1))

1 -> 2 -> 3 -> 4 -> None
(1) => 4 -> 2 -> 3 -> 


**Approach 2**
1. Split into two halves. Using f,s pointers (s incre 1, f incre 2). When f no next -> s at middle
2. Reverse second half
3. Merge side by side 2 halves

In [4]:
import math
def reorderList(head: Optional[ListNode]) -> None:
    """
    Do not return anything, modify head in-place instead.
    """
    # firstnd middle
    s, f = head, head
    while f.next and f.next.next:
        s = s.next
        f = f.next.next
    
    #split
    second = s.next #saved secondcond half
    s.next = None #cut tail at mid
    #mid 's' still belongs to head -> cut tail of head
    
    #Reversecond
    prev = None
    next = None
    while second:
        next = second.next
        second.next = prev
        prev = second
        second = next
    
    second = prev #prev is reversecondd list
    
    #Merge 2 halves
    
    first = head #first half
    while first and second:
        fnext, snext = first.next, second.next
        first.next = second
        second.next = fnext
        first, second = fnext,snext
    return head #for testing
    
head = [1,2,3,4,5,6,7,8,9]

head1 = create_list(head)

print(reorderList(head1))

(1) => 9 -> 2 -> 8 -> 3 -> 7 -> 4 -> 6 -> 5 -> 


#### [19. Remove Nth Node From End of List](https://leetcode.com/problems/remove-nth-node-from-end-of-list/)

In [205]:
from typing import Optional
def removeNthFromEnd(head: Optional[ListNode], n: int) -> Optional[ListNode]:
    
    dummy = ListNode()
    #Dummy to remove head.
    dummy.next = head 
    l, r = dummy, head
    #r go n times
    for _ in range(n-1):
       r = r.next

    #push r to last
    while r.next: 
        r = r.next
        l = l.next
    l.next = l.next.next
    
    return dummy.next # return dummy.next instead of head to avoid situation remove head

head = [1,2]

head1 = create_list(head)
print(removeNthFromEnd(head1,2))
        
       

(2) => 


#### [287. Find the Duplicate Number](https://leetcode.com/problems/find-the-duplicate-number/description/)

In [ ]:
def findDuplicate(nums: list[int]) -> int:
    #floyd's hare and tortoise
    
    s,f = 0, 0
    #do while loop instead of while s!=f to run the first loop
    while True:
        s = nums[s]
        f = nums[nums[f]]
        if s == f:
            break

    #when s and f meet, set f to beginning
    f = 0
    #each takes 1 steps
    while s != f :
        s = nums[s]
        f = nums[f]
            
    return s

#### [2. Add Two Numbers](https://leetcode.com/problems/add-two-numbers/)

In [ ]:
def addTwoNumbers(l1: Optional[ListNode], l2: Optional[ListNode]) -> Optional[ListNode]:
        res = ListNode()
        temp = res
        carr = 0 #carry
        
        #finsih loop while both l1 and l2 end
        while l1 or l2:
            
            if l1 and l2:
                ssum = l1.val + l2.val + carr
            elif l1:
                ssum = l1.val + carr 
            elif l2:
                ssum = l2.val + carr
            
            temp.val = ssum%10
            carr = ssum//10

            l1 = l1.next if l1 else None
            l2 = l2.next if l2 else None
            
            if l1 or l2:
                temp.next = ListNode()
                temp = temp.next
                
        if carr == 1:
            temp.next = ListNode()
            temp = temp.next
            temp.val = 1
        return res
l1 = create_list([9,9,9,9,9,9,9])
l2 = create_list([9,9,9,9])
addTwoNumbers(l1,l2)

(8) => 9 -> 9 -> 9 -> 0 -> 0 -> 0 -> 1 -> 

#### [146. LRU Cache](https://leetcode.com/problems/lru-cache/)

1. <b> <font color='yellow'>Lessons</font> </b>
Create nodes next to head and tail like left, right that points to head, tail. To refer head/tail, only used left.next or right.prev

-> To reduce complexity of code, reduce edge case where **head** and **tail** may not have **prev** and **next** like other nodes

<b> <font color='yellow'>Each time LRU change</font> </b>
Create 2 function: remove node, insert node to list
    

2. <b> <font color='yellow'>Approach</font> </b> LinkedList + HashMap for Put O(1) + Get O(1)
- array inefficient for changing the order of elements from most to least recently used

- To get any element O(1), we need to know the parent node of get node for order transition (getnode -> head, parent->get.next)
- To retrieve parent node O(1), node must point to **parent** 
                
            -> doule linkedlist

- hashmap for put only is ineffecient for remaining the order of most to LRU element, need to corporate **linkedlist**.




In [ ]:
class LRUCache:
#tailDummy -> nodes -> headDummy
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.head = Node(0,0) #most recent used
        self.tail = Node(0,0) #least recent used
        self.mdict = {}
        self.tail.next, self.head.prev = self.head, self.tail
        
        
    def remove(self,node):
        #all nodes have prev and next thanks to head and tail
        prev, next = node.prev, node.next
        prev.next = next
        next.prev = prev
        
    def insert(self,node):
        prev, next = self.head.prev, self.head
        prev.next = next.prev =node
        node.next, node.prev = next, prev
        
        
        
    def get(self, key: int) -> int:
        if key not in self.mdict:
            return -1
        node = self.mdict[key]
        #each time use the element, remove and reinsert
        self.remove(node)
        self.insert(node)
        return node.val
    
    def put(self, key: int, value: int) -> None:
        if key in self.mdict:
            self.remove(self.mdict[key])
            
        newNode = Node(key, value)
        #add to dict and list
        self.mdict[key] = newNode
        self.insert(newNode)
        
        #dismiss LRU
        if len(self.mdict) > self.capacity:
            lru = self.tail.next
            self.remove(lru)
            self.mdict.pop(lru.key)

    def __repr__(self):
        s = "("+ str(self.head.val) + ") => "
        tm = self.head.next
        while tm:
            s += str(tm.val) + " -> "
            tm = tm.next
        s+='\n'
        s+=str(self.mdict)
        return s
class Node():
    def __init__(self, key, val, next = None, prev=None):
        self.key = key
        self.val = val
        self.next = next
        self.prev = prev
    def __str__(self) -> str:
        return "(" +  str(self.key) + ": " + str(self.val) + ")"
    def __repr__(self) -> str:
        return "(" + str(self.key) + ": " + str(self.val) + ")"

get 1 -1
get 2 3
(0) => 
{2: (2: 3), 4: (4: 1)}


In [ ]:
#Use built-in
from collections import OrderedDict
class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.dic = OrderedDict()

    def get(self, key: int) -> int:
        if key not in self.dic:
            return -1

        self.dic.move_to_end(key)
        return self.dic[key]

    def put(self, key: int, value: int) -> None:
        if key in self.dic:
            self.dic.move_to_end(key)

        self.dic[key] = value
        if len(self.dic) > self.capacity:
            self.dic.popitem(False)

# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

#### [138. Copy List with Random Pointer](https://leetcode.com/problems/copy-list-with-random-pointer/)


Approach: Go through 2 iterations
1. Create new deep copy nodes, map old nodes to new nodes in dict

2. Linked nodes' next and rands
- Link rands by mapping new nodes in dict by using relationships in old nodes




In [ ]:
#debug helpers
def create_linked_list(head_list):
    if not head_list:
        return None

    # Create nodes and store them in a dictionary with their indices
    nodes = {}
    for i, (val, _) in enumerate(head_list):
        nodes[i] = Node(val)

    # Connect nodes based on the second element of each sublist
    for i, (_, random_index) in enumerate(head_list):
        if random_index is not None:
            nodes[i].random = nodes[random_index]

    # Connect nodes based on their order in the original list
    for i in range(len(head_list) - 1):
        nodes[i].next = nodes[i + 1]

    return nodes[0]
class Node:
    def __init__(self, x: int, next: 'Node' = None, random: 'Node' = None):
        self.val = int(x)
        self.next = next
        self.random = random
    def __repr__(self):
        current = self
        s = ""
        while current:
            value = current.val
            random_node = current.random.val if current.random else None
            s+=f"({value}, {random_node}) -> "
            current = current.next
        s+="None"
        return s
    

In [ ]:
from typing import Optional

def copyRandomList(head: 'Optional[Node]') -> 'Optional[Node]':
        
        temp = head
        mdict = {None: None} #in case mdict[temp.next] or mdict[temp.random] null 
        
        #Map in dict {Key: Val} = {Original Node: New Node}
        while temp:
                mdict[temp] = Node(temp.val)
                temp = temp.next
        
        temp = head
        
        while temp:
                copiedNode = mdict[temp]
                #copiedNode is the copied verson of temp
                # mdict[temp.random] is the copied version of temp.random
                # => mdict[temp.random] is random version of newNode
                # So as "next"
                copiedNode.next = mdict[temp.next]
                #mdict[temp.random] is the deep copied node newNode.random
                copiedNode.random = mdict[temp.random]
                temp = temp.next
                
        return mdict[head]


head_list = [[7, None], [13, 0], [11, 4], [10, 2], [1, 0]]
head_node = create_linked_list(head_list)

copyRandomList(head_node)
        
        

(7, None) -> (13, 7) -> (11, 1) -> (10, 11) -> (1, 7) -> None

[1669. Merge In Between Linked Lists](https://leetcode.com/problems/merge-in-between-linked-lists)

In [ ]:
def mergeInBetween(list1: ListNode, a: int, b: int, list2: ListNode) -> ListNode:
    #1. get end list 2
    end2 = list2
    while end2.next:
        end2 = end2.next
    
    connect = list1
    #2. l1 go to connect start
    for i in range(a-1):
        connect = connect.next
    #3. remove: go to rest part
    rest1 = connect.next
    for _ in range(b-a+1):
        rest1 = rest1.next
        
    connect.next = list2 #connect to start of list2
    end2.next = rest1
    
    return list1

[237. Delete Node in a Linked List](https://leetcode.com/problems/delete-node-in-a-linked-list)

In [ ]:
class Solution:
    def deleteNode(self, node):
        # Overwrite data of next node on current node.
        node.val = node.next.val
        # Make current node point to next of next node.
        node.next = node.next.next

## Trees
A tree include trees.
1. Base case
2. Call function on left subtree (root.left)
3. Call function on right subtree (root.right)
4. Join function/return  
**Inorder**: left -> node -> right  
**Pre**: node -> left ->; **Post**: left->right->node

In [ ]:
from collections import deque
def inorderTraversal(root):
    stack = []
    result = []
    current = root

    while stack or current:
        # Reach the leftmost node
        while current:
            stack.append(current)
            current = current.left
        # Process the node
        current = stack.pop()
        result.append(current.val)
        # Visit the right subtree
        current = current.right

    return result
def preorderTraversal(root):
    if not root:
        return []
    
    stack = [root]
    result = []

    while stack:
        node = stack.pop()
        result.append(node.val)
        
        # Push right first so that left is processed first
        if node.right:
            stack.append(node.right)
        if node.left:
            stack.append(node.left)

    return result
def postorderTraversal(root):
    if not root:
        return []
    
    stack = [root]
    result = []
    
    while stack:
        node = stack.pop()
        result.append(node.val)
        
        # Push left first so that right is processed first
        if node.left:
            stack.append(node.left)
        if node.right:
            stack.append(node.right)
    
    # Reverse the result because we need left -> right -> root
    return result[::-1]

def inorderDFS(root):
    if not root:
        return []
    return inorderDFS(root.left) + [root.val] + inorderDFS(root.right)
def BFS(root):
    q = deque([root])
    while q:
        for i in range(len(q)):
            node = q.popleft()
            print(node)
            if node.left: q.append(node.left)
            if node.right: q.append(node.right)


### Easy

#### Init Tree

In [236]:
from typing import Optional
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
    def __repr__(self):
        result = "tree: {"
        queue = [self]

        while queue:
            current = queue.pop(0)

            if current:
                result += str(current.val) + " "

                # Enqueue left and right children
                queue.append(current.left)
                queue.append(current.right)
            else:
                result += "null "
        result += "}"
        return result.rstrip()
def to_tree(lst):
    if not lst:
        return None

    root = TreeNode(lst[0])
    nodes = [root]
    i = 1

    while nodes and i < len(lst):
        current = nodes.pop(0)

        if lst[i] is not None:
            current.left = TreeNode(lst[i])
            nodes.append(current.left)

        i += 1

        if i < len(lst) and lst[i] is not None:
            current.right = TreeNode(lst[i])
            nodes.append(current.right)

        i += 1

    return root
# root = [3,9,20,None,None,15,7]
# mtree = to_tree(root)
# mtree


#### [226. Invert Binary Tree](https://leetcode.com/problems/invert-binary-tree/)

In [ ]:
#DFS stack call
def invertTree(root: Optional[TreeNode]) -> Optional[TreeNode]:
    #if have no child
    if not root:
        return
    #perform first, call stack later
    temp = root.left
    root.left = root.right
    root.right = temp
    
    invertTree(root.left)
    invertTree(root.right)
    return root

#### [938. Range Sum of BST](https://leetcode.com/problems/range-sum-of-bst/description/?envType=daily-question&envId=2024-01-08)

- In BST, inorder traversal is sorted

Approach 1: Inorder Traversal  
Approach 2: Check each node
- If a node is larger than high -> don't go right
- If a node is smaller than low -> don't go left



In [ ]:
class Solution:
    def rangeSumBST(self, root: Optional[TreeNode], low: int, high: int) -> int:
        if not root:
            return 0
        if root.val >= low and root.val <= high:
            return root+ self.rangeSumBST(root.left,low, high)+ self.rangeSumBST(root.left,low, high)
        if root.val < low:
            return self.rangeSumBST(root.right)
        if root.val > high:
            return self.rangeSumBST(root.left)
root = to_tree([10,5,15,3,7,13,18,1,None,6])
low = 6 
high = 10
Solution.rangeSumBST(Solution, root, low, high)

23

#### [872. Leaf-Similar Trees](https://leetcode.com/problems/leaf-similar-trees/description/?envType=daily-question&envId=2024-01-09)
Get int[] of leaves's val of each tree and compare

In [ ]:
class Solution:
    def leafSimilar(self, root1: Optional[TreeNode], root2: Optional[TreeNode]) -> bool:
        if not root1 and not root2:
            return True
        def getleafdfs(node, res):
            if not node.left and not node.right:
                res.append(node.val)
                return res
            if node.left:
                res = getleafdfs(node.left, res)
            if node.right:
                res = getleafdfs(node.right, res)
            return res
        return  getleafdfs(root1, []) == getleafdfs(root2, [])
root1 = [3,5,1,6,2,9,8,None,None,7,4]
root2 = [3,5,1,6,7,4,2,None,None,None,None,None,None,9,8]

#### [104. Maximum Depth of Binary Tree](https://leetcode.com/problems/maximum-depth-of-binary-tree/) 
All solutions' complexity is O(n) <a id='maxdepth'></a>

**DFS Recursive**

In [ ]:
def maxDepth(root: Optional[TreeNode]) -> int:
    if not root:
        return 0
    return 1 + max(maxDepth(root.left), maxDepth(root.right))

**DFS Iterative**
1. Use a stack to record [[node, node.depth]]. Init with [root:1]
2. DFS because the added e to stack will go deep down (keep popping and processing child of child, because child of child added to top)
3. Pseudocode  
 While stack not empty:  
    pop node -> append {children : parent.level++}  
    maxlevel = max(maxlevel, node.level)  

In [72]:
from collections import OrderedDict
def maxDepth(root: Optional[TreeNode]) -> int:
    if not root:
        return 0
    maxDepth = 1
    stack = [[root, 1]]
    while stack:
        node, nodeDepth = stack.pop()
        if node:
            stack.append([node.left, nodeDepth+1])
            stack.append([node.right, nodeDepth+1])
            maxDepth = max(maxDepth, nodeDepth)
    return maxDepth

root = [3,9,20,None,None,15,7]
print(maxDepth(to_tree(root)))

3


**BFS (Iterative)**
1. Queue: deque([root]), level = 1
2. Each time popnode = deque.popleft:  Pop "node", add node's left, child if available. level++.
3. ...Until deque empty

In [ ]:
from collections import deque 
def maxDepth(root: Optional[TreeNode]) -> int:
    #edge case: empty root
    if not root:
        return 0 

    level = 0
    #init the queue of list [] starting with root
    q = deque([root])
    
    while q:
        for i in range(len(q)):
            popnode = q.popleft() #pop parents level and add children levels
            if popnode.left: q.append(popnode.left)
            if popnode.right: q.append(popnode.right)
        level+=1 #level increase
    return level

root = [3,9,20,None,None,15,7]
mtree = to_tree(root)
print(maxDepth(mtree))

tree: {3 9 20 null null 15 7 null null null null }
3


#### [543. Diameter of Binary Tree](https://leetcode.com/problems/diameter-of-binary-tree/)

***Brainstorm**: DFS gives us:  Each iter, we all can attain depth of two subtrees:
**Lchild.depth, Rchild.depth**

- p.depth = max(Lchild.depth,Rchild.depth) + 1
- p.diameter = Lchild.depth + Rchild.depth


Use DFS to find the max diameter of each node (subtree)
1. Go down to bottom left node. Start cal from that node.
2. Cal depth of l, r of that node.
3. Cal diamter of each node, compared to current maxdiameter

(DFS recursive): l, r = dfs(left), dfs(right)


In [ ]:
def diameterOfBinaryTree(root: Optional[TreeNode]) -> int:
     #global max variable
    maxDmter = 0
    
    #find height of any node AND continously update maxDiameter
    def dfs(root):
        if not root:
            return 0 #null height is -1
        
        # count this node's diameter -> compared to global max
        nonlocal maxDmter
        maxDmter = max(maxDmter, dfs(root.left) + dfs(root.right))
        
        return 1 + max(dfs(root.left), dfs(root.right)) #depth of node
    
    dfs(root)
    return maxDmter
root = [1,2,3,4,5,None,None]
mtree = to_tree(root)
print(diameterOfBinaryTree(mtree))

0


#### [110. Balanced Binary Tree](https://leetcode.com/problems/balanced-binary-tree/)

A height-balanced binary tree: depth of the two subtrees of every node never differs by more than one.

**Learned**: More optimal because keep returning -1 when finish, Look at **count** below

- Keep returning -1 will not bother this line return 1 + max(left, right) because:
- When -1 is assigned to eather left, right. The next step is keep returning -1 to the prev assigned parent (or left, right of grandparent again) and

and keep returning -1 instead of keep cal abs and 1+max.


In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def isBalanced(self, root: Optional[TreeNode]) -> bool:
        
        def dfs(root):
            if not root:
                return 0
            left = dfs(root.left)
            if left == -1:
                return -1
            right = dfs(root.right)
            if right == -1:
                return -1
            
            if abs(left - right) > 1:
                return -1

            return 1 + max(left, right)    
        return dfs(root) != -1

In [ ]:
def isBalanced(root: Optional[TreeNode]) -> bool:
        res = True
        def maxDepth(root):
            if not root:
                return 0
            #get l, r depth
            l = maxDepth(root.left)
            r = maxDepth(root.right)
            if abs(l - r) > 1:
                nonlocal res
                res = False
                return 0
            return 1 + max(l,r)
        maxDepth(root)
        return res

root = [1,2,2,3,3,None,None,4,4, None, None, 5,5, None, None, 6,6, None, None, 7,7,None, None, 8,8]
mtree = to_tree(root)
print(isBalanced(mtree))

Backtracking count 7
False


#### [100. Same Tree](https://leetcode.com/problems/same-tree/)

**Top down approach**: Check from the root to l, r children.
- Only keep going down if not found unmatched yet.

In [ ]:
def isSameTree(p: Optional[TreeNode], q: Optional[TreeNode]) -> bool:
    if not p and not q: #equal if both NULLs
        return True
    if p and q and p.val == q.val: #only go down if equal
        
        return isSameTree(p.left, q.left) and isSameTree(p.right, q.right)
    
    #assign false to parent otherwise, stop going down here.
    return False 

True


**DFS approach**: Bottom up: check equal subtrees and pop up

In [ ]:
def isSameTree(p: Optional[TreeNode], q: Optional[TreeNode]) -> bool:
    
        if not p and not q:
            return True
        if p and q:
            l = isSameTree(p.left, q.left)
            if not l:
                return False
            r = isSameTree(p.right, q.right)
            if not r:
                return False
            return p.val == q.val
        
        return False


True


#### [572. Subtree of Another Tree](https://leetcode.com/problems/subtree-of-another-tree/description/)

Fastest: O(n+m+n) ~ Linear
1. DFS rec: get string traversal of sub and tree. check if sub in tree
Slower: O(mn)
2. DFS rec: While traversing dfs, check if subtree of each node in **root** is isSameTree as **subRoot**

In [ ]:
#Approach 1 DFS: O(n+m+n), less stack call 
def isSubtree(root: TreeNode, subRoot: TreeNode) -> bool:
    rootS = getStr(root)
    subS = getStr(subRoot)
    
    if subS in rootS:
        return True
    
    return False

def getStr(root):
    if not root:
        return " NUll "
    return str(root.val) + " "+  getStr(root.left) + getStr(root.right)


In [ ]:
#Approach 2: O(mn), more stack call, twice dfs
def isSubtree(root: TreeNode, subRoot: TreeNode) -> bool:
    if not subRoot:
        return True
    if not root:
        return False
    if isSameTree(root, subRoot):
        return True
    
    return isSubtree(root.left, subRoot) or isSubtree(root.right, subRoot)
    
    return False 

def isSameTree(q,p) -> bool:
    if not q and not p:
        return True
    
    if p and q and p.val == q.val:
        return isSameTree(p.left,q.left) and isSameTree(p.right,q.right) 
    
    return False #False otherwise    
root = [1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,2]
subRoot = [1,None,1,None,1,None,1,None,1,None,1,2]

print(isSubtree(to_tree(root), to_tree(subRoot)))

True


In [ ]:
#Approach 2: O(mn), DFS Iter
def isSubtree(root: TreeNode, subRoot: TreeNode) -> bool:
    stack = [root]
    while stack:
        popn = stack.pop()
        
        #code
        if isSameTree(popn, subRoot): return True
        
        if popn: 
            stack.append(popn.right)
            stack.append(popn.left)
        
    return False

def isSameTree(q,p) -> bool:
    if not q and not p:
        return True
    
    if p and q and p.val == q.val:
        return isSameTree(p.left,q.left) and isSameTree(p.right,q.right) 
    
    return False #False otherwise    
root = [1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,2]
subRoot = [1,None,1,None,1,None,1,None,1,None,1,2]

print(isSubtree(to_tree(root), to_tree(subRoot)))

True


### Medium

#### [235. Lowest Common Ancestor of a Binary Search Tree](https://leetcode.com/problems/lowest-common-ancestor-of-a-binary-search-tree/)

In [ ]:
#since we traverse FROM ROOT, (smaller <= node.val and node.val <= greater) is enough to make node the lowest parent.
#DFS REC
def lowestCommonAncestor(root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
    greater = max( p.val, q.val)
    smaller = min(q.val, p.val) 
    def dfsInRange(root): 
        if smaller <= root.val and root.val <= greater:
            return root
        elif root.val > greater:
            return dfsInRange(root.left) 
        elif root.val < smaller:
            return dfsInRange(root.right) 
    return dfsInRange(root)
#Regular ITER
def lowestCommonAncestor(root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
    greater = max( p.val, q.val)
    smaller = min(q.val, p.val) 
    cur = root
    while cur:
        if smaller <= cur.val and cur.val <= greater:
            return cur
        elif cur.val > greater:
            cur = cur.left
        elif cur.val < smaller:
            cur = cur.right 
    return cur

#### [236. Lowest Common Ancestor of a Binary Tree](https://leetcode.com/problems/lowest-common-ancestor-of-a-binary-tree)

In [ ]:
def lowestCommonAncestor(root: TreeNode, p: TreeNode, q: TreeNode) -> TreeNode:
        if not root or root == p or root == q:
            return root

        left = lowestCommonAncestor(root.left, p, q)
        right = lowestCommonAncestor(root.right, p, q)

        if left and right:
            return root
        return left if left else right

root = [3,5,1,6,2,0,8,None,None,7,4], 
p = 5, q = 1
lowestCommonAncestor(to_tree(root), TreeNode(p), TreeNode(q) )

In [ ]:
#iterative using stack
def lowestCommonAncestor(root: TreeNode, p: TreeNode, q: TreeNode) -> TreeNode:

    # Stack for tree traversal
    stack = [root]

    # Dictionary for parent pointers, childKey : arentNode
    parent = {root: None}

    # Iterate until we find both the nodes p and q
    while p not in parent or q not in parent:

        node = stack.pop()

        # While traversing the tree, keep saving the parent pointers.
        if node.left:
            parent[node.left] = node
            stack.append(node.left)
        if node.right:
            parent[node.right] = node
            stack.append(node.right)

    # Ancestors set() for node p.
    ancestors = set()

    # Process all ancestors for node p using parent pointers.
    while p:
        ancestors.add(p)
        p = parent[p]

    # The first ancestor of q which appears in
    # p's ancestor set() is their lowest common ancestor.
    while q not in ancestors:
        q = parent[q]
    return q

#### [102. Binary Tree Level Order Traversal](https://leetcode.com/problems/binary-tree-level-order-traversal/)

In [ ]:
from collections import deque
#BFS
def levelOrder(root: Optional[TreeNode]) -> list[list[int]]:

    if root:
        queue = deque([root])
    else: return []
    res = []
    while queue:
        temp = [] #new list each level
        #for each node in the current level
        for i in range(len(queue)):
            node = queue.popleft()  
            temp.append(node.val)
            if node.left:
                queue.append(node.left)
            if node.right: 
                queue.append(node.right)
        #after each level
        res.append(temp)
    return res
root = to_tree([3,9,20,None,None,15,7])
print(levelOrder(root))
    

[[3], [9, 20], [15, 7]]


#### [199. Binary Tree Right Side View](https://leetcode.com/problems/binary-tree-right-side-view/)

In [ ]:
from collections import deque
def rightSideView(root: Optional[TreeNode]) -> list[int]:
        if not root:
            return []

        res = [root.val]
        queue = deque([root])
        while queue:
            for i in range(len(queue)):
                popnode = queue.popleft() #the rightest node in the queue
                if popnode.right:
                    queue.append(popnode.right)
                if popnode.left:
                    queue.append(popnode.left)

            if queue:
                #On each level, append the rightest (first) node in the queue.
          
                res.append(queue[0].val)
        return res
root = to_tree([1,2,3,None,5,None,4])
print(rightSideView(root))

[1, 3, 4]


#### [1448. Count Good Nodes in Binary Tree](https://leetcode.com/problems/count-good-nodes-in-binary-tree/)

DFS Rec approach
1. Each DFS, with record the current max of the traverse path of the node. Compared node with its max to incre **good**
2. Curmax of both left and right subtree is curmax of node.

In [ ]:
def goodNodes(root: TreeNode) -> int: 
    good = 0    
    def dfsMax(node, curmax):
        if not node:
            return 0
        
        if node.val >= curmax:
            curmax = node.val
            nonlocal good
            good+=1
        
        #curmax of node feed to left, right children
        dfsMax(node.left, curmax)
        dfsMax(node.right, curmax)
    
    dfsMax(root,root.val)
    
    return good 
     
root = to_tree([3,1,4,3,None,1,5] )
goodNodes(root)

4

In [ ]:
def goodNodes(root: TreeNode) -> int: 
    #a dfs that have curNode and max of the traverse path
    def dfs(node, mmax):
        if not node:
            return 0
        goodCounts = 0
        if node.val >= mmax:
            goodCounts = 1
            mmax = node.val   
        goodCounts += dfs(node.left, mmax)
        goodCounts += dfs(node.right,mmax)
        
        return goodCounts
    return dfs(root, root.val)
        
        

#### [98. Validate Binary Search Tree](https://leetcode.com/problems/validate-binary-search-tree/)
1. Recursive dfs: Idea of compare node's subtrees with itself
2. Pass the range (min, max) to each node where:
- For node.left, min is bounded by node's min, max = node
- For node.right, max is bounded by node's max, min = node

In [ ]:
def isValidBST(self, root: Optional[TreeNode]) -> bool:
    def dfs(node, nmin, nmax) -> bool:
        if not node:
            return True #if could reach base case eventually, return True and exit.
        if not (nmin < node.val < nmax):
            return False
        #Pass node's min and node's max to check children after check self.
        return dfs(node.left, nmin, node.val) and dfs(node.right, node.max, nmax)
    return dfs(root, float("-inf"), float("inf"))

#Iter stack version
def iterative(root):
    if not root: return True
    stack = [(root, -float('inf'), float('inf'))]
    while len(stack):
        node, nmin, nmax = stack.pop()
        if not (nmin < node.val < nmax): return False
        if node.left: stack.append((node.left, nmin, node.val))
        if node.right: stack.append((node.right, node.val, nmax))
    return True

#### [230. Kth Smallest Element in a BST](https://leetcode.com/problems/kth-smallest-element-in-a-bst/)

The smallest -> largest in BST is inOrder traversal: Left - self - right

**Recursive thinking:**
- We go left, self, right where we only perform at "self". Left, and right passed to is just travelling

In [ ]:
def kthSmallest(root: Optional[TreeNode], k: int) -> int:
    count = 0
    res = 0
    def dfsInOrder(node):
        if not node:
            return
        nonlocal count, res
        dfsInOrder(node.left) 
        
        #perform code
        if count == -1: #help exit faster
            return
        count+=1
        if count == k:
            res = node.val
            count = -1
            return 

        dfsInOrder(node.right)
        
    dfsInOrder(root) 
    return res

**Iter thinking**: cur = root will be the traveller.
Use 

    while cur:
            stack.append(cur) #all e in stack is not null
            cur = cur.left
            
In the begining of each loop to get the leftest of a node.

In [ ]:
def kthSmallest(root: Optional[TreeNode], k: int) -> int:
    count = 0
    stack = [root]
    cur = root
    while stack or cur: #incase stack empty but cur right is working
        #go all the way to bottom left before performing code
        while cur:
            stack.append(cur) #all e in stack is not null
            cur = cur.left
        #after while loop, cur is always None to the left.
        cur = stack.pop() 
        #perform at node
        count+=1
        if count == k:
            return cur.val
        #go to rights
        cur = cur.right

#### [105. Construct Binary Tree from Preorder and Inorder Traversal](https://leetcode.com/problems/construct-binary-tree-from-preorder-and-inorder-traversal/)

True Premises about arrays:
- Preorder: Top is root -> left subtree -> right subtree
- Inorder: left subtree -> root(midIndex) -> right subtree
Recursion DFS approach:
- Get curr node and its inorder index: root = preorder[0], midI = inorder.index(preorder[0])
- Assign left, right subtree with:
    - preorder_left = preorder[1:mid+1], preorder_right = preorder[mid+1:], 
    - inorder_left = inorder[:midI], inorder_right = inorder[mid+1:] (exclude midI)
- Return curr node


In [ ]:
class Solution:
    def buildTree(self, preorder: list[int], inorder: list[int]) -> Optional[TreeNode]:
        if not preorder or not inorder:
            return None
        root = TreeNode(preorder[0])
        midI = inorder.index(preorder[0])
        #DFS: Go to down leftest
        root.left = self.buildTree(preorder[1:midI+1], inorder[:midI])
        root.right = self.buildTree(preorder[midI+1:], inorder[midI+1:])
        return root

#### [114. Flatten Binary Tree to Linked List](https://leetcode.com/problems/flatten-binary-tree-to-linked-list/description/?envType=company&envId=paypal&favoriteSlug=paypal-all)

In [ ]:
#Recursive
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution(object):
    def flatten(self, root):
        """
        :type root: Optional[TreeNode]
        :rtype: None Do not return anything, modify root in-place instead.
        """
        self.flatTail(root)
    # Flatten tree + Return RIGHTMOST TAIL
    def flatTail(self,node):
        if not node:
            return None
        if not node.left and not node.right: 
            return node
        
        leftTail = self.flatTail(node.left)
        righTail = self.flatTail(node.right)
        
        if leftTail:
            leftTail.right = node.right
            node.right = node.left
            node.left = None
        return righTail if righTail else leftTail
    

In [ ]:
#Iterative using stack
from collections import deque
def flatten(root: Optional[TreeNode]) -> None:

    # Handle the null scenario
    if not root:
        return None

    START, END = 1, 2

    tailNode = None
    stack = deque([(root, START)])

    while stack:
        currentNode, recursionState = stack.pop()
        # We reached a leaf node. Record this as a tail
        # node and move on.
        if not currentNode.left and not currentNode.right:
            tailNode = currentNode
            continue

        # If the node is in the START state, it means we still
        # haven't processed it's left child yet.
        if recursionState == START:

            # If the current node has a left child, we add it
            # to the stack AFTER adding the current node again
            # to the stack with the END recursion state.
            if currentNode.left:
                stack.append((currentNode, END))
                stack.append((currentNode.left, START))
            elif currentNode.right:
                # In case the current node didn't have a left child
                # we will add it's right child
                stack.append((currentNode.right, START))
        else:
            # If the current node is in the END recursion state,
            # that means we did process one of it's children. Left
            # if it existed, else right.
            rightNode = currentNode.right

            # If there was a left child, there must have been a leaf
            # node and so, we would have set the tailNode
            if tailNode:
                # Establish the proper connections.
                tailNode.right = currentNode.right
                currentNode.right = currentNode.left
                currentNode.left = None
                rightNode = tailNode.right

            if rightNode:
                stack.append((rightNode, START))

#### [863. All Nodes Distance K in Binary Tree](https://leetcode.com/problems/all-nodes-distance-k-in-binary-tree)

In [ ]:
def distanceK(root: TreeNode, target: TreeNode, k: int) -> List[int]:
    # Recursively add a parent pointer to each node.
    def add_parent(cur, parent):
        if cur:
            cur.parent = parent
            add_parent(cur.left, cur)
            add_parent(cur.right, cur)

    add_parent(root, None)

    answer = []
    visited = set()

    def dfs(cur, distance):
        if not cur or cur in visited:
            return
        visited.add(cur)
        if distance == 0:
            answer.append(cur.val)
            return
        dfs(cur.parent, distance - 1) 
        dfs(cur.left, distance - 1)
        dfs(cur.right, distance - 1)

    dfs(target, k)

    return answer

#### [2385. Amount of Time for Binary Tree to Be Infected](https://leetcode.com/problems/amount-of-time-for-binary-tree-to-be-infected/)

In [ ]:
#turn to graph then bfs
def amountOfTime(root: Optional[TreeNode], start: int) -> int:
    def dfs(node):
        if node is None:
            return
        if node.left:
            graph[node.val].append(node.left.val)
            graph[node.left.val].append(node.val)
        if node.right:
            graph[node.val].append(node.right.val)
            graph[node.right.val].append(node.val)
        dfs(node.left)
        dfs(node.right)

    graph = defaultdict(list)
    dfs(root)
    visited = set()
    queue = deque([start])
    time = -1
    while queue:
        time += 1
        for _ in range(len(queue)):
            current_node = queue.popleft()
            visited.add(current_node)
            for neighbor in graph[current_node]:
                if neighbor not in visited:
                    queue.append(neighbor)
    return time

In [239]:
#dfs, maxlength is minutes. infected subtree got negative value and propagate negative to parent

def amountOfTime(root: Optional[TreeNode], start: int) -> int:
    res = 0  # Stores the maximum time required for infection to spread
    def DFS(node, start):
        nonlocal res
        if node is None:
            return 0  # Base case: If node is null, return depth 0
        
        # Recursively get the depth of left and right subtrees
        leftDepth = DFS(node.left, start)
        rightDepth = DFS(node.right, start)
        
        if node.val == start:
            # If current node is the infection source, update result with max subtree depth
            res = max(leftDepth, rightDepth)
            return -1  # Mark infection source to track spread
        
        elif leftDepth >= 0 and rightDepth >= 0:
            # If neither subtree contains the infection source, return max depth +1
            return max(leftDepth, rightDepth) + 1
        
        # If infection source is in one subtree, calculate max spread distance
        res = max(res, abs(leftDepth - rightDepth))
        
        # Return the negative depth to propagate infection path upwards
        return min(leftDepth, rightDepth) - 1
    
    DFS(root, start)  # Start DFS from the root node
    return res  # Return the maximum infection spread time
root = [1,5,3,None,4,10,6,9,2]
start = 3
amountOfTime(to_tree(root), start)

4

## Tries

### Med

#### [208. Implement Trie (Prefix Tree)](https://leetcode.com/problems/implement-trie-prefix-tree/submissions/)

In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {} #children hashmap
        self.isWord = False
class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word: str) -> None:
        cur = self.root
        for c in word:
            if c not in cur.children:
                cur.children[c] = TrieNode()
            cur = cur.children[c] #move to appropriate child node
        cur.isWord = True
    def search(self, word: str) -> bool:
        cur = self.root
        for c in word:
            if c not in cur.children:
                return False
            cur = cur.children[c]
        #after check if last TrieNode isWord after consuming word
        if not cur.isWord:
            return False
        return True
    def startsWith(self, prefix: str) -> bool:
        cur = self.root
        for c in prefix:
            if c not in cur.children:
                return False
            cur = cur.children[c]
        return True

#### [211. Design Add and Search Words Data Structure](https://leetcode.com/problems/design-add-and-search-words-data-structure/)

In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {} #children hashmap
        self.isWord = False
    def __repr__(self) -> str:
        return "Children: " + str(self.children.keys()) + str(self.isWord)
       
class WordDictionary:
    def __init__(self):
        self.root = TrieNode()
    def addWord(self, word: str) -> None:
        cur = self.root
        for c in word:
            if c not in cur.children:
                cur.children[c] = TrieNode()
            cur = cur.children[c]
        cur.isWord = True
    def search(self, word: str) -> bool:
        print(self.root.children)
        def dfs(word, root):
            cur = root
            for i in range(len(word)):
                if word[i] == '.':
                    for child in cur.children:
                        if dfs(word[i+1:], cur.children[child]): return True
                    return False       
                else:
                    if word[i] not in cur.children:
                        return False
                    cur = cur.children[word[i]]
            
            return cur.isWord == True
        
        return dfs(word, self.root)
    
wordDictionary = WordDictionary()

op = ["WordDictionary","addWord","addWord","addWord","addWord","search","search","addWord","search","search","search","search","search","search"]
param = [[],["babaa"],["and"],["an"],["add"],["a"],[".at"],["bat"],[".at"],["an."],["a.d."],["b."],["a.d"],["."]]
res = []
for i in range(1, len(op)):
    o, p = op[i], param[i]
    if o == "addWord":
        res.append(wordDictionary.addWord(param[i][0]))
    elif o == "search":
        res.append(wordDictionary.search(param[i][0]))
print(res)


### Hard

#### [212. Word Search II](https://leetcode.com/problems/word-search-ii/)

Approach: Search all words in the list at once by traveling each square in board, then dfs from the square outwards to see if the dfs path could check any word in the list.
1. Add all searching words to a Trie.
2. Use "refs" attribute for each nodes to represents # of chars in words shared a part of Trie path
<img src="assets/wordsearchii.png" width=500px></img>

3. For each square in board, DFS each square to find most suitable words at once.
    - Start with DFS (r,c, root, ""). String "" is word gained so far. 
    - On DFS: DFS up down left right square to find if each direction (on board) matches with any of current node's children (on Trie). Each time move deeper in DFS, current node = appropriate child node.
    - If word gain so far isWord, add to res, remove from Trie.
Complexity:
- If we brute force without using a trie

In [27]:
class TrieNode:
    def __init__(self) -> None: 
        self.children = {}
        self.ref = 0
        self.isWord = False
class Trie:
    def __init__(self) -> None:
        self.root = TrieNode()
    def addWord(self, word:str) -> None:
        cur = self.root
        for c in word:
            if c not in cur.children:
                cur.children[c] = TrieNode()
            cur = cur.children[c]
            cur.ref += 1 #incre the child's refs
        cur.isWord = True
        
    def removeWord(self, word:str)->None:
        cur = self.root
        for c in word:
            cur = cur.children[c]
            cur.ref -=1 #decre the child's refs
        cur.isWord = False 
    
class Solution:
    def findWords(self,board: list[list[str]], words: list[str]) -> list[str]:
        mtrie = Trie()
        for word in words:
            mtrie.addWord(word)
        res, visit = set(), set()
        ROW, COL = len(board), len(board[0])
        
        def dfssearch(r, c, node, formingWord):
            if (r not in range(ROW) or c not in range(COL) #out of bound 
            or (r, c) in visit #visited
            or board[r][c] not in node.children #not the right char square
            or node.children[board[r][c]].ref < 1): #if already found and removed
                return
             
            visit.add((r,c))
            formingWord += board[r][c]
            node = node.children[board[r][c]]
            
            if node.isWord:
                res.add(formingWord)
                mtrie.removeWord(formingWord)
            
            dfssearch(r+1, c, node, formingWord)
            dfssearch(r-1, c, node, formingWord)
            dfssearch(r, c+1, node, formingWord)
            dfssearch(r, c-1, node, formingWord)
            
            visit.remove((r,c))
        for r in range(ROW):
            for c in range(COL):
                dfssearch(r,c, mtrie.root, "")
        return list(res)

board = [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]]
words = ["oath","pea","eat","rain"]

# Example usage
board = [
    ['C', 'A', 'T'],
    ['I', 'D', 'O'],
    ['N', 'O', 'M']
]

words = ["CAT", "TOM", "ADO", "MOM"]
print(Solution.findWords(Solution, board=board, words=words))

['CAT', 'ADO', 'TOM']


## Backtracking
<img src="assets/backtrackrec.png" width="250ptx"></img>

### Medium

#### [46. Permutations](https://leetcode.com/problems/permutations/)
<img src="assets/46. Permutations.png" width="300px"> </img>

Optimal time+space approach:
- Get perms from remain (nums.pop(0)), then append n=nums.pop(0) to each perm, extend perms to res
- End of the loop: nums.append(n) # because nums is passed as reference, we have to append n to the back. Or else nums would be pop all -> can't move to next of " for _ in nums:"



In [232]:
class Solution:
    def permute(self, nums: list[int]) -> list[list[int]]:
        # print(nums)
        res = []
        if len(nums) == 1:
            return [nums[:]]

        for _ in nums:
            n = nums.pop(0)
            perms = self.permute(self,nums)
            for perm in perms:
                # print("append n", n)

                perm.append(n)
            # print("res ext", res, perms)

            res.extend(perms)

            nums.append(n) # because nums is passed as reference, we have to append n to the back
        return res
nums = [1,2,3,4,5]
print(Solution.permute(Solution, nums))

In [ ]:
class Solution:
    def permute(self, nums: list[int]) -> list[list[int]]:
        res = []
        
        def dfs(cur, remaining):
            if not remaining:
                res.append(cur[:])
                return
            
            # Explore all possibilities by adding each element in remaining to cur
            for i in range(len(remaining)):
                cur.append(remaining[i])
                dfs(cur, remaining[:i] + remaining[i+1:])
                cur.pop()
        
        dfs([], nums)
        return res

#### [78. Subsets](https://leetcode.com/problems/subsets/)

<img src="assets/l78.jpg" width=400px></img>

In [ ]:
%%time
class Solution:
    def subsets(self, nums: list[int]) -> list[list[int]]:
        res = [[]]        
        def dfs(nums, subset):
            
            for i in range(len(nums)):
                subset.append(nums[i])
                
                res.append(subset[:])
                dfs(nums[i+1:], subset)

                subset.pop()
         
        dfs(nums, [])

        return res
nums = [n for n in range(1,12)]
Solution.subsets(Solution, nums)

Neet approach:
DFS decision tree to choose either append or not append nums[i]
<img src="assets/subset.png" width=400px></img>

In [ ]:
%%time
class Solution1:
    def subsets(self, nums: list[int]) -> list[list[int]]:
        res = []
        subset = []
        def dfs(i):
            #base case: reach the end of decision tree (completed set)
            if i >= len(nums):
                res.append(subset[:]) #add copy to keep modifying subset
                return 
            
            #decision to choose nums[i]
            subset.append(nums[i])
            dfs(i+1)
            #decision not to choose nums[i]
            subset.pop()
            dfs(i+1) #go to next level(i+1) without nums[i]
        dfs(0)
        return res
nums = [n for n in range(1,12)]
print(Solution1.subsets(Solution1, nums))

#### [39. Combination Sum](https://leetcode.com/problems/combination-sum/)
The backtracking decision tree to avoid duplicates candidates:
- Suppose decision paths through candidates = [2,3,6,7]
- Keep move from i, i or to i+1 in candidates, don't look back to avoid duplicates
    - The first paths that have first pivot elements, later path never contains previous pivot elements 
    - (Path of 3 never contains 2, path of 7 never contains 2,3,...)

<img src="assets/39. Combination Sum.png" width=500px></img>

In [ ]:
class Solution:    
    def combinationSum(self, candidates: list[int], target: int) -> list[list[int]]:
        res = []
        cur = []
        #i: index of num in candidates; cur: current comb
        #(II): This function process i before new added cur, and added total
        #And process its next index i+1 that never consider i again
        def dfs(i, total):
            if total == target:
                res.append(cur[:])
                return
            if total > target or i >= len(candidates):
                return
            
            cur.append(candidates[i])
            #Process stage: pass i itself after the first pass at (II)
            #Process i with new added cur and new total

            dfs(i, total+candidates[i]) #(I)
            
            #Next index stage: pass i+1, would never consider i
            cur.pop() #pop what just added (candidates[i])
            
            #Note** Do not add candidates[i+1] to total
            dfs(i+1, total) 
            return
        dfs(0, 0)
        return res
candidates = [2,3,6,1]
target = 7
Solution.combinationSum(Solution, candidates, target)


**Note:  because we want to pass i+1 to be processed (the process of next index will be at line (I)), not process it now. "To-be-process" so that i+1 is treated like i, where it got processed, and its +1 (i+2 this case) got processed later. This recursively occur to i+2 to be processed so i+3 will be processsed after that.


In [ ]:
#we only move forward, never repeat previous index bc `index` variable is the left boundary block that
class Solution:
    def combinationSum(self, candidates: list[int], target: int) -> list[list[int]]:
        res = []
        n = len(candidates)
        def backtrack(index, cur, total):
            if total == target:
                res.append(cur[:]) 
                return
            if total > target:
                return
            #Loop from INDEX, to n => always move forward
            for i in range(index, n):
                cur.append(candidates[i])  # Include the candidate
                # Process updated total and the same index `i` (can reuse the same candidate)
                backtrack(i, cur, total + candidates[i])
                # Backtrack by removing the last added element
                cur.pop()
        
        backtrack(0, [], 0)
        return res
#Time: O(N^ (M/T) +1  ): Max nodes of depth of the tree:T/M 
#Space:(M/T) number of recursive calls

#### [40. Combination Sum II](https://leetcode.com/problems/combination-sum-ii/description/s)

In [5]:
class Solution:
    def combinationSum2(self, candidates: list[int], target: int) -> list[list[int]]:
        res = []

        n = len(candidates)
        candidates.sort()
        def dfs(index, total, cur):
            if total == target:
                res.append(cur[:])
                return
            
            if index >= n or total > target:
                return
            
            # Explore each candidate starting at 'index'
            for i in range(index, n):
                # Skip duplicates: if the current candidate is the same as the previous one 
                # AND we're at the same recursive level (i.e., i - 1 >= index),
                # then skip this candidate to avoid duplicate combos. Already DONE building a comb start with that num.
                if i > index and candidates[i] == candidates[i - 1]:
                    continue
                
                # Choose candidates[i] and move forward
                cur.append(candidates[i])
                # Move 'i+1' because each number can be used only once
                dfs(i + 1, total + candidates[i], cur)
                # Backtrack: remove the last added number
                cur.pop()
        
                
        dfs(0,0, [])
        return res
#Time: 2^N: each number is included or excluded in a combination.
candidates = [2,3,2, 6,1]
target = 7
Solution.combinationSum2(Solution, candidates, target) 

[[2, 3, 2], [6, 1]]

#### [79. Word Search](https://leetcode.com/problems/word-search/)
O(n * m * 4^n)

- For time optimization: reverse the word if frequency of the first letter is higher than the last
- This reduces DFS search from multiple same chars in board in the beginning.

In [ ]:
from collections import Counter
class Solution:
    def exist(self, board: list[list[str]], word: str) -> bool:
        visit = set()
        ROW, COL = len(board), len(board[0])
        
        def dfs(r,c, word)->bool:
            #BASE CASE of TRUE/FALSE
            if len(word) ==0:
                return True
            
            if ((r,c) in visit
                or not (0 <= r and r <ROW)
                or not (0 <= c and c <COL) 
                or board[r][c] != word[0]):
                return False
            
            visit.add((r,c))
            print("found next at", board[r][c])

            if (dfs(r+1, c, word[1:])
            or dfs(r-1, c, word[1:])
            or dfs(r, c+1, word[1:])
            or dfs(r, c-1, word[1:])):
                return True

            visit.remove((r,c))
            return False
        #Reverse if count[word[0]] > count[word[-1]]
        c = Counter(word)
        if c[word[0]] > c[word[-1]]:
            word = word[::-1]
        
        for i in range(ROW):
            for j in range(COL):
                if dfs(i, j, word):
                    return True
        
        return False
board = [["A","B","C","E"],["S","F","C","S"],["A","D","E","E"]]
word = "ABCCED"
Solution.exist(Solution, board, word)

found next at A
found next at B
found next at C
found next at C
found next at E
found next at D


True

#### [131. Palindrome Partitioning](https://leetcode.com/problems/palindrome-partitioning/)

OG approach:
- Pass in word to partition
- Loop i-> wordlen: Check if word[:i] is Pali, if yes: pass Parti(word[i+1:])

<img src="assets/131. Palindrome Partitioning.png" width=400px></img>
<img src="assets/131. PP.png" ></img>

In [ ]:
class Solution:
    def partition(self, s: str) -> list[list[str]]:
        res = []
        cur = []
        
        def dfspart(i):
            if i >= len(s):
                res.append(cur[:])
                return
            for j in range(i,len(s)):
                pre = s[i:j+1]
                if pre == pre[::-1]:
                    cur.append(pre)
                    dfspart(j+1)
                    #after going down all the way in dfspart, 
                    #remove pre from cur to append another sibling "pre" in the for loop
                    cur.pop()
                
        dfspart(0)
        return res
s = "cbbbcc"
Solution.partition(Solution, s)

[['c', 'b', 'b', 'b', 'c', 'c'],
 ['c', 'b', 'b', 'b', 'cc'],
 ['c', 'b', 'bb', 'c', 'c'],
 ['c', 'b', 'bb', 'cc'],
 ['c', 'bb', 'b', 'c', 'c'],
 ['c', 'bb', 'b', 'cc'],
 ['c', 'bbb', 'c', 'c'],
 ['c', 'bbb', 'cc'],
 ['cbbbc', 'c']]

#### [17. Letter Combinations of a Phone Number](https://leetcode.com/problems/letter-combinations-of-a-phone-number/)

In [33]:
class Solution:
    def letterCombinations(self, digits: str) -> list[str]:
        #first thought: do for nested loop, but don't know how many times => backtracking
        #2 3 4 5
        mmap = {"2": "abc", "3": "def", "4":"ghi", "5":"jkl", "6":"mno", "7": "pqrs", "8":"tuv", "9":"wxyz"}

        def backtracking(digits):
            if len(digits) == 0:
                return []
            if len(digits) == 1:
                return list(mmap[digits[0]])
            cur = backtracking(digits[1:])
            temp = []
            for comb in cur:
                for letter in list(mmap[digits[0]]):
                    temp.append(letter+ comb)

            return temp
        return backtracking(digits)

#### [90. Subsets II](https://leetcode.com/problems/subsets-ii/)

In [ ]:
class Solution:
    def subsetsWithDup(self, nums: list[int]) -> list[list[int]]:
        res = []
        cur = []
        nums.sort()
        def dfs(i):
            if i >= len(nums):
                
                res.append(cur[:])
                return
            
            #if choose nums[i], go dfs
            cur.append(nums[i])
            dfs(i+1)
            #if not choose nums[i]
            cur.pop()
            count = 0
            #after pop(), don't dfs nums[i+1] if , skip i to end of same element sequence
            while i+1 < len(nums) and nums[i] == nums[i+1]:
                count+=1
                i+=1
            
            #dfs without num[i]
            dfs(i+1)
            
        dfs(0)
        
        return res
nums = [1,2,3,2,2,2,3,4,4]
# print(Solution.subsetsWithDup(Solution, nums))    

2056. Number of Valid Move Combinations On Chessboard

In [ ]:
from typing import List

class Solution:
    def countCombinations(self, pieces: List[str], positions: List[List[int]]) -> int:
        # Define movement directions for each piece type
        d_rook = [(1, 0), (-1, 0), (0, 1), (0, -1)]  # Vertical & horizontal moves
        d_bishop = [(1, 1), (1, -1), (-1, 1), (-1, -1)]  # Diagonal moves
        d_queen = d_rook + d_bishop  # Queen can move like both rook and bishop
        
        # Assign movement directions based on piece type
        directions = [
            d_rook if piece == 'rook' else 
            d_bishop if piece == 'bishop' else 
            d_queen for piece in pieces
        ]
        
        # Define board size and max steps allowed for movement
        m = n = max_step = 8  # Standard chessboard size
        
        # Track occupied steps for each position on the board
        occupied_by_step = [[[] for _ in range(n)] for _ in range(m)]
        end_step = [[max_step] * n for _ in range(m)]
        
        def backTrack(i):
            """Recursive backtracking function to explore all move combinations."""
            if i == len(pieces):  # Base case: all pieces have moved
                return 1
            
            res = 0  # Count of valid move combinations
            x, y = positions[i][0] - 1, positions[i][1] - 1  # Convert position to 0-based index
            
            # Case 1: The piece does not move
            if end_step[x][y] == max_step and not occupied_by_step[x][y]:
                end_step[x][y] = 0  # Mark as occupied at step 0
                res += backTrack(i + 1)
                end_step[x][y] = max_step  # Reset after backtracking
            
            # Case 2: The piece moves in one of its allowed directions
            for r, c in directions[i]:
                x, y = positions[i][0] - 1, positions[i][1] - 1  # Reset position
                visit = []  # Track visited positions
                
                for step in range(1, max_step):  # Move up to max_step
                    x, y = x + r, y + c  # Move in the given direction
                    
                    # Check if the move is within bounds and does not conflict with another piece
                    if 0 <= x < m and 0 <= y < n and step not in occupied_by_step[x][y] and step < end_step[x][y]:
                        last_occupied_step = max(occupied_by_step[x][y] or [0])  # Track last occupied step
                        occupied_by_step[x][y].append(step)  # Mark step as occupied
                        visit.append((x, y))
                        
                        # Stop at this position and proceed with the next piece
                        if end_step[x][y] == max_step and step > last_occupied_step:
                            end_step[x][y] = step
                            res += backTrack(i + 1)
                            end_step[x][y] = max_step  # Reset after backtracking
                    else:
                        break  # Stop moving in this direction
                
                # Backtrack: Remove occupied steps from visited positions
                for x, y in visit:
                    occupied_by_step[x][y].pop()
            
            return res
        
        return backTrack(0)  # Start exploring moves from the first piece


## [Heap / Priority Queue](https://leetcode.com/problems/kth-largest-element-in-a-stream/)

- **Heap**: A tree-based, implement PQ (often stored in an array) where each parent satisfies (parent ≤ children min-heap).  
- **Left, right, parent child**: \(2i + 1\), \(2i + 2\), (i - 1)//2 
- **Push** and **pop** in **O(\(\log n\))**
- **Common Patterns**:  
  - **Top K** problems (kth largest or smallest)  
      - Lots of Kth problems can be solved by 
         - **Quick Select**, or modified **binary search on selected threshold for unsorted array** 
         - avg O(n) by roughly halving the total number each iteration, worst O(n^2)

  - **Merging** multiple sorted lists or data streams  
  - **Scheduling** or event simulation (process highest or lowest priority first)  
  - **Shortest Path** algorithms (Dijkstra’s) where you repeatedly extract the smallest distance  
- Note: for lots of heap problems of kth, we want to init heap to have k size that fills -inf elements if heap.size < k to avoid later edge cases.

In [ ]:
import heapq

heap = []            # creates an empty heap
heappush(heap, item) # pushes a new item on the heap
item = heappop(heap) # pops the smallest item from the heap
item = heap[0]       # smallest item on the heap without popping it
heapify(x)           # transforms list into a heap, in-place, in linear time
item = heappushpop(heap, item) # pushes a new item and then returns
                               # the smallest item; the heap size is unchanged
item = heapreplace(heap, item) # pops and returns smallest item, and adds
                               # new item; the heap size is unchanged

In [ ]:
class MinHeap:
    """
    A min-heap ensures the smallest element is always at the root (index 0).
    """

    def __init__(self):
        self.heap = []  # The heap is stored in a list.

    # Helper methods to get parent and children indices:
    def parent(self, i):
        return (i - 1) // 2

    def left_child(self, i):
        return 2 * i + 1

    def right_child(self, i):
        return 2 * i + 2

    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    def heapify_up(self, i):
        """
        'heapify_up' moves a newly added element upwards if it's smaller
        than its parent—fixing violation from bottom to top.
        """
        # While we haven't reached the root AND the current item is smaller than its parent:
        while i > 0 and self.heap[i] < self.heap[self.parent(i)]:
            # Swap with its parent to move the smaller item up.
            self.swap(i, self.parent(i))
            # Update 'i' to continue checking higher up the heap.
            i = self.parent(i)

    def heapify_down(self, i):
        """
        'heapify_down' moves an element downwards if it's larger than one
        of its children—fixing violations from top to bottom.
        """
        n = len(self.heap)
        while True:
            smallest = i  # Assume the parent is smallest initially.
            left = self.left_child(i)
            right = self.right_child(i)

            # Check if left child is smaller.
            if left < n and self.heap[left] < self.heap[smallest]:
                smallest = left

            # Check if right child is even smaller.
            if right < n and self.heap[right] < self.heap[smallest]:
                smallest = right

            # If the smallest is not the current node, swap and continue down.
            if smallest != i:
                self.swap(i, smallest)
                i = smallest
            else:
                # If the current node is already smaller than both children, stop.
                break

    def heapify(self, array):
        """
        'heapify' builds a heap out of an entire array at once by
        calling 'heapify_down' on non-leaf nodes, from right to left.
        This is more efficient (O(n)) than inserting items one by one.
        """
        self.heap = array  # Initialize the heap with the given array.

        # Start from the last non-leaf node down to the root.
        # For each, push it down if necessary to satisfy min-heap rules.
        for i in range(len(self.heap) // 2 - 1, -1, -1):
            self.heapify_down(i)

    def push(self, item):
        """
        Insert 'item' into the heap:
          1) Append to the end.
          2) 'heapify_up' to fix any violations from bottom to top.
        """
        self.heap.append(item)
        self.heapify_up(len(self.heap) - 1)

    def pop(self):
        """
        Remove and return the smallest element (root).
          1) Swap root with the last element, pop it.
          2) 'heapify_down' from the root to re-heapify.
        """
        if len(self.heap) == 0:
            return None

        root = self.heap[0]
        self.heap[0] = self.heap[-1]
        self.heap.pop()
        self.heapify_down(0)
        return root


Note: 
- Set of k largest: Use min heap size k of the largest set, heap[0] is the smallest, used as boundary.
- Set of k smallest: k largest but negative val.

### Easy

#### [703. Kth Largest Element in a Stream](https://leetcode.com/problems/kth-largest-element-in-a-stream/)

Approach:
- Use min-heap of size k. 
    -  CREATE K-SIZE HEAP by appending from the largest in nums

nums = [2,3,4,5,6], k =3 -> minheap = [4,5,6], heap[0] = 4

- If (adding val to nums) val > heap[0], minheap.push(val), minheap.pop(). Return heap[0]

In [80]:
import heapq
class KthLargest:
    def __init__(self, k: int, nums: list[int]):
        self.heap = nums
        heapq.heapify(self.heap)
        while len(self.heap) > k:
            heapq.heappop(self.heap)
        
        #add -inf to heap if size heap < k, so that we'll don't have to deal with edge case later in add()
        if len(self.heap) < k: 
            heapq.heappush(self.heap, float('-inf')) #-> any next val can be pushed to heap
            

    def add(self, val: int) -> int:
        #must be larger than the smallest in the upperlarge-heap to join heap
        if val > self.heap[0]:
            heapq.heappop(self.heap)
            heapq.heappush(self.heap, val)
        return self.heap[0]
        
            
kthLargest = KthLargest(3, [4, 5, 8, 2])
print(kthLargest.add(3))
print(kthLargest.add(5))
print(kthLargest.add(10))
print(kthLargest.add(9))
print(kthLargest.add(4))

4
5
5
8
8


#### [1046. Last Stone Weight](https://leetcode.com/problems/last-stone-weight/)

- Heapify -stones for maxheap
- Keep heappop the largest two elements in heap -> compare -> decide to heappush different largest - second

In [ ]:
import heapq
#work on negative sign heap -> change sign all operations
def lastStoneWeight(stones: list[int]) -> int:
    heap = [-x for x in stones]
    
    heapq.heapify(heap)
    
    while len(heap) >1:
        largest = heapq.heappop(heap)
        #largest and second large comparison
        second = heapq.heappop(heap)
        if largest != second:
            heapq.heappush(heap, largest - second)
    return 0 if not heap else -heap[0]
    
stones = [2,7,4,1,8,1]
lastStoneWeight(stones)

1

#### [973. K Closest Points to Origin](https://leetcode.com/problems/k-closest-points-to-origin/)
- Create min heap size k, only contains k largest -distance (smallest real distance)
- Heap form = [(val, [key])]

In [94]:
import heapq
def kClosest( points: list[list[int]], k: int) -> list[list[int]]:
    heap = [[float('-inf'), 0, 0]]*k
    #maxheap to store min distance, root is max of kth min distances
    for p in points:
        dis = p[0]**2 + p[1]**2

        if -dis > heap[0][0]:
            heapq.heappush(heap, [-dis, p[0], p[1]])
            heapq.heappop(heap)
    
    return [[x,y] for [z, x, y] in heap]

points = [[3,3],[5,-1],[-2,4]]
k = 2
kClosest(points, k)

[[-2, 4], [3, 3]]

**Modifed binary search approach** (similar to Quick select).
- **Algorithm**: On each iteration, pick a distance `mid` and split points into two groups: those within `mid` (closer) and those beyond `mid` (farther).  
- **Discard Step**: Depending on the size of the “closer” group relative to \(k\), we discard one group entirely (either the closer group if it’s too big, or the farther group otherwise).  
- **Halving Effect**: Because we throw away about half the points on each partition in the average case, the total work sums up to about \(O(n)\) overall 

In [134]:
#modify binary search
import math
def kClosest(points, k):
    #get the mid threshold from 0 and max(distances)
    #divide points to closer and farther halves, work until len(closer)==k
    
    dist = [point[0]**2+point[1]**2 for point in points ]
    lo, hi = 0, max(dist)
    #remain points to work with, after each iteration of halving distances
    remainPoints = [i for i in range(len(points))]
    totalClose = []
    while True:
        mid = (hi + lo)/2
        close, far = splitPoints(remainPoints, dist, mid)
        
        if len(close) == k:
            totalClose.extend(close)
            return [points[i] for i in totalClose]
        elif len(close) < k:
            #move mid farther
            lo = mid
            k -= len(close) #remain k to be added after count all close points within k
            totalClose.extend(close) # add to total
            remainPoints = far

        else:
            #ignore farther
            hi = mid
            remainPoints = close
            
def splitPoints(remain, distance, mid):
    close, far = [], []
    for idx in remain:
        if distance[idx]> mid:
            far.append(idx)
        else:
            close.append(idx)
    return [close, far]

    
kClosest([[0,1],[1,0]],2)

[[0, 1], [1, 0]]

#### 215. Kth Largest Element in an Array

Heap aproach: Avg + Worst: Nlogn

- Heapify nums -> pop k times

In [ ]:
def findKthLargest(nums: list[int], k: int) -> int:
    nums = [-i for i in nums]
    heapq.heapify(nums)
    for i in range(k-1):
        heapq.heappop(nums)
    return -nums[0]
nums = [3,2,1,5,6,4]
k = 2
findKthLargest(nums, k)

5

Quicksort descending new: most efficient - 
- Random pivot
- Create 3 arrays: left(all nums > pivot), mid(nums==pivot), right(nums< pivot)
- if k <= len(left) => recursively go to left
- if k > len(left)+len(mid) => recursively go to right
- else return mid[0], k is the pivot(s)

In [135]:
import random
def findKthLargest(nums: list[int], k: int) -> int:
    pivot = random.choice(nums)
    left = [num for num in nums if num > pivot]
    right = [num for num in nums if num < pivot]
    mid = [num for num in nums if num == pivot]
    
    if k <= len(left): return findKthLargest(left, k)
    if k > len(left)+len(mid): return findKthLargest(right, k-len(left)-len(mid))
    return mid[0]

Approach: Old QuickSelect Pivot - RTL error
- Avg: O(n), worst O(n^2)
- Quick sort until kth pivot, quicksort is recursive
- Quicksort(l,r) (start with 0, len(nums))
    - Init: **pivotNumber, pivotPointer** = nums[r], l
    - for i in range(l, r):
        - If nums[i] >= nums[pivot], swap(num[i], num[p]); p+=1 
            - i+=1 (**-> p reserve the next spot for the next larger**))
    - swap nums[p], nums[pivot](**After the loop, p always reserve the spot to where pivot should be**)
    

In [137]:
def findKthLargest(nums: list[int], k: int) -> int:
    def quickShort(l,r):
        p, pivot = l, nums[r]
        for i in range(l,r):
            #even don't have to swap for "=" case, still use
            #"=" to p++, reserve spot for next ">=" nums[i]       
            if nums[i] >= pivot:
                nums[i], nums[p] = nums[p], nums[i]
                p+=1
        nums[p], nums[r] = pivot, nums[p]
        #if p < k, go to larger partition: the right
        if p < k: return quickShort(p+1, r)
        if p > k: return quickShort(l, p-1)
        return nums[p]
    return quickShort(0, len(nums)-1)

#### [621. Task Scheduler](https://leetcode.com/problems/task-scheduler/)
Intuition: Most frequent task processed first to minimize idle time ->
1. use **heap** to track most frequent **IN USE**task
2. use **queue** to track FIFO **WAITING TASK**
Think of tasks as their frequents: AAABBCC -> 3,2,2

Approach: Implement maxheap of counts of most frequent letters, use **queue** to FIFO waiting time - Form:[[count1:time1], [count2:time2],....]
- While **heap or q** #A letter on **ALWAYS- EITHER- wait** ||  **heap**
    - Each time used a letter => Heap[0]+=1, add to q, pop heap
    - If q[0].time == currentTime: add back to **IN USE** heap

In [138]:
import heapq
from collections import Counter, deque
def leastInterval(tasks: list[str], n: int) -> int:
    c = Counter(tasks)
    heap = [-x for x in c.values()]
    heapq.heapify(heap)
    q = deque()
    time = 0
    while heap or q: #A letter ALWAYS EITHER on wait or on heap
        time +=1

        #if not heap -> all letters IDLE in q
        #if heap, consume root(most freq letter)
        if heap:
            if heap[0]+1 != 0: #if there's still letters after consuming 1
                q.append([heap[0]+1, time+n]) #add to q end
            heapq.heappop(heap) #Used top letter
        
        #push back to heap the first letter ends to wait in IDLE
        if q and q[0][1] == time:
                heapq.heappush(heap, q.popleft()[0]) #pop IDLE, push count to heap    
    return time
    
tasks = ["B","F","J","J","H","A","D","C","C","D","J","E","B","E","C","H","E","E","G","E","H","I","I","E","H","F","C","G","H","F","E","E","I","D","D","A","E","A","C","E","F","J","E","F","G","J","A","A","A","B","E","J","H","C","A","E","D","A","E","I","H","B","A","C","E","F","J","D","F","I","I","H","J","E","B","H","H","A","J","C","C","D","C","B","C","H","H","I","H","D","C","B","D","C","H","A","F","A","J","D","F","E","H","I","D","A","E","B","H","J","F","D","C","J","J","I","A","I","J","H","A","I","I","G","C","C","H","D","B","B","B","H","I","D","B","C","H","I","I","G","E","D","D","A","E","D","H","C","J","H","C","E","I","F","A","I","E","G","A","E","F","I","B","J","B","J","B","G","A","D","C","B","I","A","C","J","J","J","B","E","E","E","B","E","B","H","B","J","H","F","G","B","B","I","C","D","I","D","D","A","I","D","H","A","J","D","J","D","I","G","F","F","B","G","A","F","I","I","H","C","B","H","H","E","F","A","H","G","F","D","G","F","A","C","F","D","F","C","J","A","D","H","G","D","D","G","C","G","C","I","B","G","E","J","C","G","J","G","E","F","F","I","D","D","E","G","F","A","F","H","D","C","B","E","H","G","H","B","A","D","A","J","A","C","B","G","C","A","J","E","I","G","H","J","A","J","I","D","I","G","B","G","I","A","B","J","B","H","E","G","G","I","G","G","C","J","J","J","H","G","A","G","B","G","I","G","A","D","B","A","E","I","H","E","G","H","J","F","H","I","A","E","A","I","A","D","D","B","D","G","E","C","J","C","C","B","F","F","B","E","A","G","D","I","G","D","J","D","G","I","B","J","J","B","F","D","I","C","I","B","J","B","D","D","A","D","C","G","A","G","C","G","C","E","C","G","A","J","C","E","F","C","C","E","J","D","H","H","F","E","D","G","E","E","I","B","G","A","C","C","E","J","F","H","C","A","G","H","G","A","G","A","D","B","E","F","H","C","C","B","I","F","H","I","C","H","G","G","C","E","A","J","F","F","F","B","D","I","E","I","F","E","F","G","G","A","D","I","A","J","A","H","J","I","H","H","A","B","B","B","I","C","J","C","G","J","G","E","I","H","D","E","H","J","A","G","A","G","A","C","F","C","G","E","G","J","D","H","H","B","J","F","J","J","B","J","D","D","G","B","I","I","H","G","B","I","E","D","H","H","I","B","A","A","A","D","H","J","H","D","J","E","D","G","J","B","B","F","G","J","G","E","G","E","E","F","H","F","J","A","G","I","D","C","H","A","C","F","B","J","E","I","A","G","J","D","I","F","I","J","H","E","D","J","A","E","G","B","B","J","I","J","H","F","D","F","F","A","G","F","F","I","C","H","E","E","G","A","D","F","I","D","D","C","B","G","C","D","G","H","F","D","J","D","B","A","J","J","I","H","F","G","D","J","H","I","H","I","A","A","I","C","B","H","G","I","C","F","E","J","J","F","F","D","F","A","J","H","B","G","A","F","D","G","D","C","F","J","F","G","D","H","J","C","A","E","C","G","J","G","I","C","G","H","G","J","D","D","G","D","F","F","J","B","D","C","E","F","G","D","A","J","H","D","F","C","B","H","C","I","D","C","F","E","C","D","J","D","E","G","C","D","H","J","E","H","I","I","A","C","E","C","I","B","A","B","E","E","H","E","B","H","C","G","B","C","C","D","G","G","A","F","A","B","D","G","F","A","H","G","C","E","D","B","H","D","F","F","G","A","J","H","E","B","C","B","B","B","H","D","F","B","B","C","G","A","C","E","J","H","F","F","D","G","J","D","F","J","G","H","B","D","B","D","D","G","J","H","B","D","F","E","E","G","D","H","B","A","I","E","B","E","B","D","I","C","A","A","E","J","A","B","A","F","C","J","F","F","F","A","I","J","F","H","G","C","F","E","D","C","B","C","G","G","G","A","B","J","J","F","J","G","C","D","B","C","F","H","I","F","D","C","I","J","D","I","G","B","G","I","J","E","H","G","G","J","J","A","E","E","I","G","E","H","F","F","C","A","J","I","I","D","C","E","G","A","A","H","F","A","I","B","H","J","H","B","H","A","A","A","G","I","I","D","F","F","C","H","D","B","J","F","G","E","F","J","A","I","G","J","J","F","F","G","B","B","D","F","E","G","D","D","A","D","D","G","C","C","I","C","H","I","C","E","G","C","E","J","F","G","J","B","I","B","B","C","B","G","A","J","H","C","G","D","E","H","E","H","A","H","I","A","J","C","G","B","G","H","G","H"]
n = 36
leastInterval(tasks, n)

4256

#### [355. Design Twitter](https://leetcode.com/problems/design-twitter/)

Notes:
- getNewsFeed(userId): O(n log k) - Where 'n' is the total number of tweets from the user and their followees, and 'k' is the number of tweets to return (10 in this case). This is due to the heapq.nlargest operation.

Attributes:  
*   **`follower`:** defaultdict(set)  
*   **`tweet`:** Dict map tweets for each user with timestamps, defaultdict(list)
*   **`time`:** Global counter to order tweets.  

Methods: 
*   **`postTweet`:** Adds a tweet with a timestamp.
*   **`getNewsFeed`:**
    *   Gathers tweets from the user and those they follow.
    *   Uses `heapq.nlargest` to get the 10 most recent.
*   **`follow` / `unfollow`:** Updates the follower relationships.

This structure allows for efficient retrieval of recent tweets by combining followed users' tweets and using a heap for the top 10.


In [140]:
from collections import defaultdict
tweets = defaultdict(list)
tweets["a"].extend([[3,2],[1,2],[4,5]])
v=tweets["a"][:]
v.append([99])
print(v, tweets["a"])

[[3, 2], [1, 2], [4, 5], [99]] [[3, 2], [1, 2], [4, 5]]


In [162]:
from collections import defaultdict
import heapq
class Twitter:

    def __init__(self):
        self.tweets = defaultdict(list)
        self.followers = defaultdict(set)
        self.time = 0

    def postTweet(self, userId: int, tweetId: int) -> None:
        self.time+=1
        self.tweets[userId].append((tweetId,self.time)) #userID:[(tweetID, time)]
    def getNewsFeed(self, userId: int) -> list[int]:
        #Get all tweets from followers and user themselves
        validTweets = self.tweets[userId][:] #copy list of tweets to validtweets
        for followeeID in self.followers[userId]:
            validTweets.extend(self.tweets[followeeID])
            
        return [tweetID for tweetID,_ in heapq.nlargest(10, validTweets)]

    def follow(self, followerId: int, followeeId: int) -> None:
        self.followers[followerId].add(followeeId)

    def unfollow(self, followerId: int, followeeId: int) -> None:
        if followeeId in self.followers[followerId]:
            self.followers[followerId].remove(followeeId)

    def do(self, ops, params):
        self.tw = Twitter()
        res = []
        for i in range(len(ops)):
            if ops[i] == "Twitter":
                res.append(None)  # Initialization returns None
            elif ops[i] == "postTweet":
                res.append(self.tw.postTweet(*params[i]))
            elif ops[i] == "getNewsFeed":
                res.append(self.tw.getNewsFeed(*params[i]))
            elif ops[i] == "follow":
                res.append(self.tw.follow(*params[i]))
            elif ops[i] == "unfollow":
                res.append(self.tw.unfollow(*params[i]))
        return res
    def do(self,op, param):
        res = []
        for i in range(len(op)):
            if op[i] == "Twitter":
                continue
            if op[i] == "postTweet":
                res.append(self.postTweet(*param[i]))
            elif op[i] == "getNewsFeed":
                res.append(self.getNewsFeed(*param[i]))
            elif op[i] == "follow":
                res.append(self.follow(*param[i]))
            elif op[i] == "unfollow":
                res.append(self.unfollow(*param[i]))
        return res 

In [163]:
# TEST, put do inside twitter class
           
# op = ["Twitter", "follow","follow","follow","postTweet","postTweet","postTweet","postTweet","postTweet","postTweet","postTweet","postTweet","postTweet","postTweet","postTweet","getNewsFeed"]
# pa = [[],[1,5],[1,6],[1,7],[7,77],[1,505],[1,111],[5,555],[6,66],[1,5],[2,3],[3,101],[4,13],[1,22],[1,11],[1]]

op = ["Twitter","postTweet","getNewsFeed","follow","getNewsFeed","unfollow","getNewsFeed"]
pa = [[],[1,1],[1],[2,1],[2],[2,1],[2]]

t = Twitter()
t.do(op,pa)
#[None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,[222,204,200,201,205,11,333,94,2,101],None,[211,222,233,208,204,202,200,213,201,203],None,[222,204,200,201,205,11,333,94,2,101]]

[None, [1], None, [1], None, []]

#### [Meeting Rooms II](https://leetcode.com/problems/meeting-rooms-ii/editorial/)


In [235]:
#sort start time, then minheap for endtime
def minMeetingRooms( intervals: list[list[int]]) -> int:
        intervals.sort(key= lambda x: x[0]) # Sort meetings by start time.
        rooms = [] # heap contains ending times, imply rooms used

        # For all the remaining meeting rooms
        for start,end in intervals:
            # if earliest ending <= current start, free earliest ending from heap
            if rooms and rooms[0] <= start:
                heapq.heappop(rooms)

            # add new ending time
            heapq.heappush(rooms, end)
        
        #  size of the ending time heap implied minimum rooms required used, 
        #size of the heap (which stores end times) ultimately reflects the maximum number of rooms needed at any point in time
        # Because we don't pop when all meeting finish, only pop when one finish before another start (use the same room)
        #the code says: heap gets increment every time a new room allocated, and stays the same if previous meeting popped. heap reflects all room has been used all the time.
        return len(rooms)
minMeetingRooms([(0,5 ), (0,1), (0,2), (0,3), (5,10)])

MY HEAP,  [1, 5]
MY HEAP,  [1, 5, 2]
MY HEAP,  [1, 3, 2, 5]
MY HEAP,  [2, 3, 5, 10]


4

In [ ]:
#Approach 2: sort start and end time, use 2 pointers start and end
#keep track of roomusedCount, if start < end: new room allocated,else, use old room.
'''
When encounter an ending event => some meeting that started earlier has ended now. we don't care which meeting has ended. we know meeting ended thus making a room available.

'''
def minMeetingRooms(intervals: list[list[int]]) -> int:
        
        # If there are no meetings, we don't need any rooms.
        if not intervals:
            return 0
# [1,6] [2,5] [3,4] [ 2,3] => 1 2 2 3 , end 3 4 5 6 => 
        used_rooms = 0

        startTime = sorted([i[0] for i in intervals])
        endTime = sorted(i[1] for i in intervals)
        
        end_pointer = 0
        start_pointer = 0

        # Until all the meetings have been processed
        while start_pointer < len(intervals):
            # If there is a meeting that has ended by the time the meeting at `start_pointer` starts
            if startTime[start_pointer] >= endTime[end_pointer]:
                # Free up a room and increment the end_pointer.
                used_rooms -= 1
                end_pointer += 1

            # If a room got free, then this used_rooms += 1 wouldn't have any effect. used_rooms would
            # remain the same in that case. If no room was free, then this would increase used_rooms
            used_rooms += 1    
            start_pointer += 1   

        return used_rooms

#### [2402. Meeting Rooms III](https://leetcode.com/problems/meeting-rooms-iii)

In [34]:
from heapq import heappush, heappop, heapify
from typing import List

class Solution:
    def mostBooked(self, n: int, meetings: List[List[int]]) -> int:
        unusedRoom = [i for i in range(n)] #minheap for unusedroom bc we retrieve smallest roomNum
        usedRoom = [] #min heap for usedRoom currently in use, (end_time, room_index) bc retrieve smallest endtime when unusedroom == 0
        meeting_count = [0] * n
        
        # Process meetings in order of their start times
        for start, end in sorted(meetings): #M⋅logM for soting, M⋅logN for using n-sized heap looping M
            
            # Free up all rooms that (meetings that have ended by "start")
            while usedRoom and usedRoom[0][0] <= start:
                _, room = heappop(usedRoom)  
                heappush(unusedRoom, room)  
            
            # Assign a room to the current meeting
            if unusedRoom:
                # If there are available rooms, take the one with the smallest index
                room = heappop(unusedRoom)
                heappush(usedRoom, [end, room])  # Schedule the meeting in this room
            else:
                # get minEndtime from usedRooms for min delay.
                endTime, room = heappop(usedRoom)
                heappush(usedRoom, [end + (endTime - start), room])  
            
            # Increment the meeting count for the assigned room
            meeting_count[room] += 1
        
        # Return the index of the smallest room with the maximum number of meetings held
        return meeting_count.index(max(meeting_count))
#time/: O(M⋅logM+M⋅logN), space O(N+sort)

#### [1229. Meeting Scheduler](https://leetcode.com/problems/meeting-scheduler/)

In [ ]:
# using sorting
def minAvailableDuration(slots1: List[List[int]], slots2: List[List[int]], duration: int) -> List[int]:
    slots1.sort()
    slots2.sort()
    m, n = len(slots1), len(slots2)
    i = j = 0
    while i < m and j < n:
        intersectStart = max(slots1[i][0], slots2[j][0])
        intersectEnd = min(slots1[i][1], slots2[j][1])
        if intersectEnd - intersectStart >= duration:
            return [intersectStart, intersectStart + duration]
        if slots1[i][1] < slots2[j][1]:
            i += 1
        else:
            j += 1
    return []

In [ ]:
'''
Initialize a heap timeslots and push time slots that last longer than duration into it.
Iterate until there's only one time slot remaining in timeslots:
Pop the first time slot [start1, end1] from timeslots.
Retrieve the next time slot [start2, end2], which is the current top element in timeslots.
If we find end1 >= start2 + duration, because start1 <= start2, the common slot is longer than duration and we can return it.
If we don't find the common slot that is longer than duration, return an empty array.
'''
class Solution:
    def minAvailableDuration(self, slots1: List[List[int]], slots2: List[List[int]], duration: int) -> List[int]:
        # build up a heap containing time slots last longer than duration
        timeslots = list(filter(lambda x: x[1] - x[0] >= duration, slots1 + slots2))
        heapq.heapify(timeslots)

        while len(timeslots) > 1:
            start1, end1 = heapq.heappop(timeslots)
            start2, end2 = timeslots[0]
            if end1 >= start2 + duration:
                return [start2, start2 + duration]
        return []
"""
Time : O((M+N)log(M+N)), when M is length of slots1 and N is length of slots2.
There are two parts to be analyzed: 1) building up the heap; 2) the iteration when we keep popping elements from the heap.
Space : O(M+N)"""

In [ ]:
#1999. Smallest Greater Multiple Made of Two Digits
from collections import deque
class Solution:
    def findInteger(self, k: int, digit1: int, digit2: int) -> int:
        q = deque()
        d1 = min(digit1,digit2)
        d2 = max(digit1,digit2)
        if d1!= 0:
            q.append(d1)
        if d2!= 0:
            q.append(d2)
        maxInt = 2**31-1
        while(q):
            nextInteger = q.popleft()
            if nextInteger > maxInt: return -1
            elif nextInteger > k and nextInteger % k == 0: return nextInteger

            q.append(nextInteger*10+d1)
            q.append(nextInteger*10+d2)
        return -1

        

## Graphs

### Init Utilities

In [124]:
from collections import deque
class Node:
    def __init__(self, val = 0, neighbors = None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []
    def __repr__(self) -> str:
        s = "{ "+str(self.val)+" } -> ["
        for i in self.neighbors:
            s += str(i.val) + ", "
        s+= " ]"
        return s
def toAdjList(adjList: list) -> list:
    nodes = {}
    
    for i, neighbors in enumerate(adjList, start=1):
        nodes[i] = Node(i)

    for i, neighbors in enumerate(adjList, start=1):
        nodes[i].neighbors = [nodes[neighbor] for neighbor in neighbors]

    return nodes

            

#### [200. Number of Islands](https://leetcode.com/problems/number-of-islands/)

In [123]:
from collections import deque
class Solution:
    def numIslands(self, grid: list[list[str]]) -> int:
        if not grid:
            return 0
        
        visit = set()
        out = 0
        
        def bfs(r,c):
            q = deque()
            direction = [(0,1),(0,-1), (1,0), (-1,0)]
            q.append((r,c))
            while q:
                
                r,c = q.popleft() #Process a cell
                #Add cell's children to
                for dr, dc in direction:
                    row, col = r + dr, c + dc
                    if(row in range(len(grid))and 
                    col in range(len(grid[0]))and
                    (row, col) not in visit and 
                    grid[row][col] == "1"):
                        visit.add((row,col))
                        q.append((row, col))
            
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if (i,j) not in visit and grid[i][j] == "1":
                    bfs(i,j)
                    out +=1
        return out
                
grid = [
  ["1","1","0","0","0"],
  ["1","1","0","0","0"],
  ["0","0","1","0","0"],
  ["0","0","0","1","1"]]
print(Solution.numIslands(Solution,grid))

3


In [ ]:
#DFS REC
class Solution:
    def numIslands(self, grid: list[list[str]]) -> int:
        if not grid:
            return 0
        
        visit = set()
        out = 0
        
        def dfs(r,c):
            
            if ((r,c) in visit or 
            r not in range(len(grid)) or
            c not in range(len(grid[0])) or 
            grid[r][c] != "1"):
                return 
                        
            visit.add((r,c))
            dfs(r+1, c)
            dfs(r, c+1) 
            dfs(r-1, c)
            dfs(r, c-1)
        
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if (i,j) not in visit and grid[i][j] == "1":
                    dfs(i,j)
                    out +=1
        return out
                
grid = [
  ["1","1","0","0","0"],
  ["1","1","0","0","0"],
  ["0","0","1","0","0"],
  ["0","0","0","1","1"]]
print(Solution.numIslands(Solution,grid))

3


#### [133. Clone Graph](https://leetcode.com/problems/clone-graph/)

- A dictionary nodes = {oldNodes:newNode}
- DFS recursive():
    - nodes[oldNodes] = dfs(oldNodes)
    - Base: Return nodes[oldNodes] if oldNodes in nodes?
        - Return when see a node already created

In [129]:
from typing import Optional
class Solution:
    def cloneGraph(node: Optional['Node']) -> Optional['Node']:
        cloneNodes = {}
        def dfs(oldNode):
            if oldNode in cloneNodes: #if this oldNode already cloned
                return cloneNodes[oldNode]
            
            cloneNodes[oldNode] = Node(val=oldNode.val)
            
            for i in oldNode.neighbors:
                cloneNodes[oldNode].neighbors.append(dfs(i))
                
            return cloneNodes[oldNode]
            
        return dfs(node)

adjList = toAdjList([[2,4],[1,3],[2,4],[1,3]])
print(adjList)
Solution.cloneGraph(adjList[1])


{1: { 1 } -> [2, 4,  ], 2: { 2 } -> [1, 3,  ], 3: { 3 } -> [2, 4,  ], 4: { 4 } -> [1, 3,  ]}


{ 1 } -> [2, 4,  ]

In [134]:
#BFS
from typing import Optional
class Solution1:
    def cloneGraph(node: Optional['Node']) -> Optional['Node']:
        if not node: 
            return None
        q = deque([node])
        visit = set()
        cloneNodes = {}

        while q:
            oldNode = q.popleft()
            if oldNode not in cloneNodes:
                cloneNodes[oldNode] = Node(val = oldNode.val)

            for oldNeibor in oldNode.neighbors:
                if oldNeibor not in cloneNodes:
                    cloneNodes[oldNeibor] = Node(val = oldNeibor.val)
                cloneNodes[oldNode].neighbors.append(cloneNodes[oldNeibor])
                if oldNeibor not in visit:
                    visit.add(oldNeibor)
                    q.append(oldNeibor)
            
        return cloneNodes[node]
adjList = toAdjList([[2,4],[1,3],[2,4],[1,3]])
print(adjList)
Solution1.cloneGraph(adjList[1])

{1: { 1 } -> [2, 4,  ], 2: { 2 } -> [1, 3,  ], 3: { 3 } -> [2, 4,  ], 4: { 4 } -> [1, 3,  ]}


{ 1 } -> [2, 4, 2, 4,  ]

#### [695. Max Area of Island](https://leetcode.com/problems/max-area-of-island/description/)

In [ ]:
from collections import deque
class Solution:
    def maxAreaOfIsland(self, grid: list[list[int]]) -> int:
        def bfs(r,c):
            count = 0
            q = deque()
            q.append((r,c))
            while q:
                r,c = q.popleft() #turn q.popleft() -> q.pop(): DFS interative
                dir = [(1,0),(0,1),(-1,0),(0,-1)]
                for dr, dc in dir:
                    if (r+dr not in range(len(grid)) 
                        or c+dc not in range(len(grid[0]))
                        or (r+dr,c+dc) in visit
                        or grid[r+dr][c+dc] != 1):
                            continue

                    count+=1
                    visit.add((r+dr,c+dc))
                    q.append((r+dr,c+dc))
                
            return count
        visit,mx = set(), 0
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == 1 and (i,j) not in visit:
                    mx = max(mx, bfs(i,j))
        return mx
grid = [[0,0,1,0,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,0,0,1,1,1,0,0,0],[0,1,1,0,1,0,0,0,0,0,0,0,0],[0,1,0,0,1,1,0,0,1,0,1,0,0],[0,1,0,0,1,1,0,0,1,1,1,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1,1,1,0,0,0],[0,0,0,0,0,0,0,1,1,0,0,0,0]]
Solution.maxAreaOfIsland(Solution, grid)

6

In [ ]:
#DFS RECURSIVE
from collections import deque
class Solution:
    def maxAreaOfIsland(self, grid: list[list[int]]) -> int:
        def dfs(r,c):
            if (r not in range(len(grid)) 
                or c not in range(len(grid[0]))
                or (r,c) in visit
                or grid[r][c] != 1):
                    return 0
            visit.add((r,c))
            return 1 + dfs(r+1, c) + dfs(r, c+1) + dfs(r-1, c) + dfs(r, c-1)
            
        visit,mx = set(), 0
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == 1 and (i,j) not in visit:
                    mx = max(mx, dfs(i,j))
        return mx
grid = [[0,0,1,0,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,0,0,1,1,1,0,0,0],[0,1,1,0,1,0,0,0,0,0,0,0,0],[0,1,0,0,1,1,0,0,1,0,1,0,0],[0,1,0,0,1,1,0,0,1,1,1,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1,1,1,0,0,0],[0,0,0,0,0,0,0,1,1,0,0,0,0]]
Solution.maxAreaOfIsland(Solution, grid)

6

#### [417. Pacific Atlantic Water Flow](https://leetcode.com/problems/pacific-atlantic-water-flow/)
**Optimal Approach**: O(m.n)
From borders, DFS inward to see what cells go to each ocean. For passing in DFS in the first loop:
- From P up to down, P left to right
- From A down to up, A right left

Then go 4 directions from each cell.

<img src="assets/417. Pacific Atlantic Water Flow.png" height=200></img>

For each cell, if cell is both in A,P, add to ouput

**OG Approach**: O((m.n)^2)
- DFS 4 directions from each cell

In [ ]:
#Improvised than Neetcode: starts DFS 1 step inward from borders cuz borders -> auto PA/AT
#Note: Have to add border cell before DFS in the loop in range(CO)/range(RO)
class Solution:
    def pacificAtlantic(self, heights: list[list[int]]) -> list[list[int]]:
        al,pa,out = set(), set(),[]
        RO, CO = len(heights), len(heights[0])
        
        def dfs(r,c, visit ,prevh):

            if (r not in range(RO) or 
                c not in range(CO) or
                (r,c) in visit or
                heights[r][c] < prevh):
                return
            
            visit.add((r,c))
            dfs(r+1, c, visit, heights[r][c])
            dfs(r, c+1, visit, heights[r][c])
            dfs(r-1, c, visit, heights[r][c])
            dfs(r, c-1, visit, heights[r][c])
            return
        
        #All cells could go to Pacific/Atlantic will be added to pa, al sets
        for i in range(CO):
            #Add border to visit + pass border as prevheight (<=)
            pa.add((0,i))
            dfs(1, i, pa , heights[0][i])
            al.add((RO-1,i))
            dfs(RO-2, i, al , heights[RO-1][i])
        for i in range(RO):
            pa.add((i,0))
            dfs(i, 1, pa ,heights[i][0])
            al.add((i, CO-1))
            dfs(i, CO-2, al , heights[i][CO-1])
            
        # print(pa,'\n',al)
        for i in range(RO):
            for j in range(CO):
                if (i,j) in al and (i,j) in pa:
                    out.append([i,j])
        return out    
heights = [[1,2,2,3,5],[3,2,3,4,4],[2,4,5,3,1],[6,7,1,4,5],[5,1,1,2,4]]
Solution.pacificAtlantic(Solution, heights)


[[0, 4], [1, 3], [1, 4], [2, 2], [3, 0], [3, 1], [4, 0]]

In [ ]:
#Original Neetcode (starts dfs from border)
class Solution:
    def pacificAtlantic(self, heights: list[list[int]]) -> list[list[int]]:
        al,pa,out = set(), set(),[]
        RO, CO = len(heights), len(heights[0])
        
        def dfs(r,c, visit ,prevh):

            if (r not in range(RO) or 
                c not in range(CO) or
                (r,c) in visit or
                heights[r][c] < prevh):
                return
            
            visit.add((r,c))
            dfs(r+1, c, visit, heights[r][c])
            dfs(r, c+1, visit, heights[r][c])
            dfs(r-1, c, visit, heights[r][c])
            dfs(r, c-1, visit, heights[r][c])
            return
        
        #All cells could go to Pacific/Atlantic will be added to pa, al sets
        for i in range(CO):
            #pass 0 so any border cells work (<=)
            dfs(0, i, pa , 0)
            dfs(RO-1, i, al , 0)
        for i in range(RO):
            dfs(i, 0, pa ,0)
            dfs(i, CO-1, al , 0)
            
        # print(pa,'\n',al)
        for i in range(RO):
            for j in range(CO):

                if (i,j) in al and (i,j) in pa:
                    out.append([i,j])
        return out    
heights = [[1,2,2,3,5],[3,2,3,4,4],[2,4,5,3,1],[6,7,1,4,5],[5,1,1,2,4]]
Solution.pacificAtlantic(Solution, heights)


[[0, 4], [1, 3], [1, 4], [2, 2], [3, 0], [3, 1], [4, 0]]

#### [130. Surrounded Regions](https://leetcode.com/problems/surrounded-regions/description/)

Marked all the Os adjacent to Os in border.

In [ ]:
%%time
class Solution:
    def solve(self, board: list[list[str]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        
        CO, RO = len(board[0]), len(board)        
        def dfs(r,c):
            # print(r,c)
            if (r not in range(RO) or
                c not in range(CO) or 
                board[r][c] != "O"):
                return
        
            board[r][c] = "V"
            
            dfs(r+1, c)
            dfs(r, c+1)
            dfs(r-1, c)
            dfs(r, c-1)
       
        for i in range(RO):
            for j in range(CO):
                if board[i][j] == "O" and i in [0,RO-1] or j in [0, CO-1]:
                    dfs(i,j)

        for i in range(RO):
            for j in range(CO):
                if board[i][j] == "O":
                    board[i][j] = "X" 
        
        for i in range(RO):
            for j in range(CO):
                if board[i][j] == "V":
                    board[i][j] = "O" 
        
        return board
board = [["X","O","X","O","X","O"],["O","X","O","X","O","X"],["X","O","X","O","X","O"],["O","X","O","X","O","X"]]
Solution.solve(Solution,board)

CPU times: user 87 µs, sys: 0 ns, total: 87 µs
Wall time: 88.9 µs


[['X', 'O', 'X', 'O', 'X', 'O'],
 ['O', 'X', 'X', 'X', 'X', 'X'],
 ['X', 'X', 'X', 'X', 'X', 'O'],
 ['O', 'X', 'O', 'X', 'O', 'X']]

In [ ]:
%%time
class Solution:
    def solve(self, board: list[list[str]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        
        CO, RO = len(board[0]), len(board)
        visit = set()
        
        def dfs(r,c, start):
            # print(r,c)
            if (r not in range(RO) or
                c not in range(CO) or 
                (r,c) in visit and start == False or 
                board[r][c] != "O"):
                return
            # if start==True:
                # print("First add", r,c)
            visit.add((r,c))
            
            dfs(r+1, c, False)
            dfs(r, c+1, False)
            dfs(r-1, c, False)
            dfs(r, c-1, False)
       
        for i in range(CO):
            if board[0][i] == "O":
                dfs(0, i, True)
            if board[RO-1][i] == "O":
                dfs(RO-1, i, True)

        for i in range(RO):
            if board[i][0] == "O":
                dfs(i, 0, True)
            if board[i][CO-1] == "O":
                dfs(i, CO-1, True)

        for i in range(RO):
            for j in range(CO):
                if board[i][j] == "O" and (i,j) not in visit:
                    board[i][j] = "X" 
        
        return board
board = [["X","O","X","O","X","O"],["O","X","O","X","O","X"],["X","O","X","O","X","O"],["O","X","O","X","O","X"]]
Solution.solve(Solution,board)

CPU times: user 52 µs, sys: 1 µs, total: 53 µs
Wall time: 54.1 µs


[['X', 'O', 'X', 'O', 'X', 'O'],
 ['O', 'X', 'X', 'X', 'X', 'X'],
 ['X', 'X', 'X', 'X', 'X', 'O'],
 ['O', 'X', 'O', 'X', 'O', 'X']]

#### [994. Rotting Oranges](https://leetcode.com/problems/rotting-oranges/description/)
**Approach**: Traverse all neighbors of certain nodes at a time -> BFS

- Go through board to init the BFS queue with all rottens + count freshes
- Start BFS to count minutes and decre freshes.

In [ ]:
from collections import deque
class Solution:
    def orangesRotting(self, grid: list[list[int]]) -> int:
        minTime = 0
        q = deque()
        fresh = 0 # need to know #fresh to stop early
        dir = [(0,1),(1,0), (-1,0), (0,-1)]
        #Count fresh and init rotten in q
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == 2:
                    q.append((i,j))
                elif grid[i][j] == 1:
                    fresh += 1
        #question: is visited == turning val 1->2 
        #BFS
        #while q: there's still unprocessed rottens;
        #-> no unprocessed rottens left OR no fresh left(mission done) ->terminate loop
        while q and fresh > 0:
            l = len(q)
            for _ in range(l):
                r,c = q.popleft()
                
                for dr,dc in dir:
                    if (r+dr in range(len(grid)) 
                        and c+dc in range(len(grid[0])) 
                        and grid[r+dr][c+dc] == 1) :
                        
                        grid[r+dr][c+dc] = 2 #same as add to visit
                        q.append((r+dr,c+dc))
                        fresh-=1
            minTime+=1
        return minTime if fresh <=0 else -1
grid = [[2,1,1],[1,1,0],[0,1,1]]           
Solution.orangesRotting(Solution,grid)


4

#### [207. Course Schedule](https://leetcode.com/problems/course-schedule/description/)

Thought process to detect cycle using DFS:
- **Traverse EACH DFS path independently to see anypath is cycle, recorded nodes in the path by visitSet, return F if a node in visitSet**
    - Note that, at one time, the visitSet contains all nodes in ONE DFS path, not the whole graph.
        - Therefore, if we see **one node on a DFS path twice**, cycle happened
    - To obtain visitSet containing only ONE DFS PATH at a time *(instead of DFS WHOLE GRAPH like previous simple graph traversal problems)*,  add node to visitSet, then DFS all its child nodes, THEN SET its CHILD SET to EMPTY, THEN immediately REMOVE NODE. 
    - REMOVE NODE helps visit only records one path at a time, in case visit see the NODE traverse other path but it's not a a cycle

Approach: prerequisites = lists of adjacent nodes
- **HashMap (dict) to store adjacent of each node.**
- Cycle detected -> Infinite loop to finish courses.
    - Regular DFS with one note: After add node to visit, remove it after the path to its leaf.  

Init Graph:
- numCourses = #vertices. Vertices values: 0 -> #verticies
- prerequisites = edges

<img src="assets/207. Course Schedule.png" width=400></img>

In [9]:
from collections import defaultdict
class Solution:
    def canFinish(self, numCourses: int, prerequisites: list[list[int]]) -> bool:
        graph = defaultdict(list)
        visit = set()
        #Init graph
        for des, src in prerequisites:
            # if src not in graph:
            #     graph[src] = set()
            graph[src].append(des)
        
        def dfs(node):
            if node in visit:
                return False
            if [] == graph[node]:
                return True
            
            visit.add(node)
            for child in graph[node]:
                if not dfs(child):
                    return False
            graph[node] = []
            visit.remove(node)
            
            return True
        
        for src in list(graph):
            if not dfs(src):
                return False
        return True
    
    
numCourses = 2
prerequisites = [[1,0]]
# prerequisites = [[1,0],[0,1]]
Solution.canFinish(Solution, numCourses, prerequisites)

True

Alternative Approach: Topological Sort 
Read more at [Topo Sort](#topological-sort-for-directed-graph)

In [10]:
from collections import defaultdict, deque

def canFinish(numCourses, prerequisites):

    graph = defaultdict(list)
    in_degree = [0] * numCourses
    for dest, src in prerequisites:
        graph[src].append(dest)
        in_degree[dest] += 1

    queue = deque([i for i in range(numCourses) if in_degree[i] == 0])
    count = 0

    while queue:
        popnode = queue.popleft()
        count += 1 #Finishing processing popnode, add to final topo count
        for neighbor in graph[popnode]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)

    return count == numCourses  # If all courses can be taken, count will equal numCourses

#### [Islands and Treasure](https://neetcode.io/problems/islands-and-treasure)

Find min distance from each land node to treasure node. => Use BFS for min distance on a whole because BFS exlore all neighbor nodes per level from treasure node.  
<img src="assets/islandtreas.jpg" width=200/> 

In [48]:
class Solution:
    def islandsAndTreasure(self, grid: list[list[int]]) -> None:
        m,n = len(grid), len(grid[0])
        #Determine what this q contains: 0 and mindist
        q = deque()
        #init q with treasures 0 
        for i in range(m):
            for j in range(n):
                if grid[i][j] == 0:
                    q.append((i,j))
        
        dir = [(0,1), (1,0), (-1,0),(0,-1)]
        while q:
            r,c = q.popleft()
            for dr, dc in dir:
                if ((r+dr) in range(m) and (c+dc) in range(n) 
                    and grid[r+dr][c+dc] == 2147483647): #cell is neither filled/-1
                    q.append((r+dr, c+dc))
                    grid[r+dr][c+dc] = grid[r][c]+1 #minDist = parent+1
grid= [
  [2147483647,-1,0,2147483647],
  [2147483647,2147483647,2147483647,-1],
  [2147483647,-1,2147483647,-1],
  [0,-1,2147483647,2147483647]
]
Solution.islandsAndTreasure(Solution, grid)
grid
            

[[3, -1, 0, 1], [2, 2, 1, -1], [1, -1, 2, -1], [0, -1, 3, 4]]

#### [210. Course Schedule II](https://leetcode.com/problems/course-schedule-ii/description/)

In [57]:
class Solution:
    def findOrder(self, numCourses: int, prerequisites: list[list[int]]) -> list[int]:
        topo = []
        degree = [0]*numCourses
        graph = defaultdict(list)
        q = deque()
        #1. Get indegrees and init graph
        for des, src in prerequisites:
            degree[des] +=1
            graph[src].append(des)
            
        #2. Init Q with 0-degree, Q only has 0 degree nodes
        # (node's parent processed in topo)
        for node in range(numCourses):
            if degree[node] == 0:
                q.append(node)
        #3. Go through Q, -1 each visit of non-0 degree child, add to res and Q if child.degree = 0
        while q:
            popNode = q.popleft()
            topo.append(popNode) #Add TOPO, degree always == 0
            for child in graph[popNode]:
                if degree[child] > 0: degree[child] -=1
                if degree[child] == 0: q.append(child)
        if len(topo) == numCourses: return topo
        return []

#### [Valid Tree](https://neetcode.io/problems/valid-tree)
For a undirected TREE cycle detection: 
- All nodes connected + no loop.
- We can traverse a whole undirected tree at ANY NODE, and we CAN visit each node ONLY ONCE (by keeping a prev variable prevent one node to go back)
- Add all nodes to visitSet throughout tree
    - DONT REMOVE visit node after process children like [DIRECTED GRAPH cycle detect](#207-course-schedule)
    - in directed graph, there's case where 2 nodes point to 1 nodeX create 2 dfs path, nodeX must be removed to be revisted in another DFS path
    - in undirected, 3 nodes point to each other, don't have to revisit any node.


In [63]:
class Solution:
    # Use visitSet to detect revisited node (loop)
    # Memorize prevNode when DFS each Node to prevent Node go back, only go forward
    def validTree(self, n: int, edges: list[list[int]]) -> bool:
        graph = defaultdict(list)
        visit = set()
        for n1, n2 in edges:
            graph[n1].append(n2)
            graph[n2].append(n1)
        
        def dfs(node, prev):
            if node in visit:
                return False
            visit.add(node)
            for child in graph[node]:
                if child != prev: #dfs all children except prev (parent)
                    if not dfs(child, node):
                        return False
            return True
        return dfs(0, -1) and len(visit) == n
edges = [[0,1],[0,2],[0,3],[1,4]]
n =5
Solution.validTree(Solution, n, edges)

True

#### [323. Number of Connected Components in an Undirected Graph](https://leetcode.com/problems/number-of-connected-components-in-an-undirected-graph/description/)

##### DFS approach/BFS
- Only use visit to track all nodes, no need prev because prev already in visit which we won't visit again 
    - and Don't care about if cycle exist, so don't separate about prev and other VISITED CHILD like [Valid tree](#valid-tree)
- Overall count++ whenever dfs node not in visit  
    DFS/BFS will traverse to node's all connected children/grandchildren

In [70]:
class Solution:
    def countComponents(self, n: int, edges: list[list[int]]) -> int: 
        graph= defaultdict(list)
        for n1, n2 in edges:
            graph[n1].append(n2)
            graph[n2].append(n1)
        count = 0
        visit = set()

        def dfs(node):
            visit.add(node)
            for child in graph[node]:
                if child not in visit:
                    dfs(child)          
            
        for i in range(n):
            if i not in visit:
                dfs(i) #pass dfs of a new component
                count+=1
        return count
        
n=6
edges=[[0,1], [1,2], [2,3], [4,5]]

Solution.countComponents(Solution, n, edges)

2

In [71]:
class Solution:
    def countComponents(self, n: int, edges: list[list[int]]) -> int: 
        graph= defaultdict(list)
        for n1, n2 in edges:
            graph[n1].append(n2)
            graph[n2].append(n1)
        count = 0
        visit = set()

        def bfs(node):
            q = deque()
            q.append(node)
            while q:
                popNode = q.popleft()
                for child in graph[popNode]:
                    if child not in visit:
                        visit.add(child)
                        q.append(child)
        for i in range(n):
            if i not in visit:
                bfs(i) #pass dfs of a new component
                count+=1
        return count
        
n=6
edges=[[0,1], [1,2], [2,3], [4,5]]

Solution.countComponents(Solution, n, edges)

2

##### Union Find (Disjoint set)

**I. General Idea**  
For each edge [A-B], we connect A to B's parent (of a system of connected nodes)  
**#connected_nodes = #independent_nodes - times of unioning node**

(*Before unioning nodes, connected_nodes = #independent_nodes.  
Each time Union, one node from #nodes connected to another node, #independent_nodes -= 1*)

**II. Data structure used:**   
self.parent: A list par stores the parent of each node. par[i] is par of i

self.rank: A list that stores rank of each node, used to determine which will be parent (higher rank node) of a system when union 2 systems

**III. Path compresion**  
**Best compresion:**  def findRoot(node): rankPar[node] = findRoot(rankPar[node])

OG: In findRoot func, while current node is not parent, set par[cur] = grandpar[cur] (node point further to root)  
Pseudo: While cur not root: node.next= node.next.next; node = node.next

E.g
Path before: 5-4-3-2-1 path compress:
cur = 5, par[5] -> 3 =Nextloop=> cur = 3, path[3] -> 1 =Nextloop=> cur = 1. done
Path after: 5->3, 3->1 (check)


In [2]:
class Solution:
    def countComponents(self, n: int, edges: list[list[int]]) -> int:
        rankPar = [-1] * n #Store negative ranks for roots, Positive parent idx for children.
        
        # While current is not root, set par[cur] = grandpar[cur] (point further to root)
        def findRoot(node: int) -> int: 
            if rankPar[node] < 0:
                return node
            rankPar[node] = findRoot(rankPar[node]) #All nodes point to 1 root
            return rankPar[node] #Node's root
        
    #    def findRoot(node: int) -> int: # Stack instead of callbacks
    #         stack = []
    #         while rankPar[node] >= 0:  # node not root
    #             stack.append(node)
    #             node = rankPar[node]
    #         #After finish find loop above (reach base), node is root
    #         while stack: #Pop stack after reaching base
    #             rankPar[stack.pop()] = node #All nodes point to 1 root
    #         return node
 
        def union(u, v) -> bool:
            pu, pv = findRoot(u), findRoot(v)
            if pu == pv: #Same parents
                return False
            #ADD CHILD RANK FIRST, ASSIGN PARENTS LATER
            if rankPar[pu] < rankPar[pv]: # "< negative" == "> positive"
                rankPar[pu] += rankPar[pv]
                rankPar[pv] = pu
            else:
                rankPar[pv]+= rankPar[pu]
                rankPar[pu] = pv
            return True
        
        count = n # Initially, each node is a separate component
        
        for u, v in edges:
            if union(u,v): 
                count -=1 
        return count
    
    def countComponentsOG(self, n: int, edges: list[list[int]]) -> int:
        rank = [1] * n # all nodes init with rank 1
        par = [i for i in range(n)] # Each node is init to be parent of itself
 
        # While current is not root, set par[cur] = grandpar[cur] (point further to root)
        def findRoot(node: int) -> int:
            cur = node
            while cur != par[cur]: 
                par[cur] = par[par[cur]]
                cur = par[cur]
            return cur
        
        def union(u, v) -> bool:
            pu, pv = findRoot(u), findRoot(v)
            if pu == pv: #Same parents
                return False
            if rank[pu] > rank[pv]:
                par[pv] = pu
                rank[pu] += rank[pv]
            else:
                par[pu] = pv
                rank[pv] += rank[pu]
            return True
        
        count = n # Initially, each node is a separate component
        
        for u, v in edges:
            if union(u,v): 
                count -=1 
        return count
edges = [[0,1],[0,2]]
n = 3
Solution.countComponents(Solution, n, edges)

1

#### [851. Loud and Rich](https://leetcode.com/problems/loud-and-rich/description/)
Graph:
Richer: directed edges with no cycle (richer topologically)  
Quiet: Value of vertices (different from index of vertices)  
<img src="assets/851. Loud and Rich.jpeg" width=400></img>  
dfs(nodeX) return LeastQNode at nodeX  

        LeastQNode at dfs(nodeX): init as nodeX  
        for dfs each child: get childLeastQNode, compare to LeastQNode
        ans[node] = LeastQNode found, return LeastQNode

Lesson learn: find a temporary min value of each node to their children for every node

In [42]:
class Solution:
    def loudAndRich(self, richer: list[list[int]], quiet: list[int]) -> list[int]:
        graph = defaultdict(list)
        ans = [-1]*len(quiet)
        for des, src in richer:
            graph[src].append(des)
        def dfs(node):
            # if node's child list empty or ans[node] filled
            if graph[node] == [] or ans[node] != -1:
                if ans[node] == -1:
                    ans[node] = node 
                return ans[node]
            leastQNode = node
            #find leastQNode for node among children
            for child in graph[node]:
                childLeastQNode = dfs(child)
                if quiet[childLeastQNode] < quiet[leastQNode]:
                    leastQNode = childLeastQNode
            #after dfs all node's childs
            ans[node] = leastQNode
            return leastQNode
        for node in range(len(quiet)):
            if ans[node] == -1:
                dfs(node)
        return ans
#time + space: O(n^2), n is #pple , graph has n^2 edges  
richer = [[1,0],[2,1],[3,1],[3,7],[4,3],[5,3],[6,3]]
quiet = [3,2,5,4,6,1,7,0]
Solution.loudAndRich(Solution, richer , quiet)

[5, 5, 2, 5, 4, 5, 6, 7]

### Graph Theory:
#### Topological sort for Directed Graph
- Topological ordering: When orders of nodes matters/got directed (schedule, prerequisites, dependency....). One graph may have many orders.
- A Directed Acyclic Graph (DAG) has a cycle + have NO VALID topo order.
- Uses:
    - Find topological ordering in O(V+E).

<font color='yellow'>DFS</font> 
- For each node in the graph:
    - DFS node that not in visit:
    - After DFS to the end, write on topo order and return back
- Note: Topo order is from **start node-> directed end**, but dfs add **directed end-> start node**:
    - Ways to get right order:
    1. Return Reverse topo: topo[::-1]
    2. In DFS: Use param i where i from n-1 -> 0. Assign order[i]=visiting each dfs; i-=1

In [4]:
#Util funcion turns adj to graph
from collections import defaultdict
def toGraph(n:int, adj:list, start:int)->dict:
    graph = defaultdict(list)
    #Create vertices
    for i in range(start,n+start):
        graph[i] = []
    #Adj
    for u,v in adj:
        graph[u].append(v)
      
    return graph

In [5]:
def topsort(graph: dict) -> list:
    n = len(graph)
    visit = set()
    order = [-1]*n
    index = n-1
    
    #In each dfs, go to neighbors -> assign to ordering
    def dfs(index, node):
        nonlocal graph, visit, order
        #Mark as visit
        visit.add(node)
        #Go to neighbors
        for adj in graph[node]:
            if adj not in visit:
                #Index is decre 
                index = dfs(index, adj) 
        order[index] = node 
        return index - 1
    
    for node in graph:
        if node not in visit:
            #DFS -> assign to ordering -> decre index
            index = dfs(index, node) 
    return order

adj = toGraph(5,[[1, 2],[1, 3],[2, 3],[2, 4],[3, 4],[3,5]],start=1)
topsort(adj)
graph = toGraph(6, [[1,0],[4,1],[4,5],[4,2],[0,2],[0,3],[2,3],[3,5],[5,2]], start=0)
topsort(graph)


[4, 1, 0, 2, 3, 5]

<font color='yellow'>Kahn's: BFS - in degree - Detect cycle</font> 

<img src="assets/kahn topo.png" width=300></img>
1. Init the queue with all nodes with degree = 0
2. BFS via the queue (which only contains 0-degree nodes)
- While q:
    - Decre indegree of all popnode.neighbors
    - If a neighbor's indegree == 0 => add to q
- If len(topo) != len(graph):
    - print("Detect cycle")

In [8]:
from collections import deque
def topsortKahn(graph: dict) -> list:
    q = deque()
    topo = []
    indegree = [0]*(len(graph))
    
    #Get indegree of each vertex:
    for node in graph:
        for v in graph[node]:
            indegree[v] += 1
    
    #Add 0-indegree vertices to q:
    for i in range(0, len(indegree)):
        if indegree[i] == 0:
            q.append(i)
            
    #BFS
    while q:
        popn = q.popleft()
        topo.append(popn) #To-be-finish processing popNode, add to topo sort
        #Visit adj
        for adj in graph[popn]:
            indegree[adj] -= 1
            if indegree[adj] == 0:
                q.append(adj)
    if len(topo) != len(graph):
        print("Detect cycle")
    return topo

#Test detect cycle
graph = toGraph(6, [[1,0],[4,1],[4,5],[4,2],[0,2],[0,3],[2,3],[3,5],[5,2]], start=0)
graph1 = toGraph(5,[[0, 1],[0, 2],[1, 2],[1, 3],[2, 3],[2,4]],start=0)

topsortKahn(graph)

# topsortKahn(graph1, start=1)


Detect cycle


[4, 1, 0]

#### Shortest path algorithm

**1. Breadth-First Search (BFS)**

* **Use case:** Finding shortest paths in unweighted graphs or graphs with equal edge weights.
* **Time complexity:** O(V + E), where V is the number of vertices and E is the number of edges.

In [ ]:
from collections import deque

def bfs_shortest_path(graph, start, end):
    """
    Finds the shortest path between two nodes in an unweighted graph using BFS.

    Args:
        graph: A dictionary representing the graph where keys are nodes
               and values are lists of adjacent nodes.
        start: The starting node.
        end: The ending node.

    Returns:
        A list representing the shortest path, or None if no path exists.
    """
    queue = deque([(start, [start])])
    visited = set()

    while queue:
        node, path = queue.popleft()
        visited.add(node)
        if node == end:
            return path
        for neighbor in graph[node]:
            if neighbor not in visited:
                queue.append((neighbor, path + [neighbor]))
    return None


**2. Dijkstra's Algorithm**

* **Use case:** Finding shortest paths in weighted graphs with non-negative edge weights.
* **Time complexity:** O(E + V log V) using a binary heap.


In [68]:
import heapq

def dijkstra(graph, start, end):
    """
    Finds the shortest path between two nodes in a weighted graph with non-negative
    edge weights using Dijkstra's algorithm.

    Args:
        graph: A dictionary representing the graph where keys are nodes
               and values are dictionaries mapping neighbors to edge weights.
        start: The starting node.
        end: The ending node.

    Returns:
        A tuple containing the shortest distance and the path, or (float('inf'), None)
        if no path exists.
    """
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    visited = set()
    pq = [(0, start)]

    while pq:
        curr_dist, curr_node = heapq.heappop(pq)
        if curr_node in visited:
            continue
        visited.add(curr_node)
        if curr_node == end:
            break  # Early stopping if the end node is reached
        for neighbor, weight in graph[curr_node].items():
            new_dist = curr_dist + weight
            if new_dist < distances[neighbor]:
                distances[neighbor] = new_dist
                heapq.heappush(pq, (new_dist, neighbor))

    # Reconstruct the path
    path = []
    if distances[end] != float('inf'):
        node = end
        while node != start:
            path.append(node)
            neighbors = graph[node]
            node = min(neighbors, key=lambda neighbor: distances[neighbor] + neighbors[neighbor])
        path.append(start)
        path.reverse()

    return distances[end], path

**3. Bellman-Ford Algorithm**

* **Use case:** Finding shortest paths in weighted graphs, even with negative edge weights (but no negative cycles).
* **Time complexity:** O(V * E)

In [67]:
def bellman_ford(graph, start, end):
    """
    Finds the shortest path between two nodes in a weighted graph,
    allowing for negative edge weights (but no negative cycles),
    using the Bellman-Ford algorithm.

    Args:
        graph: A dictionary representing the graph where keys are nodes
               and values are dictionaries mapping neighbors to edge weights.
        start: The starting node.
        end: The ending node.

    Returns:
        A tuple containing the shortest distance and the path, or (float('inf'), None)
        if no path exists or a negative cycle is detected.
    """
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    predecessors = {node: None for node in graph}

    for _ in range(len(graph) - 1):
        for node in graph:
            for neighbor, weight in graph[node].items():
                if distances[node] != float('inf') and distances[node] + weight < distances[neighbor]:
                    distances[neighbor] = distances[node] + weight
                    predecessors[neighbor] = node

    # Check for negative cycles
    for node in graph:
        for neighbor, weight in graph[node].items():
            if distances[node] != float('inf') and distances[node] + weight < distances[neighbor]:
                return float('inf'), None  # Negative cycle detected

    # Reconstruct the path
    path = []
    if distances[end] != float('inf'):
        node = end
        while node is not None:
            path.append(node)
            node = predecessors[node]
        path.reverse()

    return distances[end], path


**4. Floyd-Warshall Algorithm**

* **Use case:** Finding shortest paths between all pairs of nodes in a weighted graph.
* **Time complexity:** O(V^3)


In [66]:
def floyd_warshall(graph):
    """
    Finds the shortest paths between all pairs of nodes in a weighted graph
    using the Floyd-Warshall algorithm.

    Args:
        graph: A dictionary representing the graph where keys are nodes
               and values are dictionaries mapping neighbors to edge weights.

    Returns:
        A 2D list representing the shortest distances between all pairs of nodes.
    """
    nodes = list(graph.keys())
    num_nodes = len(nodes)
    distances = [[float('inf')] * num_nodes for _ in range(num_nodes)]

    # Initialize distances with edge weights and self-distances
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i == j:
                distances[i][j] = 0
            elif nodes[j] in graph[nodes[i]]:
                distances[i][j] = graph[nodes[i]][nodes[j]]

    # Compute shortest paths
    for k in range(num_nodes):
        for i in range(num_nodes):
            for j in range(num_nodes):
                distances[i][j] = min(distances[i][j], distances[i][k] + distances[k][j])

    return distances

**5. A\* Search Algorithm**

* **Use case:** Finding shortest paths in weighted graphs, often used in pathfinding with a heuristic function to guide the search.
* **Time complexity:** Depends on the heuristic function and the graph structure.


In [65]:
import heapq

def astar(graph, start, end, heuristic):
    """
    Finds the shortest path between two nodes in a weighted graph using
    the A* search algorithm with a heuristic function.

    Args:
        graph: A dictionary representing the graph where keys are nodes
               and values are dictionaries mapping neighbors to edge weights.
        start: The starting node.
        end: The ending node.
        heuristic: A function that estimates the distance from a node to the end node.

    Returns:
        A tuple containing the shortest distance and the path, or (float('inf'), None)
        if no path exists.
    """
    open_set = set([start])
    closed_set = set()
    g_score = {node: float('inf') for node in graph}
    g_score[start] = 0
    f_score = {node: float('inf') for node in graph}
    f_score[start] = heuristic(start, end)
    came_from = {}

    while open_set:
        current = min(open_set, key=lambda node: f_score[node])
        if current == end:
            # Reconstruct the path
            path = [current]
            while current in came_from:
                current = came_from[current]
                path.append(current)
            path.reverse()
            return g_score[end], path

        open_set.remove(current)
        closed_set.add(current)

        for neighbor, weight in graph[current].items():
            if neighbor in closed_set:
                continue
            tentative_g_score = g_score[current] + weight
            if neighbor not in open_set:
                open_set.add(neighbor)
            elif tentative_g_score >= g_score[neighbor]:
                continue

            came_from[neighbor] = current
            g_score[neighbor] = tentative_g_score
            f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, end)

    return float('inf')

### Advance Graphs


#### [332. Reconstruct Itinerary](https://leetcode.com/problems/reconstruct-itinerary/description/)

**DFS**: `all tickets form at least one valid itinerary` ==> Exist 1 DFS
- Sort list lexically to have the same node always visit smaller lexical edge.

In [ ]:
class Solution:
    def findItinerary(self, tickets: List[List[str]]) -> List[str]:
        

#### [787. Cheapest Flights Within K Stops](https://leetcode.com/problems/cheapest-flights-within-k-stops/description/) Bellman Ford

## 1-D Dynamic Programming
Qs to look back:
OA cisco 12/31/24
https://www.geeksforgeeks.org/count-numbers-smaller-than-or-equal-to-n-with-given-digit-sum/


#### [70. Climbing Stairs](https://leetcode.com/problems/climbing-stairs/)
- Value of DP[i] = dp[i-1]+dp[i-2] (similar to Fibonacci) because at Stair[]
- Instead of use dp[i] cost O(n) space complexity -> Use 2 vars "one", "two" to store dp[i-1], dp[i-2]

<img src="assets/70. Climbing Stairs.png" width=400></img>

dp shows number of ways to reach the top from 3 to 0 (4,5 is init as 1)

In [ ]:
#As from 3, it takes 1 way to reach 5, 1 way to reach 4 
#-> Two init as 1 because dp[n-2] takes 1 step to reach two init.
class Solution:
    def climbStairs(self, n: int) -> int:
        one, two = 1,1
        #Shift one and two value
        for i in range(n-1): #stop at n-2
            temp = one
            one = one + two
            two = temp
        return one

#### [746. Min Cost Climbing Stairs](https://leetcode.com/problems/min-cost-climbing-stairs/description/)

In [ ]:
"""_summary_
"""
class Solution:
    def minCostClimbingStairs(self, cost: List[int]) -> int:
        mincost, curcost = 0
        
        

In [70]:
def fib(n,memo):
    if n in memo:
        return memo[n]
    
    memo[n] = fib(n-1, memo)+fib(n-2, memo)
    print(memo)
    return memo[n]
fib(7, {0:0, 1:1})

{0: 0, 1: 1, 2: 1}
{0: 0, 1: 1, 2: 1, 3: 2}
{0: 0, 1: 1, 2: 1, 3: 2, 4: 3}
{0: 0, 1: 1, 2: 1, 3: 2, 4: 3, 5: 5}
{0: 0, 1: 1, 2: 1, 3: 2, 4: 3, 5: 5, 6: 8}
{0: 0, 1: 1, 2: 1, 3: 2, 4: 3, 5: 5, 6: 8, 7: 13}


13

In [ ]:
#152. Maximum Product Subarray
class Solution:
    def maxProduct(self, nums: List[int]) -> int:
        curMax, curMin = 1, 1
        res = nums[0]
        
        for n in nums:
            vals = (n, n * curMax, n * curMin)
            curMax, curMin = max(vals), min(vals)
			
            res = max(res, curMax)
            
        return res

In [ ]:
#198. House Robber
class Solution:
    def rob(self, nums: List[int]) -> int:
        prev_rob = max_rob = 0

        for cur_val in nums:
            temp = max(max_rob, prev_rob + cur_val)
            prev_rob = max_rob
            max_rob = temp
        
        return max_rob

## 2D Arrays

## Greedy

## Other leets

In [14]:
def solution(matrix):
    rows, cols = len(matrix), len(matrix[0])
    
    # Directions for diagonals: (row increment, col increment)
    directions = [
        (1, 1),  # Top-left to bottom-right
        (1, -1), # Top-right to bottom-left
        (-1, 1), # Bottom-left to top-right
        (-1, -1) # Bottom-right to top-left
    ]
    
    # Function to check the diagonal length
    def get_diagonal_length(r, c, dr, dc):
        length = 0
        current_pattern = 1  # First element must always be 1
        
        while 0 <= r < rows and 0 <= c < cols:
            if matrix[r][c] != current_pattern:
                break
            length += 1
            
            # Update the next expected value in the pattern
            if current_pattern == 1:
                current_pattern = 2
            elif current_pattern == 2:
                current_pattern = 0
            elif current_pattern == 0:
                current_pattern = 2  # Alternates between 2 and 0
            
            # Move to the next diagonal element
            r += dr
            c += dc
        
        return length
    
    # Iterate through the matrix
    max_length = 0
    for r in range(rows):
        for c in range(cols):
            if matrix[r][c] == 1:  # Start of the pattern
                # Explore all four directions
                for dr, dc in directions:
                    max_length = max(max_length, get_diagonal_length(r, c, dr, dc))
    
    return max_length


matrix = [
    [2, 1, 2, 2, 0],
    [0, 2, 0, 2, 0],
    [0, 0, 0, 0, 0],
    [0, 2, 2, 2, 2],
    [2, 1, 2, 2, 1],
    [0, 2, 0, 0, 2]
]
# matrix = [[0,0,1,1],
#           [2,2,2,0],
#           [2,1,0,1]]
solution(matrix)  # Output: 4



4

In [25]:
def solution(cityLine):
    n = len(cityLine)
    max_square = 0
    
    # Monotonic stack to find left and right limits
    left = [-1] * n  # Stores the index of the nearest smaller element to the left
    right = [n] * n  # Stores the index of the nearest smaller element to the right
    stack = []
    
    # Calculate left limits
    for i in range(n):
        while stack and cityLine[stack[-1]] >= cityLine[i]:
            stack.pop()
        if stack:
            left[i] = stack[-1]
        stack.append(i)
    
    # Clear the stack for reuse
    stack = []
    
    # Calculate right limits
    for i in range(n - 1, -1, -1):
        while stack and cityLine[stack[-1]] >= cityLine[i]:
            stack.pop()
        if stack:
            right[i] = stack[-1]
        stack.append(i)
    
    # Calculate the maximum square
    for i in range(n):
        # Maximum width for the current bar
        width = right[i] - left[i] - 1
        # The largest square side is min(width, cityLine[i])
        side = min(width, cityLine[i])
        # Update max square area
        max_square = max(max_square, side * side)
    
    return max_square

cityLine = [1, 2, 3, 2, 1]
cityLine = [4,3,4]
solution(cityLine)

9

#### [contiguous subarrays that represent a sawtooth sequence of at least two elements.](https://www.geeksforgeeks.org/count-subarrays-with-elements-in-alternate-increasing-decreasing-order-or-vice-versa/)
Notes:
- All cases Continuous flips(n) is a sum of arithm sequence: n(n+1)//2
    - Continuous flips(5) = 5*6//2 = 15
    - Explanation:
    
        Arithmetic sequence: Continuous flips(5) = 1+2+3+4+5
        
        Continuous flips(n) = Continuous flips(n-1) + n
        
    - e.g: continuous flips(4) = 10 (e.g [1, 10, 2, 8, 3]) 
    
    - continuous flips(5) (e.g [1, 10, 2, 8, 3, 9]) 
    
        = 15 
        
        = All continuous flips(4) + 5 new subs from 5th (9):
    [3, 9], [8, 3, 9], [2, 8, 3, 9], [10, 2, 8, 3, 9], [1, 10, 2, 8, 3, 9]

In [ ]:
def solution(arr):
    totalCount, flips = 0, 0
    up = -1
    if arr[0] < arr[1]:
        up = 1
        flips = 1
    elif arr[0] > arr[1]:
        up = 0
        flips = 1
        
    for i in range(1, len(arr)-1):
        #Keep flipping | or | end flipping and reset
        if up == 0 and arr[i] < arr[i+1] :
            up = 1
            flips+=1
        elif up == 1 and arr[i] > arr[i+1] :
            up = 0
            flips+=1            
        #end flipping and reset, flips = 1 -> res = 1, flips = 4 -> res = 10
        else:
            totalCount += flips*(flips+1)//2
            flips = 1 if arr[i] != arr[i+1] else 0        
    #unresolve flips
    if flips > 0:
        totalCount+= flips*(flips+1)//2
    return totalCount
arr1= [-779, -510, -829, 190, -35, -35, 106, -888, -400, -320, -320, 564, -513, 634, -280, -280, -73, -771, -630, 112]
print(solution(arr1)) #34


arr = [9,8,7,6,5]
print(solution(arr)) # 4

arr2 = [1,2,1,3,4,-2]
print(solution(arr2)) # 9

arr3 = [1,2,1,2,1]
print(solution(arr3)) # 10

arr4 = [10,10,10]
print(solution(arr4)) # 0

# from medium article comments
arr5 = [-442024811,447425003,365210904,823944047,943356091,-781994958,872885721,-296856571,230380705,944396167,-636263320,-942060800,-116260950,-126531946,-838921202]
print(solution(arr5)) # 31

34
4
9
10
0
31


#### [2043. Simple Bank System](https://leetcode.com/problems/simple-bank-system/)

In [ ]:
from typing import List
class Bank:
    
    #e.g account 3 's balance is balance[2]
    def __init__(self, balance: List[int]):
        self.balance = balance
    def isValid(self, account1: int, money: int) -> bool:
        # a = len(self.balance)
        # if account1 > a:
        #     return False
        # if self.balance[account1-1] < money:
        #     return False
        # return True
        return account1 <= len(self.balance) and self.balance[account1-1] >= money
        
    def transfer(self, account1: int, account2: int, money: int) -> bool:
        if not self.isValid(account1, money):
            return False
        self.balance[account1-1] -= money
        self.balance[account2-1] +=money
        return True

    def deposit(self, account: int, money: int) -> bool:
        if account > len(self.balance):
            return False
        self.balance[account-1] += money
        return True

    def withdraw(self, account: int, money: int) -> bool:
        if not self.isValid(account, money):
            return False
        self.balance[account-1] -= money
        return True
    def perform(self, ops, data) -> list:
        res = ["null"]
        for i in range(1, len(ops)):
            if ops[i] == "deposit":
                res.append(str(self.deposit(data[i][0],data[i][1])))
            if ops[i] == "transfer":
                res.append(str(self.transfer(data[i][0],data[i][1], data[i][2])))
            if ops[i] == "withdraw":
                res.append(self.withdraw(data[i][0],data[i][1]))
        return res

# Your Bank object will be instantiated and called as such:
ops = ["Bank","withdraw","transfer","deposit","transfer","withdraw"]
data = [[[10,100,20,50,30]],[3,10],[5,1,20],[5,20],[3,4,15],[10,50]]
mbank = Bank(data[0][0])

print(mbank.perform(ops, data))

['null', True, 'True', 'True', 'False', False]


#### [1492. The kth Factor of n](https://leetcode.com/problems/the-kth-factor-of-n/description/)

In [ ]:
%%time
def kthFactor(n: int, k: int) -> int:
    #we could add 2 factors at one time, start with 1,n on the outside and work inwards
    #work from 1 -> sqrt(n): return i (ascd i)
    #keep going with sqrt(n)->1 : return n//i (desc i 0> => ascd n//i )
    for i in range(1, int(n**0.5)+1): #1-> sqrt(n)
        if n%i ==0:
            k -= 1
            if k == 0:
                print(res)
                return i

    for i in range(int(n**0.5), 0,-1 ): #1-> sqrt(n)+1 -> 1
        if n//i==i: continue #exclude repeated factors
        if n%i ==0:
            k -= 1
            if k == 0:
                return n//i
    return -1
    
n = 2287
k = 8
print(kthFactor(n,k))
#7625597484987

-1
CPU times: user 48 µs, sys: 4 µs, total: 52 µs
Wall time: 53.9 µs


#### [Sum of all the multiples of 3 or 5 below N](https://www.hackerrank.com/contests/projecteuler/challenges/euler001/problem?isFullScreen=true)

iSum of Arithmetic sequence:

        an = a1 + (n-1)d

        => n = (an - a1)//d + 1

        S = n/2(a1+an) (basic) 
    
        or S = n/2(2a1 + (n-1)d)
or (1) Triangle number:
<img src ="assets/triangle number.png" width=300></img>

    

In [ ]:
def mul(n):
    def sumseq(n,m):
        #Find last term below n (100) 99-95
        last = n-1 - ((n-1)%m)
        #Find #terms 
        k = (last - m)//m + 1
        #Sum
        return (k*(last + m)//2)
    def sumtri(n,m):
        d = n//m #
        return m * (d*(d+1))//2
    # return sumseq(n,3)+sumseq(n,5) - sumseq(n,15)
    return sumtri(n,3)+sumtri(n,5) - sumtri(n,15)
mul(29)

195

In [ ]:
last = 15 - 1 - ((15 - 1) % 3) 
k = (15 - 3) // 3 + 1 
result1 = int(5 / 2 * (15 + 3))  # Result1 = 37
result2 = (5 * (15 + 3)) // 2     # Result2 = 40
result1,result2

(45, 45)

#### [Bitwise AND count pairs](https://www.geeksforgeeks.org/count-of-pairs-whose-bitwise-and-is-a-power-of-2/) xxxx dont understand

Power of 2 (num) means: num and num&(a-1)==0

In [ ]:
from collections import defaultdict
def countPairs(arr: list[int], n: int) -> int:
    # Initialize answer and maximum value in the array
    ans, mx = 0, 0
    # Create a defaultdict to store the frequency of each integer in the array
    mp = defaultdict(int)
    # Iterate through each integer in the array
    for ai in arr:
        # Update the frequency of each integer in the defaultdict
        mp[ai] += 1
        # Update the maximum value in the array
        mx = max(mx, ai)
    # Iterate through each integer i from 0 to mx
    for i in range(mx+1):
        # If i is not present in the defaultdict, skip to the next integer
        if i not in mp:
            continue
        # Iterate through each integer j from i to mx
        for j in range(i, mx+1):
            # If j is not present in the defaultdict, skip to the next integer
            if j not in mp:
                continue
            # Check if the bitwise AND of i and j has only one set bit
            if bin(i & j).count('1') == 1:
                # If i is equal to j, add the product of nCr(mp.get(i), 2) 
                # to the answer
                if i == j:
                    ans += (mp[i] * (mp[i]-1)) // 2
                # If i is not equal to j, add the product of mp.get(i) 
                # and mp.get(j) to the answer
                else:
                    ans += mp[i] * mp[j]
    # Return the answer
    return ans
arr = [6, 4, 2, 3]
n = len(arr)
print(countPairs(arr, n))

4


#### [Largest Rectangle Area after drawing each boundary](https://leetcode.com/discuss/interview-question/1225894/largest-rectangle-area-after-drawing-each-boundary)

Passed sample

In [ ]:
import math

class Node:
    def __init__(self, parent, l, r, op=max):
        self.parent = parent
        self.l = l
        self.r = r
        self.lc = None
        self.rc = None
        self.val = r - l
        self.op = op
    
    def split(self, x):
        # No balancing, but doesn't seem to give timeouts.
        assert self.l <= x <= self.r
        if x == self.l or x == self.r:
            # Split lies on borders.
            return
        if self.lc:
            if x == self.lc.r:
                # Split lies on mid split.
                return
            if x < self.lc.r:
                self.lc.split(x)
            else:
                self.rc.split(x)
            self.val = self.op(self.lc.val, self.rc.val)
        else:
            self.lc = Node(parent=self, l=self.l, r=x)
            self.rc = Node(parent=self, l=x, r=self.r)
            self.val = self.op(x - self.l, self.r - x)
        
def getMaxArea(w, h, isVertical, distance):
    w_root = Node(parent=None, l=0, r=w)
    h_root = Node(parent=None, l=0, r=h)
    ans = []
    for iv, d in zip(isVertical, distance):
        if iv:
            w_root.split(d)
        else:
            h_root.split(d)
        ans.append(w_root.val * h_root.val)
    return ans
    

w = 4
h = 4
isVertical = [0,1]
distance = [3,1]
print(getMaxArea(w,h,isVertical,distance))


[12, 9]


Below passed 11/15 tests, passed cert

In [ ]:
def getMaxArea(w, h, isVertical, distance):
    # Write your code here
    def insort(test_list, k):
        for i in range(len(test_list)):
            if test_list[i] > k:
                break
        return test_list[:i] + [k] + test_list[i:]
        
    maxy = 0
    wline = [0,w]
    hline = [0,h]
    res = []
    for i in range(len(isVertical)):
        if isVertical[i]:
            wline = insort(wline, distance[i])
        else:
            hline = insort(hline, distance[i])
        wmax = 0
        hmax = 0
        for i in range(1,len(wline)):
            if wline[i] - wline[i-1] > wmax:
                wmax = wline[i] - wline[i-1]
        for i in range(1,len(hline)):
            if hline[i] - hline[i-1] > hmax:
                hmax = hline[i] - hline[i-1]
     
        res.append(hmax * wmax)
    return res


#### Road Repairs

In [ ]:
#(5-9, 3-3, 1-1, 4-15, 6-8)
def getMinCost(crew_id, job_id):
    crew_id.sort()
    job_id.sort()
    cost = 0
    for i in range(len(crew_id)):
        cost += abs(job_id[i] - crew_id[i])
    return cost

crewId = [4,5,1,4,2]
jobId = [4,4,5,9,10] 
print(getMinCost(crewId, jobId))

16


In [1]:
def image_blur(image, radius):
    rows, cols = len(image), len(image[0])
    result = [[0] * cols for _ in range(rows)]
    
    for i in range(rows):
        for j in range(cols):
            neighbors = []
            
            # Collect valid neighbors within the specified radius, excluding (i, j) itself
            for k in range(max(0, i - radius), min(rows, i + radius + 1)):
                for l in range(max(0, j - radius), min(cols, j + radius + 1)):
                    if k == i and l == j:
                        continue  # Skip the center cell
                    neighbors.append(image[k][l])
            
            # Calculate mean of neighbors and apply the formula
            if neighbors:
                mean_neighbors = sum(neighbors) // len(neighbors)
                result[i][j] = (image[i][j] + mean_neighbors) // 2
            else:
                # No neighbors, so the pixel remains the same
                result[i][j] = image[i][j]
                
    return result
image = [[9, 6], [3,0]]

image_blur(image, 1)

[[6, 5], [4, 3]]

In [5]:
def solution(paragraphs, width):
    """
    Formats text on a newspaper page according to the given specifications.

    Args:
      paragraphs: A list of paragraphs, where each paragraph is a list of words.
      width: The maximum number of characters per line.

    Returns:
      A list of strings representing the formatted newspaper page.
    """
    result = []
    for paragraph in paragraphs:
        # Add a blank line between paragraphs only if the previous paragraph added content
        if result and any(paragraph):
            result.append("*" + " " * width + "*")
        
        line = ""
        for word in paragraph:
            if len(line) + len(word) + (1 if line else 0) > width:
                # Calculate spaces for center alignment
                spaces = width - len(line)
                leading_spaces = spaces // 2
                trailing_spaces = spaces - leading_spaces
                result.append("*" + " " * leading_spaces + line + " " * trailing_spaces + "*")
                line = word
            else:
                line += (" " if line else "") + word
        
        # Add the last line of the paragraph if any content remains
        if line:
            spaces = width - len(line)
            leading_spaces = spaces // 2
            trailing_spaces = spaces - leading_spaces
            result.append("*" + " " * leading_spaces + line + " " * trailing_spaces + "*")
    
    # Add the border
    result.insert(0, "*" * (width + 2))
    result.append("*" * (width + 2))

    return result

# Example usage
paragraphs = [
    ["Ann", "and", "Bob", "loved", "each", "other"],
]
width = 9
formatted_page = solution(paragraphs, width)
print("\n".join(formatted_page))


***********
* Ann and *
*Bob loved*
*  each   *
*  other  *
***********


**need to be solved**

#### [Total number of triangles formed when there are H horizontal and V vertical lines](https://www.geeksforgeeks.org/total-number-of-triangles-formed-when-there-are-h-horizontal-and-v-vertical-lines/)
[Largest area possible after removal of a series of horizontal & vertical bars](https://www.geeksforgeeks.org/largest-area-possible-after-removal-of-a-series-of-horizontal-vertical-bars/)

[Count number of triangles cut by the given horizontal and vertical line segments](https://www.geeksforgeeks.org/count-number-of-triangles-cut-by-the-given-horizontal-and-vertical-line-segments/)


#### [1465. Maximum Area of a Piece of Cake After Horizontal and Vertical Cuts](https://leetcode.com/problems/maximum-area-of-a-piece-of-cake-after-horizontal-and-vertical-cuts/)

#### OA 1
<img src="./assets/oa1.png" width="400px"/>

In [3]:
def solution(matrix, radius):
    rows = len(matrix)
    cols = len(matrix[0])
    max_sum = float('-inf')
    
    # Function to calculate the sum of elements in the rhombic area
    def rhombic_sum(centerX, centerY):
        total = 0
        for dx in range(-radius + 1, radius):  # range of x-offsets
            for dy in range(-radius + 1, radius):  # range of y-offsets
                # Calculate the Manhattan distance
                if abs(dx) + abs(dy) < radius:
                    x = centerX + dx
                    y = centerY + dy
                    # Check if (x, y) is within matrix bounds
                    if 0 <= x < rows and 0 <= y < cols:
                        total += matrix[x][y]
        return total

    # Iterate over each possible center in the matrix
    for i in range(rows):
        for j in range(cols):
            # Check if rhombic area is within bounds
            if i + radius <= rows and j + radius <= cols and i - radius >= -1 and j - radius >= -1:
                current_sum = rhombic_sum(i, j)
                max_sum = max(max_sum, current_sum)
    
    return max_sum


# Example usage:
matrix = [[0, 2, 4, 1, 6, 4],
          [5, 1, 3, 4, 1, 5],
          [0, 1, 2, 1, 2, 1],
          [1, 3, 2, 1, 1, 2],
          [4, 1, 3, 6, 5, 5],
          [6, 7, 5, 3, 1, 2]]
radius = 3
print(solution(matrix, radius))  # Output should be 35.


35


#### OA2

Given an infinite integer number line, you would like to build some blocks and obstacles on it. Specifically, you have to implement code which supports two types of operations:

[1, x] - builds an obstacle at coordinate x along the number line. It is guaranteed that coordinate x does not contain any obstacles when the operation is performed.

[2, x, size] - checks whether it's possible to build a block of size size which ends immediately before x on the number line. For example, if x = 6 and size = 2, this operation checks coordinates 4 and 5. Produces "1" if it is possible, i.e. if there are no obstacles at the specified coordinates, or produces "0" otherwise. Please note that this operation does not actually build the block, it only checks whether a block can be built.

Given an array of operations containing both types of operations described above, your task is to return a binary string representing the outputs for all [2, x, size] operations.

Example

For

operations = [[1, 2],

              [1, 5],

              [2, 5, 2],

              [2, 6, 3],

              [2, 2, 1],

              [2, 3, 2]]

the output should be solution(operations) = "1010".

Explanation:

Let's consider all operations:

[1, 2] - builds an obstacle at coordinate 2.

[1, 5] - builds an obstacle at coordinate 5.

[2, 5, 2] - checks and produces "1" as it is possible to build a block occupying coordinates 3 and 4.

[2, 6, 3] - checks and produces "0" as it is not possible to build a block occupying coordinates 3, 4, and 5, because there is an obstacle at coordinate 5.

[2, 2, 1] - checks and produces "1" as it is possible to build a block occupying coordinate 1.

[2, 3, 2] - checks and produces "0" as it is not possible to build a block occupying coordinates 1 and 2 because there is an obstacle at coordinate 2.

So the output is "1010". 


In [18]:
from sortedcontainers import SortedList

def solution(operations):
    obstacles = SortedList()
    result = []

    for op in operations:
        if op[0] == 1:
            # Operation [1, x]: Place an obstacle at x
            obstacles.add(op[1])
        elif op[0] == 2:
            # Operation [2, x, size]: Check if we can place a block of 'size' ending at 'x-1'
            x, size = op[1], op[2]
            start = x - size
            # Find the first obstacle that is >= start
            idx = obstacles.bisect_left(start)
            
            # If there's an obstacle before 'x' in the range [start, x-1], it's not possible
            if idx < len(obstacles) and obstacles[idx] < x:
                result.append('0')
            else:
                result.append('1')

    return ''.join(result)

# Example usage
operations = [[1, 2], [1, 5], [2, 5, 2], [2, 6, 3], [2, 2, 1], [2, 3, 2]]
print(solution(operations))  # Output: "1010"

1010
CPU times: user 39 µs, sys: 4 µs, total: 43 µs
Wall time: 44.1 µs


In [11]:
def countMinimumOperations(prices, queries):
    # Step 1: Sort the prices for efficient processing
    prices.sort()
    
    # Step 2: Precompute cumulative sums (prefix sums)
    n = len(prices)
    cumulative_sums = [0] * (n + 1)
    for i in range(n):
        cumulative_sums[i + 1] = cumulative_sums[i] + prices[i]

    # Step 3: Process each query
    results = []
    for target in queries:
        # Find the split point: how many items are less than the target
        low, high = 0, n
        while low < high:
            mid = (low + high) // 2
            if prices[mid] < target:
                low = mid + 1
            else:
                high = mid
        # `low` now represents the number of items less than the target

        # Calculate the operations for items below and above the target
        below_cost = target * low - cumulative_sums[low]
        above_cost = (cumulative_sums[n] - cumulative_sums[low]) - target * (n - low)

        # Store the total operations for this query
        results.append(below_cost + above_cost)

    return results
prices = [1, 2, 3]
queries = [3, 2, 1, 5]

countMinimumOperations(prices, queries)

def smashTheBricks(bigHits, newtons):
    n = len(newtons)
    
    # Step 1: Sort bricks by force (newtons), keeping track of original indices
    idx = [(force, idx + 1) for idx, force in enumerate(newtons)]
    idx.sort(reverse=True, key=lambda x: x[0])  # Sort by force (descending)
    
    # Step 2: Use the big hammer
    bigs = []
    smalls = []
    blows = 0
    
    for i, (force, idx) in enumerate(idx):
        if i < bigHits:
            bigs.append(idx)  # Smash with big hammer
            blows += 1  # 1 blow per brick for big hammer
        else:
            smalls.append(idx)  # Smash with small hammer
            blows += force  # Small hammer requires 'force' blows
    
    # Step 3: Sort indices of hits
    bigs.sort()
    smalls.sort()
    
    # Step 4: Handle case where no big or small hammer is used
    if not bigs:
        bigs = [-1]
    if not smalls:
        smalls = [-1]
    
    # Step 5: Return the results
    return [[blows], bigs, smalls]

# Example Usage
bigHits = 4
newtons = [3, 2, 5, 4, 6, 7, 9]
smashTheBricks(bigHits, newtons)


[[13], [3, 5, 6, 7], [1, 2, 4]]

In [14]:
def count_numbers_with_total(X, Y):
    # Convert X to a string for easier manipulation
    X_str = str(X)

    # Memoization for digit DP
    def digit_dp(pos, temp, total):
        # Base case: all positions processed
        if pos == len(X_str):
            return 1 if total == 0 else 0
        
        # If total becomes negative, it's invalid
        if total < 0:
            return 0
        
        # Memoize states (position, temp, total)
        if (pos, temp, total) in memo:
            return memo[(pos, temp, total)]
        
        # Determine the limit for the current digit
        limit = int(X_str[pos]) if temp else 9
        result = 0
        
        # Try all digits from 0 to `limit`
        for digit in range(0, limit + 1):
            result += digit_dp(
                pos + 1,
                temp and (digit == limit),
                total - digit
            )
        
        # Store in memo
        memo[(pos, temp, total)] = result
        return result

    # Initialize memoization dictionary
    memo = {}
    
    # Start digit DP from position 0, temp bound, and the target digit sum
    result = digit_dp(0, True, Y)
    return result if result > 0 else -1


# Example usage
inputNum1 = 100  # X
inputNum2 = 10   # Y
print(count_numbers_with_total(inputNum1, inputNum2)) 


9


## Bitwise

In [117]:
#2429. Minimize XOR
from collections import Counter
def minimizeXor(num1: int, num2: int) -> int:

    """ 
    xor is minimal, reduced to 0 when most bits are the same 
    => prioritize match leftmost bits (make large bits of xor == 0)
    
    rearrage num2 1s to match num1 as many as possbile, from left to right
    
    """
    #turns to bits f'{num1:08b}' or bin(num1)
    b1 = bin(num1)[2:]
    b2 = bin(num2)[2:]
    ones = Counter(b2)['1']
    num = ''
    print(ones, b2, b1)
    index1 = 0
    #match ones from left of num1
    while ones > 0 and index1 < len(b1):
        if b1[index1] == '1':
            num += '1' 
            ones -=1
        else:
            num += '0' 
        index1+=1
    # num1=11001; num=11001 so far with 1s left, where to put?-> num=11011
    #replace rightest 0 in num -> 1 to minimize digit diff with num1
    if ones > 0:
        for i in range(len(num)-1, -1, -1 ):
            if num[i] == '0':
                ones -=1
                num= num[:i] + '1' + num[i+1:]
            if ones == 0:
                break
    if ones > 0: #still some ones after replacement -> add to num's left to not affect xor calculated from right
        num = ones*'1' + num
    # still index in num1-> add 0s to num's right -> match rest of num1
    if index1 < len(b1):
        num+= (len(b1) - index1)*'0'
    return int(num, 2)
# num1 = 3
# num2 = 5
num1 = 65
num2 = 84
minimizeXor(num1, num2)

3 1010100 1000001


67

In [2]:
#2425. Bitwise XOR of All Pairings
'''
xor property: n ^ n = 0 ; 0 ^ n = n
commutative ( n1 ^ m1 ) ^ (n1 ^ m2) = n1^n1 ^ m1^m2 = m1^m2

Calculate (n1 n2 n3) and (m1 m2 m3 m4)
(n1^m1) ^ (n1^m2) ^ (n1^m3) ^ (n1^m4)^ (n2^m1) ^ (n2^m2) ^ (n2^m3) ^ (n2^m4)
^ (n3^m1) ^ (n3^m2) ^ (n3^m3) ^ (n2^m4)

= (n1, n2,n3 XOR itself 4 times) ^ (m1, m2,m3, m4 XOR itself 3 times) 

= 0 ^ m1 ^ m2 ^ m3 ^ m4 = m1^m2^m3^m4

=> result = (All n XOR itself len(m)) ^ (All m XOR itself len(n)) 
=> If len(m) % 2 == 0 => out = 0; else out = XOR of all n. Vice versa to len(n) 
'''
def xorAllNums(nums1: list[int], nums2: list[int]) -> int:
    n = len(nums1)
    m = len(nums2)
    xor = 0
    if m % 2 != 0: # if m even, nxor remain 0
        for n1 in nums1:
            xor ^= n1
    if n % 2 != 0: # if n even, mxor remain 0
        for m1 in nums2:
            xor ^= m1
    return xor


nums1 = [2,1,3]
nums2 = [10,2,5,0]
xorAllNums(nums1, nums2)

13

In [ ]:
''' 
3*7 < 4*6 .For any a <= b, (a−1)⋅(b+1)<a⋅b => By swap bits, incre smaller + decre larger, keep sum, increase product.
=> Maximize product: for all pairs a, b (i, 2^p-1 - i) for i starts at 1, decrease all i to 1.
 p = 4 => We have 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, largest is 2^p-1 = 15.
decrease all pairs (1, 14), (2, 13),...
we have: (1, 1, 1, 1, 1, 1, 1, 14, 14, 14, 14, 14, 14, 14, 15). after decrease all i (first smaller half) to 1.
-> minProduct = secondLargest**its counts * large = (large-1) ** ((large-1)//2) * large
'''
# pow(base,exp,mod)
def minNonZeroProduct( p: int) -> int:
    
    MOD = 10**9 + 7  # Modulo 
    large = 2 ** p - 1     #Largest number in the range
    # e.g p = 3 => (1, 2, 3, 4, 5, 6, 7) -> minProd = (1, 1, 1, 6, 6, 6, 7) 
    #minProduct = secondLargest**its counts * large = (large-1)**((large-1)//2) * large
    
    return (pow(large-1,(large-1)//2, MOD) * large) % MOD
minNonZeroProduct(3)

1512

## Sorting

In [77]:
cities = [
	{
		"name": "New York City",
		"country": "United States",
		"population": 20.14,
		"area": 1223.59
	},
	{
		"name": "Tokyo",
		"country": "Japan",
		"population": 37.47,
		"area": 2194.07,
	},
 	{
		"name": "Los Angeles",
		"country": "United States",
		"population": 13.2,
		"area": 1299.01,
	}
]
# Sort by country, then by name
cities = sorted(cities, key=lambda city: (city['country'], city['name']))
# Sort by population Ascending, then country DESCENDING, 
cities = sorted(cities, key=lambda city: (-city['population'], city['country']), reverse=True )

def complexFunc(city):
    
	return (city['population']-35) **2 #return a number or str, it will be the comparatees

sorted(cities, key=complexFunc, reverse=True)

[{'name': 'Los Angeles',
  'country': 'United States',
  'population': 13.2,
  'area': 1299.01},
 {'name': 'New York City',
  'country': 'United States',
  'population': 20.14,
  'area': 1223.59},
 {'name': 'Tokyo', 'country': 'Japan', 'population': 37.47, 'area': 2194.07}]

# CODEBOOK

42. Trapping Rain Water 2 POINTERS
```python
class Solution:
    def trap(self, height):
        """
        :type height: List[int]
        :rtype: int
        """
        # Step 1: Initialize Pointers and Variables
        left, right = 0, len(height) - 1  # Two pointers at the start and end of the array
        ans = 0  # Stores total trapped water
        left_max, right_max = 0, 0  # Track max height seen so far from left and right
        
        # Step 2: Two-Pointer Traversal
        while left < right:
            # Step 2a: Process the Shorter Side First
            if height[left] < height[right]:
                # Step 2b: Update left_max and Calculate Water Trapped
                left_max = max(left_max, height[left])  # Update max height seen on the left
                ans += left_max - height[left]  # Water trapped at index left
                left += 1  # Move left pointer to the right
            else:
                # Step 2c: Update right_max and Calculate Water Trapped
                right_max = max(right_max, height[right])  # Update max height seen on the right
                ans += right_max - height[right]  # Water trapped at index right
                right -= 1  # Move right pointer to the left
        
        return ans  # Return the total trapped water
```

1. **Two-Pointer Approach**:
   - Instead of using a stack, this solution **uses two pointers (`left` and `right`)** to track the boundaries.
   - It efficiently calculates trapped water in **O(n) time** without extra space.

2. **Core Idea**:
   - **Water trapped at any index depends on the shorter boundary** (either `left_max` or `right_max`).
   - The algorithm **processes the shorter side first** because water is limited by the shorter boundary.

3. **How It Works**:
   - **If `height[left] < height[right]`**: Process the left side, because water is trapped **only up to `left_max`**.
   - **Otherwise**: Process the right side, because water is trapped **only up to `right_max`**.
   - **Update `left_max` and `right_max`** as we move the pointers.

4. **Efficiency**:
   - **Time Complexity:** `O(n)` (Each element is processed once)
   - **Space Complexity:** `O(1)` (Only a few variables are used, no extra memory)


In [ ]:
class Solution:
    def vowelStrings(self, words: List[str], queries: List[List[int]]) -> List[int]:
        # Step 1: Initialize Variables
        n = len(words)  # Number of words
        Prefix = [0] * (n + 1)  # Prefix array to store cumulative counts of valid words
        vowels = {'a', 'e', 'i', 'o', 'u'}  # Set of vowels for quick lookup

        # Step 2: Build the Prefix Array
        for i in range(n):
            # Carry forward the previous count
            Prefix[i + 1] = Prefix[i]
            # Check if the word starts and ends with a vowel
            if words[i][0] in vowels and words[i][-1] in vowels:
                Prefix[i + 1] += 1  # Increment the count if valid

        # Step 3: Process Queries Using the Prefix Array
        res = []  # Result array to store answers for each query
        for query in queries:
            # Calculate the count of valid words in the range [query[0], query[1]]
            res.append(Prefix[query[1] + 1] - Prefix[query[0]])

        # Step 4: Return the Results
        return res


In [ ]:
#2524. Maximum Frequency Score of a Subarray - slide window
from collections import Counter

class Solution:
    def max_frequency_score(self, nums, k):
        # Define the modulus for the calculations as per the problem statement, which is 10^9 + 7
        mod = 10**9 + 7

        # Create a Counter for the first 'k' elements in the list
        counter = Counter(nums[:k])

        # Calculate the initial score by raising 'k' to the power of the frequency count, modulo 'mod'
        current_score = sum(pow(num, frequency, mod) for num, frequency in counter.items()) % mod

        # Set the 'answer' to the initial score, it will keep track of the highest score
        answer = current_score

        # Start iterating from the 'k'th element in 'nums'
        i = k
        while i < len(nums):
            # 'previous_num' is the number which will be excluded from the current window
            # 'new_num' is the number which will be included in the current window
            previous_num, new_num = nums[i - k], nums[i]

            # Only process if we're examining a new number; otherwise, skip re-calculation
            if previous_num != new_num:
                # Increase the score by the power of the count of the new number, if it already exists
                if counter[new_num]:
                    current_score += (new_num - 1) * pow(new_num, counter[new_num], mod)
                else:
                    current_score += new_num

                # Decrease the score by the power of the count of the previous number, if necessary
                if counter[previous_num] > 1:
                    current_score -= (previous_num - 1) * pow(previous_num, counter[previous_num] - 1, mod)
                else:
                    current_score -= previous_num

                # Ensure the current score does not exceed 'mod'
                current_score %= mod

                # Update the counter for the new window
                counter[new_num] += 1
                counter[previous_num] -= 1

                # Update the answer if the current score is higher
                answer = max(answer, current_score)

            # Move to the next element
            i += 1

        # Return the highest score found
        return answer


#4. Median of Two Sorted Arrays
```python
class Solution:
    def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
        # Step 1: Ensure nums1 is the smaller array
        n1, n2 = len(nums1), len(nums2)
        # Swap arrays if nums1 is larger to ensure efficiency in binary search
        if n1 > n2:
            return self.findMedianSortedArrays(nums2, nums1)

        # Step 2: Initialize variables for binary search
        low, high = 0, n1  # Binary search range in nums1
        left = (n1 + n2 + 1) // 2  # The total elements on the left side of the split
        total = n1 + n2  # Total number of elements in nums1 and nums2

        # Step 3: Perform Binary Search
        while low <= high:
            # Calculate partitions for nums1 and nums2
            mid1 = (low + high) // 2  # Partition index in nums1
            mid2 = left - mid1  # Partition index in nums2

            # Assign boundaries for nums1 and nums2
            l1 = nums1[mid1 - 1] if mid1 > 0 else float('-inf')  # Left of partition in nums1
            l2 = nums2[mid2 - 1] if mid2 > 0 else float('-inf')  # Left of partition in nums2
            r1 = nums1[mid1] if mid1 < n1 else float('inf')  # Right of partition in nums1
            r2 = nums2[mid2] if mid2 < n2 else float('inf')  # Right of partition in nums2

            # Step 3a: Check if the partitions are correct
            if l1 <= r2 and l2 <= r1:
                # If total length is odd, the median is the max of left partitions
                if total % 2 == 1:
                    return max(l1, l2)
                # If total length is even, the median is the average of max left and min right
                else:
                    return (max(l1, l2) + min(r1, r2)) / 2

            # Step 3b: Adjust partitions if not correct
            elif l1 > r2:
                # Move left in nums1
                high = mid1 - 1
            else:
                # Move right in nums1
                low = mid1 + 1

        # Step 4: Return 0 (should never reach here)
        return 0  # This line is unreachable
```

---

 **Summary of Key Concepts**

1. **Binary Search on the Smaller Array**:
   - To minimize time complexity, we always perform binary search on the smaller array (`nums1`).
   - If `nums1` is larger, we swap the inputs.

2. **Core Idea**:
   - **Divide both arrays into two halves** such that:
     - All elements in the left half are ≤ all elements in the right half.
   - Use binary search to find the correct partition index.

3. **Partitioning and Median**:
   - If the total number of elements is odd, the median is the maximum element of the left half.
   - If even, the median is the average of the maximum of the left half and the minimum of the right half.

4. **Efficiency**:
   - **Time Complexity:** `O(log(min(n1, n2)))` because we perform binary search on the smaller array.
   - **Space Complexity:** `O(1)` as no extra space is used.

-